# Generador de datos sintéticos para eCommerce de moda

Este proyecto construye un **dataset sintético, reproducible y coherente** de un eCommerce de moda con canal **online** y **físico**, orientado a análisis de negocio, visualización (BI) y modelado predictivo (especialmente devoluciones).

El pipeline genera datos a distintos niveles de granularidad (productos, clientes, tickets, ítems) y aplica reglas explícitas de calendario, mix de categorías, promociones, comportamiento del cliente y logística de devoluciones.

El objetivo no es maximizar aleatoriedad, sino **realismo estructural**: que las relaciones entre variables tengan sentido económico, temporal y operativo.

---

## Principios generales del diseño

Antes de detallar las tablas, el generador sigue tres principios clave:

### 1) Granularidad clara
- **Productos**: catálogo maestro y variantes (SKU).
- **Clientes**: una fila por cliente.
- **Tickets**: una fila por pedido.
- **Ítems**: una fila por producto vendido.
- **Devoluciones**: eventos logísticos derivados de ítems devueltos.

Cada tabla tiene una responsabilidad clara y no mezcla niveles.

### 2) Reproducibilidad
- Se usan semillas fijas y generadores deterministas basados en claves (`customer_id`, `id_producto`) cuando es necesario.
- Ejecutar el pipeline dos veces produce exactamente los mismos resultados.

### 3) Invariantes y coherencia
- Los totales agregados se respetan en todas las desagregaciones.
- No se venden productos fuera de su ventana de vida.
- No se crean devoluciones sin venta previa.
- Las reglas nunca alteran el número total de filas, solo redistribuyen atributos.

---

## 1. Productos

### 1.1 Tabla `productos`

**Granularidad:**  
Una fila por producto (`id_producto`).

**Descripción:**  
Catálogo maestro de productos, usado principalmente para definir el **ciclo de vida temporal** de cada producto.

**Campos principales:**
- `id_producto`
- `categoria`
- `lanzamiento` (YYYY-MM)
- `retiro` (YYYY-MM o nulo)

**Regla clave (calendario de producto):**  
Un producto solo puede aparecer en ventas si la fecha del ítem está comprendida entre su `lanzamiento` y su `retiro` (si existe).  
Esta regla se aplica de forma estricta en todos los ajustes posteriores (mix, cooc, promociones).

---

### 1.2 Tabla `productos_variantes`

**Granularidad:**  
Una fila por SKU.

**Descripción:**  
Catálogo de variantes de producto (talla, color, precio y coste).

**Campos principales:**
- `sku`
- `id_producto`
- `categoria`
- `talla`
- `color`
- `precio`
- `coste_bruto`
- `lanzamiento`, `retiro` (si aplica a nivel SKU)

**Uso principal:**  
Asignación determinista de SKU a cada ítem vendido, garantizando:
- coherencia temporal
- estabilidad entre ejecuciones
- variedad realista dentro del catálogo activo

---

## 2. Promociones

### 2.1 Tabla `promociones`

**Granularidad:**  
Una fila por combinación `(evento, categoría)`.

**Descripción:**  
Calendario anual de promociones con descuentos por categoría.

**Eventos incluidos:**
- Rebajas de enero
- Rebajas de julio
- Black Friday

**Campos principales:**
- `promotion_id`
- `nombre`
- `tipo` (rebajas / black_friday)
- `fecha_inicio`, `fecha_fin`
- `categoria`
- `descuento_pct`
- `prioridad`

---

### 2.2 Reglas de asignación de descuentos

Las promociones no asignan descuentos aleatorios. Se utilizan **buckets de intensidad**:

- `strong`
- `medium`
- `light`
- `none`

Cada evento reparte las categorías entre buckets con una estructura fija, garantizando:
- pocas categorías muy agresivas
- la mayoría con descuentos moderados
- alguna categoría sin promoción

#### Categorías estacionales obligatorias
En rebajas (enero y julio) existen categorías que **nunca pueden quedar fuera** (por ejemplo, abrigos en enero o camisetas en julio).  
Si caen en `none`, se reasignan como mínimo a `light`.

#### Empuje adicional
Las categorías estacionales reciben un incremento adicional sobre el descuento base, respetando un tope máximo por evento.

---

## 3. Clientes

### 3.1 Tabla `clientes`

**Granularidad:**  
Una fila por cliente (`customer_id`).

**Descripción:**  
Base de clientes con información agregada de actividad.

**Campos principales:**
- `customer_id`
- `provincia`, `comunidad`
- `fecha_primer_compra`
- `fecha_ultima_compra`
- `n_pedidos`
- `n_items_comprados`

---

### 3.2 Reglas de comportamiento del cliente

- La primera compra siempre cae dentro del mes de alta.
- Las compras posteriores se distribuyen en el tiempo usando funciones suaves (no uniformes).
- Se evita la concentración artificial de compras al final del periodo.
- Se introduce estacionalidad suave en la recurrencia.

---

## 4. Tickets

### 4.1 Tabla `tickets_online`

**Granularidad:**  
Una fila por ticket online.

**Descripción:**  
Desagregación de la actividad agregada del cliente en pedidos individuales.

**Invariantes:**
- Número de tickets ≤ `n_pedidos`.
- Suma de `n_items` = `n_items_comprados`.
- Provincia constante por cliente.

---

### 4.2 Tabla `tiendas`

Catálogo de tiendas físicas con:
- `store_id`
- `provincia`
- `fecha_apertura`

Se utiliza para:
- asignar tickets físicos
- modelar ramp-up tras apertura

---

### 4.3 Tabla `tickets_fisico`

**Granularidad:**  
Una fila por ticket físico.

**Reglas principales:**
- Crecimiento progresivo tras apertura de tiendas.
- Reparto anual objetivo por tienda.
- Distribución mensual con estacionalidad.
- Sesgo por día de la semana (más tickets en fin de semana).
- Número de ítems por ticket basado en UPT realista.

Los tickets físicos no identifican cliente.

---

### 4.4 Tabla `tickets_total`

Unión de tickets online y físicos con esquema homogéneo.  
Es la tabla base para expandir a nivel ítem.

---

## 5. Ítems de venta

### 5.1 Tabla `items_venta`

**Granularidad:**  
Una fila por ítem vendido.

**Pipeline de construcción:**
1. Expansión de tickets a ítems.
2. Asignación de `store_id` solo en canal físico.
3. Asignación determinista de SKU activo en la fecha.
4. Aplicación de promociones por fecha y categoría.
5. Cálculo económico (precio neto, coste, margen).
6. Validaciones de coherencia.

Esta tabla es el **núcleo del proyecto**.

---

## 6. Reglas de categorías (MIX)

### 6.1 Ajuste de mix por contexto

El mix de categorías se ajusta por:
- mes
- canal
- zona geográfica

Ejemplos:
- Verano: más camisetas, menos abrigos.
- Invierno: más abrigos y calzado.
- Rebajas: refuerzo de categorías estacionales.

Los ajustes:
- tienen límites máximos
- se realizan mediante swaps
- no alteran totales globales

---

### 6.2 Reglas de co-ocurrencia (cross-sell)

Se introducen combinaciones realistas dentro del ticket:
- pantalón → cinturón
- calzado → calcetines
- abrigo → bufanda

Las reglas operan por bloques temporales y respetan:
- calendario de producto
- número total de ítems
- límites de swaps

---

## 7. Tallas, colores y físico

### 7.1 Tallas
- Una talla coherente por cliente.
- Ropa: XS–XL.
- Calzado: 39–45.
- Accesorios: OneSize.
- El SKU se actualiza para reflejar la talla.

### 7.2 Colores
- Asignación condicionada por:
  - zona
  - temporada
  - categoría
- Uso de tipos de color (oscuro, vivo, neutro).

### 7.3 Perfil físico del cliente
Se genera un perfil estable por cliente:
- altura
- peso
- BMI

Coherente con su talla dominante, con un pequeño porcentaje de mismatch controlado.

---

## 8. Devoluciones

### 8.1 Tabla `items_6`

Contiene todos los ítems con una variable binaria `devuelto`.

La probabilidad de devolución depende de:
- canal (gap online vs físico)
- distancia a talla ideal
- fit del producto (solo ropa)
- categoría
- descuento y promoción
- precio
- color
- estacionalidad
- BMI
- historial del cliente
- sesgo individual del cliente
- sesgo del producto

---

### 8.2 Tabla `devoluciones`

**Granularidad:**  
Una fila por ítem devuelto.

**Campos añadidos:**
- `zona_logistica`
- `dias_hasta_devolucion`
- `fecha_devolucion`
- `coste_devolucion`

**Reglas clave:**
- Distribución distinta de días según canal.
- Coste compuesto por:
  - base por categoría
  - recargo por zona
  - ajuste por canal físico
  - ruido controlado

---

## 9. Resultado final

El pipeline produce un ecosistema completo de tablas interrelacionadas, listas para:
- análisis exploratorio
- dashboards de negocio
- modelado predictivo
- simulaciones de impacto (promos, devoluciones, logística)

Todas las reglas están explícitas, documentadas y controladas, priorizando coherencia y trazabilidad frente a aleatoriedad pura.


# Librerías

In [25]:
from __future__ import annotations

# Librerías estándar de Python
import os
import math
import random

import hashlib
import calendar
import sqlite3
import unicodedata
from datetime import date, datetime
from pathlib import Path
from typing import Dict, Iterable, List, Tuple, Optional

# Librerías de análisis de datos
import numpy as np
import pandas as pd

# Módulos propios del proyecto
from edades import build_month_samplers, sample_age_from_weights
from growth_curve import example_config, build_monthly_new_customers
from geografia import asignar_provincia


# Productos 

In [9]:
# Catálogo de productos
# - Dataset maestro de referencias (SKUs) para la tienda simulada.
# - Incluye ventana de vida comercial: lanzamiento (YYYY-MM) y retiro (YYYY-MM o None si sigue activo).
# - Se exporta a CSV y se replica en SQLite para consultas y generación de tablas derivadas.

columnas = ["id_producto", "nombre", "categoria", "lanzamiento", "retiro", "precio", "coste bruto"]

valores = [
    # 2017 (base de catálogo: referencias estables)
    ["P001","Essential Tee","Camiseta","2017-08",None,25,10],
    ["P002","Classic Tee","Camiseta","2017-08",None,26,10],
    ["P003","Core Hoodie","Sudadera","2017-08",None,60,25],
    ["P004","Basic Sweatpants","Pantalón","2017-08",None,50,22],
    ["P005","Heritage Coat","Abrigo","2017-08",None,95,45],

    # 2018 (introducción de drops estacionales SS/FW, manteniendo al menos un “clásico” por temporada)
    ["P006","Summer Tee","Camiseta","2018-03",None,27,11],
    ["P007","Light Chinos","Pantalón","2018-03","2018-08",60,26],
    ["P008","Thermal Pants","Pantalón","2018-09",None,62,27],
    ["P009","Quilted Jacket","Abrigo","2018-09","2019-02",95,46],

    # 2019 (más variedad y primeras referencias de accesorios recurrentes)
    ["P010","Summer Cap","Gorra","2019-03",None,21,8],
    ["P011","Logo Tee","Camiseta","2019-03","2019-08",27,11],
    ["P012","Coastal Shirt","Camisa","2019-03","2019-08",45,18],
    ["P013","Urban Parka","Abrigo","2019-09",None,100,50],
    ["P014","Puffer Hoodie","Sudadera","2019-09","2020-02",66,28],
    ["P015","Work Trousers","Pantalón","2019-09","2020-02",66,29],

    # 2020 (se incorpora calzado y se amplían las colecciones por temporada)
    ["P016","Oversized Tee","Camiseta","2020-03",None,28,12],
    ["P017","Track Joggers","Pantalón","2020-03","2020-08",55,24],
    ["P018","Breeze Shirt","Camisa","2020-03","2020-08",46,19],
    ["P019","Air Sneakers","Calzado","2020-03","2020-08",85,42],
    ["P020","Heritage Coat 2","Abrigo","2020-09",None,110,55],
    ["P021","Fleece Hoodie","Sudadera","2020-09","2021-02",68,29],
    ["P022","Winter Boots","Calzado","2020-09","2021-02",108,52],
    ["P023","Flannel Shirt","Camisa","2020-09","2021-02",52,22],

    # 2021 (entrada de básicos de complementos y continuidad de líneas core)
    ["P024","Coastal Shorts","Pantalón","2021-03",None,45,20],
    ["P025","Retro Tee","Camiseta","2021-03","2021-08",29,12],
    ["P026","Linen Shirt","Camisa","2021-03","2021-08",48,19],
    ["P027","Street Sneakers","Calzado","2021-03","2021-08",88,44],
    ["P028","Leather Belt","Cinturón","2021-09",None,30,12],
    ["P029","Down Jacket","Abrigo","2021-09","2022-02",125,60],
    ["P030","Storm Hoodie","Sudadera","2021-09","2022-02",70,30],
    ["P031","Thermal Trousers","Pantalón","2021-09","2022-02",70,32],

    # 2022 (incluye calcetines como básico recurrente y bufanda como accesorio de invierno)
    ["P032","Everyday Socks","Calcetines","2022-03",None,12,5],
    ["P033","Performance Tee","Camiseta","2022-03","2022-08",30,13],
    ["P034","Breeze Pants","Pantalón","2022-03","2022-08",50,22],
    ["P035","Fresh Linen Shirt","Camisa","2022-03","2022-08",50,20],
    ["P036","Summer Slides","Calzado","2022-03","2022-08",60,28],
    ["P037","Wool Scarf","Bufanda","2022-09",None,25,10],
    ["P038","Varsity Jacket","Abrigo","2022-09","2023-02",100,50],
    ["P039","Puffer Hoodie 22","Sudadera","2022-09","2023-02",70,30],
    ["P040","Trail Boots","Calzado","2022-09","2023-02",112,55],
    ["P041","Corduroy Shirt","Camisa","2022-09","2023-02",55,22],

    # 2023 (mantiene la rotación estacional; aparece una camiseta “invernal” como excepción de catálogo)
    ["P042","Essential Polo","Camisa","2023-03",None,46,19],
    ["P043","Graphic Tee","Camiseta","2023-03","2023-08",29,12],
    ["P044","Coastal Chinos","Pantalón","2023-03","2023-08",52,23],
    ["P045","Air Mesh Sneakers","Calzado","2023-03","2023-08",87,42],
    ["P046","Summer Cap 23","Gorra","2023-03","2023-08",23,8],
    ["P047","Heritage Parka","Abrigo","2023-09",None,115,58],
    ["P048","Heavy Hoodie","Sudadera","2023-09","2024-02",72,31],
    ["P049","Smart Trousers","Pantalón","2023-09","2024-02",72,33],
    ["P050","Mountain Boots","Calzado","2023-09","2024-02",115,55],
    ["P051","Winter Tee","Camiseta","2023-09","2024-02",30,13],

    # 2024 (catálogo más amplio; se mantiene coherencia de costes/precios por familia)
    ["P052","Everyday Belt","Cinturón","2024-03",None,28,12],
    ["P053","Art Tee","Camiseta","2024-03","2024-08",29,12],
    ["P054","Breeze Shorts","Pantalón","2024-03","2024-08",48,21],
    ["P055","Fresh Linen","Camisa","2024-03","2024-08",50,20],
    ["P056","Urban Sneakers","Calzado","2024-03","2024-08",90,45],
    ["P057","Summer Cap 24","Gorra","2024-03","2024-08",24,9],
    ["P058","Timber Coat","Abrigo","2024-09",None,120,60],
    ["P059","Puffer Pro","Abrigo","2024-09","2025-02",130,65],
    ["P060","Thermal Hoodie","Sudadera","2024-09","2025-02",70,30],
    ["P061","Rugged Boots","Calzado","2024-09","2025-02",118,56],
    ["P062","Heavy Flannel","Camisa","2024-09","2025-02",56,22],
    ["P063","Winter Tee 24","Camiseta","2024-09","2025-02",30,13],

    # 2025 (colección SS sin bloque FW en este dataset)
    ["P064","Signature Tee","Camiseta","2025-03",None,30,13],
    ["P065","Coastal Pants 25","Pantalón","2025-03","2025-08",52,23],
    ["P066","Linen Shirt 25","Camisa","2025-03","2025-08",51,21],
    ["P067","Trail Runners","Calzado","2025-03","2025-08",92,46],
    ["P068","Summer Cap 25","Gorra","2025-03","2025-08",24,9],
    ["P069","Breeze Vest","Abrigo","2025-03","2025-08",82,38],
]

# Construcción del DataFrame 
productos_df = pd.DataFrame(valores, columns=columnas)

# Salida visible en el notebook 
display(productos_df.head(10))
display(pd.DataFrame({
    "n_filas": [len(productos_df)],
    "n_productos_unicos": [productos_df["id_producto"].nunique()],
    "rango_lanzamiento": [f'{productos_df["lanzamiento"].min()} → {productos_df["lanzamiento"].max()}']
}))

# Persistencia: CSV + SQLite 
productos_df.to_csv("data/productos.csv", index=False)

con = sqlite3.connect("database/mi_base.db")
productos_df.to_sql("productos", con, if_exists="replace", index=False)
con.close()


,id_producto,nombre,categoria,lanzamiento,retiro,precio,coste bruto
0,P001,Essential Tee,Camiseta,2017-08,None,25,10
1,P002,Classic Tee,Camiseta,2017-08,None,26,10
2,P003,Core Hoodie,Sudadera,2017-08,None,60,25
3,P004,Basic Sweatpants,Pantalón,2017-08,None,50,22
4,P005,Heritage Coat,Abrigo,2017-08,None,95,45
5,P006,Summer Tee,Camiseta,2018-03,None,27,11
6,P007,Light Chinos,Pantalón,2018-03,2018-08,60,26
7,P008,Thermal Pants,Pantalón,2018-09,None,62,27
8,P009,Quilted Jacket,Abrigo,2018-09,2019-02,95,46
9,P010,Summer Cap,Gorra,2019-03,None,21,8


,n_filas,n_productos_unicos,rango_lanzamiento
0,69,69,2017-08 → 2025-03


# Variantes de productos

In [10]:
# Variantes de producto (SKU): expansión de catálogo por color y talla
# - Parte del catálogo maestro `productos_df` y genera SKUs vendibles.
# - Cada SKU combina: id_producto + color + talla.
# - La selección de colores intenta ser coherente por categoría y temporada, introduciendo un pequeño % de “mismatch”
#   para simular decisiones de producto/stock menos perfectas.
# - Output: CSV + tabla SQLite para usarlo como dimensión en modelos/consultas.

# --- 1) Carga y normalización mínima del catálogo maestro ---
if "productos_df" not in globals():
    productos_df = pd.read_csv("productos.csv", dtype=str)
    for col in ["precio", "coste bruto"]:
        productos_df[col] = pd.to_numeric(productos_df[col], errors="coerce")

productos_df["lanzamiento"] = productos_df["lanzamiento"].astype(str)
productos_df["retiro"] = productos_df["retiro"].replace({"None": pd.NA})
productos_df.loc[productos_df["retiro"].isna(), "retiro"] = pd.NA


# --- 2) Reglas de tallas por categoría ---
tallas_por_categoria = {
    "Camiseta": ["XS","S","M","L","XL"],
    "Sudadera": ["XS","S","M","L","XL"],
    "Pantalón": ["XS","S","M","L","XL"],
    "Abrigo":   ["XS","S","M","L","XL"],
    "Calzado":  [str(n) for n in range(39, 46)],
    "Camisa":   ["XS","S","M","L","XL"],
    "Gorra":    ["OneSize"],
    "Cinturón": ["OneSize"],
    "Calcetines":["OneSize"],
    "Bufanda":  ["OneSize"],
}

# --- 3) Catálogo de colores y tipología ---
COLORS = {
    "BLK": "Negro",
    "GRY": "Gris",
    "NAV": "Azul marino",
    "RED": "Rojo",
    "LME": "Verde lima",
    "WHT": "Blanco",
    "BEI": "Beige",
    "BRN": "Marrón",
    "BLU": "Azul",
}

COLOR_TIPO = {
    "BLK": "oscuro",
    "GRY": "oscuro",
    "NAV": "oscuro",
    "BRN": "oscuro",
    "RED": "vivo",
    "LME": "vivo",
    "BLU": "vivo",
    "WHT": "neutro",
    "BEI": "neutro",
}

# --- 4) Lógica de temporada y coherencia por categoría ---
SEASON_FAV = {
    "SS": {"WHT","LME","RED","BLU","BEI"},
    "FW": {"BLK","GRY","NAV","BRN","BEI"},
    "ALL": set(COLORS.keys()),
}
SEASON_BAD = {
    "SS": {"BLK","GRY","BRN"},
    "FW": {"LME","RED","BLU","WHT"},
    "ALL": set(),
}

CAT_NATURAL = {
    "Camiseta": {"WHT","RED","LME","BLU","NAV","BEI"},
    "Sudadera": {"BLK","GRY","NAV","RED","BRN","BEI"},
    "Pantalón": {"BLK","GRY","BEI","NAV","BRN"},
    "Abrigo":   {"BLK","GRY","NAV","BRN","BEI"},
    "Camisa":   {"WHT","BLU","BEI","GRY","NAV"},
    "Calzado":  {"BLK","NAV","BRN","WHT","BEI"},
    "Gorra":    {"WHT","RED","BEI","NAV","BLK"},
    "Cinturón": {"BRN","BLK","BEI","NAV"},
    "Bufanda":  {"BRN","GRY","BEI","RED"},
    "Calcetines":{"WHT","GRY","BLK","RED","LME"},
}

CAT_CONFLICT = {
    "Camiseta": {"BLK","GRY","BRN"},
    "Sudadera": {"LME","WHT"},
    "Pantalón": {"LME","RED"},
    "Abrigo":   {"LME","RED"},
    "Camisa":   {"LME","RED","BRN"},
    "Calzado":  {"LME","RED"},
    "Gorra":    {"GRY","BRN"},
    "Cinturón": {"LME","RED"},
    "Bufanda":  {"LME"},
    "Calcetines":{"BRN","NAV"},
}

RNG = np.random.default_rng(42)
MISMATCH_RATE = {"SS": 0.15, "FW": 0.10, "ALL": 0.05}
CLASSIC_UP_TO_5_PROB = 0.35


def parse_season(lanzamiento: str, retiro) -> str:
    """
    Determina la temporada del producto.
    - Si el producto no tiene retiro (clásico), se etiqueta como ALL.
    - Si tiene ventana de temporada, se usa el mes de lanzamiento para SS (3-8) o FW (resto).
    """
    if pd.isna(retiro):
        return "ALL"
    try:
        month = int(str(lanzamiento)[5:7])
    except Exception:
        month = 3
    return "SS" if 3 <= month <= 8 else "FW"


def choose_n_colors(is_classic: bool) -> int:
    """
    Define cuántos colores tendrá un producto.
    - Clásicos: 4 colores normalmente, con probabilidad de llegar a 5.
    - Estacionales: 2-4, con más masa en 3 (para simular drops no excesivos).
    """
    if is_classic:
        return 5 if RNG.random() < CLASSIC_UP_TO_5_PROB else 4
    return int(RNG.choice([2, 3, 3, 3, 4]))


def pick_colors(categoria: str, season: str, n: int):
    """
    Selección de colores por producto:
    - Prioriza colores “naturales” de la categoría.
    - Para SS/FW, intenta alinearse con la paleta preferida de la temporada.
    - Introduce un porcentaje controlado de colores menos coherentes (mismatch) para realismo.
    """
    natural = CAT_NATURAL.get(categoria, set(COLORS.keys())).copy()
    conflict = CAT_CONFLICT.get(categoria, set()).copy()

    fav = SEASON_FAV[season]
    bad = SEASON_BAD[season]

    if season == "ALL":
        coherent_pool = natural
    else:
        coherent_pool = (natural & fav) if (natural & fav) else natural

    mismatch_pool = (bad | conflict) - coherent_pool
    mismatch_pool = mismatch_pool & set(COLORS.keys())

    k_bad = int(round(MISMATCH_RATE[season] * n))
    if season != "ALL":
        k_bad = min(max(k_bad, 1 if n >= 3 else 0), n - 1)
    else:
        k_bad = min(k_bad, max(0, n - 3))

    k_good = n - k_bad
    chosen = []

    good_pool = list(coherent_pool)
    RNG.shuffle(good_pool)
    chosen += good_pool[:min(k_good, len(good_pool))]

    remaining = n - len(chosen)
    if remaining > 0:
        bad_pool = list((mismatch_pool - set(chosen)) or (set(COLORS.keys()) - set(chosen)))
        RNG.shuffle(bad_pool)
        chosen += bad_pool[:remaining]

    return chosen[:n]


# --- 5) Expansión a nivel SKU ---
rows = []

for _, r in productos_df.iterrows():
    cat = r["categoria"]
    is_classic = pd.isna(r["retiro"])
    season = parse_season(r["lanzamiento"], r["retiro"])

    n_colors = choose_n_colors(is_classic)
    n_colors = min(n_colors, 5 if is_classic else 4)

    color_codes = pick_colors(cat, season, n_colors)
    tallas = tallas_por_categoria.get(cat, ["OneSize"])

    for ccode in color_codes:
        cname = COLORS[ccode]
        tipo = COLOR_TIPO.get(ccode, "neutro")

        for talla in tallas:
            sku = f"{r['id_producto']}-{ccode}-{talla}"
            rows.append({
                "sku": sku,
                "id_producto": r["id_producto"],
                "nombre": r["nombre"],
                "categoria": cat,
                "color_codigo": ccode,
                "color": cname,
                "tipo_color": tipo,
                "talla": talla,
                "lanzamiento": r["lanzamiento"],
                "retiro": r["retiro"] if not pd.isna(r["retiro"]) else None,
                "precio": float(r["precio"]) if pd.notna(r["precio"]) else None,
                "coste bruto": float(r["coste bruto"]) if pd.notna(r["coste bruto"]) else None,
            })

variants_df = pd.DataFrame(
    rows,
    columns=[
        "sku","id_producto","nombre","categoria","color_codigo","color","tipo_color",
        "talla","lanzamiento","retiro","precio","coste bruto",
    ],
)

# --- 6) Exportación y persistencia ---
Path("data").mkdir(parents=True, exist_ok=True)
variants_df.to_csv("data/productos_variantes.csv", index=False)

print(f"[OK] Variantes creadas: {len(variants_df):,} filas → data/productos_variantes.csv")
print("Columnas:", list(variants_df.columns))
display(variants_df.head(10))

con = sqlite3.connect("database/mi_base.db")
variants_df.to_sql("productos_variantes", con, if_exists="replace", index=False)
con.close()


[OK] Variantes creadas: 1,120 filas → data/productos_variantes.csv
Columnas: ['sku', 'id_producto', 'nombre', 'categoria', 'color_codigo', 'color', 'tipo_color', 'talla', 'lanzamiento', 'retiro', 'precio', 'coste bruto']


,sku,id_producto,nombre,categoria,color_codigo,color,tipo_color,talla,lanzamiento,retiro,precio,coste bruto
0,P001-RED-XS,P001,Essential Tee,Camiseta,RED,Rojo,vivo,XS,2017-08,None,25.0,10.0
1,P001-RED-S,P001,Essential Tee,Camiseta,RED,Rojo,vivo,S,2017-08,None,25.0,10.0
2,P001-RED-M,P001,Essential Tee,Camiseta,RED,Rojo,vivo,M,2017-08,None,25.0,10.0
3,P001-RED-L,P001,Essential Tee,Camiseta,RED,Rojo,vivo,L,2017-08,None,25.0,10.0
4,P001-RED-XL,P001,Essential Tee,Camiseta,RED,Rojo,vivo,XL,2017-08,None,25.0,10.0
5,P001-NAV-XS,P001,Essential Tee,Camiseta,NAV,Azul marino,oscuro,XS,2017-08,None,25.0,10.0
6,P001-NAV-S,P001,Essential Tee,Camiseta,NAV,Azul marino,oscuro,S,2017-08,None,25.0,10.0
7,P001-NAV-M,P001,Essential Tee,Camiseta,NAV,Azul marino,oscuro,M,2017-08,None,25.0,10.0
8,P001-NAV-L,P001,Essential Tee,Camiseta,NAV,Azul marino,oscuro,L,2017-08,None,25.0,10.0
9,P001-NAV-XL,P001,Essential Tee,Camiseta,NAV,Azul marino,oscuro,XL,2017-08,None,25.0,10.0


# Promociones

In [8]:
# Promociones (calendario + descuento por categoría)
# - Genera un plan anual de eventos promocionales entre YEAR_START y YEAR_END.
# - Cada evento aplica un descuento por categoría, con lógica de “buckets” (strong/medium/light/none).
# - En rebajas (ene/jul) se fuerzan categorías “idóneas” para que nunca queden sin descuento.
# - Salida: CSV (data/promociones.csv) + tabla SQLite (promociones).

# -----------------------------
# Configuración global
# -----------------------------
YEAR_START = 2017
YEAR_END   = 2025

PROMO_TYPES = ("rebajas_ene", "rebajas_jul", "black_friday")


def window_rebajas_ene(year: int):
    """Rebajas de invierno: 7 enero – 15 febrero."""
    return date(year, 1, 7), date(year, 2, 15)

def window_rebajas_jul(year: int):
    """Rebajas de verano: 1 julio – 15 agosto."""
    return date(year, 7, 1), date(year, 8, 15)

def window_black_friday(year: int):
    """Black Week / Black Friday: 24 – 30 noviembre."""
    return date(year, 11, 24), date(year, 11, 30)


WINDOW_BY_TYPE = {
    "rebajas_ene":  window_rebajas_ene,
    "rebajas_jul":  window_rebajas_jul,
    "black_friday": window_black_friday,
}

# Prioridad: 1 = se impone si coincide en fechas con otra promo
PRIORITY_BY_TYPE = {
    "black_friday": 1,
    "rebajas_ene":  2,
    "rebajas_jul":  2,
}

# Abreviaturas para IDs consistentes
PTYPE_ABBR = {
    "rebajas_ene":  "ENE",
    "rebajas_jul":  "JUL",
    "black_friday": "BF",
}

def pretty_name(ptype: str, year: int) -> str:
    """Nombre legible del evento promocional."""
    if ptype == "rebajas_ene":
        return f"Rebajas Invierno {year}"
    if ptype == "rebajas_jul":
        return f"Rebajas Verano {year}"
    if ptype == "black_friday":
        return f"Black Friday {year}"
    return f"Promo {ptype} {year}"

def tipo_from_ptype(ptype: str) -> str:
    """Normaliza el tipo para reporting/joins."""
    if "rebajas" in ptype:
        return "rebajas"
    if ptype == "black_friday":
        return "black_friday"
    return ptype


# Categorías “idóneas” por temporada (siempre participan y nunca quedan en 0%)
CATS_FIJAS = {
    "rebajas_ene": {"Abrigo", "Sudadera", "Pantalón", "Bufanda", "Camisa"},
    "rebajas_jul": {"Camiseta", "Camisa", "Pantalón", "Calzado", "Gorra"},
    "black_friday": set(),
}

# Código por categoría para IDs únicos por año+evento+categoría
CATEGORY_CODE = {
    "Abrigo":      "ABR",
    "Bufanda":     "BUF",
    "Calcetines":  "CLC",
    "Calzado":     "CLZ",
    "Camisa":      "CAM",
    "Camiseta":    "CMT",
    "Cinturón":    "CIN",
    "Gorra":       "GOR",
    "Pantalón":    "PAN",
    "Sudadera":    "SUD",
}

# Descuentos “redondos” (porcentaje), por tipo y bucket
DISCOUNT_CHOICES = {
    "rebajas_ene": {
        "strong": [25, 30, 35, 40],
        "medium": [15, 20, 25],
        "light":  [5, 10, 15],
        "none":   [0],
    },
    "rebajas_jul": {
        "strong": [20, 25, 30, 35],
        "medium": [15, 20, 25],
        "light":  [5, 10, 15],
        "none":   [0],
    },
    "black_friday": {
        "strong": [25, 30, 35],
        "medium": [15, 20, 25],
        "light":  [5, 10, 15],
        "none":   [0, 5],
    },
}

MAX_PCT = {
    "rebajas_ene": 40,
    "rebajas_jul": 35,
    "black_friday": 35,
}

# Reparto del número de categorías por bucket (por evento)
REPARTO_DEFAULT = {
    "rebajas_ene":  {"strong": 2, "medium": 2, "light": "resto-1", "none": 1},
    "rebajas_jul":  {"strong": 2, "medium": 2, "light": "resto-1", "none": 1},
    "black_friday": {"strong": 3, "medium": "resto-1", "light": 1, "none": 1},
}



# Helpers de asignación

def assign_buckets_for_event(categories, ptype, rng):
    """
    Asigna a cada categoría un bucket (strong/medium/light/none) según REPARTO_DEFAULT.
    Ajuste adicional: las categorías “fijas” del evento nunca pueden quedar en 'none'.
    """
    cats = categories.copy()
    rng.shuffle(cats)

    rep = REPARTO_DEFAULT[ptype]
    n = len(cats)

    fixed_count = sum(v for v in rep.values() if isinstance(v, int))
    resto_menos = 1 if any(v == "resto-1" for v in rep.values()) else 0
    resto = max(0, n - fixed_count - resto_menos)

    counts = {}
    for bucket, v in rep.items():
        counts[bucket] = resto if v == "resto-1" else v

    order = ["strong", "medium", "light", "none"]
    idx = 0
    bucket_of_cat = {}

    for bucket in order:
        c = counts.get(bucket, 0)
        for _ in range(c):
            if idx < n:
                bucket_of_cat[cats[idx]] = bucket
                idx += 1

    # Si por cualquier razón sobran categorías, se integran como 'light'
    while idx < n:
        bucket_of_cat[cats[idx]] = "light"
        idx += 1

    # Categorías fijas: nunca 'none'
    fixed_cats = CATS_FIJAS.get(ptype, set())
    for cat in fixed_cats:
        if cat in bucket_of_cat and bucket_of_cat[cat] == "none":
            bucket_of_cat[cat] = "light"

    return bucket_of_cat


def pick_discount(ptype, bucket, rng, is_fixed):
    """
    Selecciona un descuento (en decimal) desde DISCOUNT_CHOICES y aplica tope MAX_PCT.
    Si la categoría es fija del evento y el descuento es > 0, se empuja +5pp (sin superar el máximo).
    """
    choices = DISCOUNT_CHOICES[ptype][bucket]
    pct_int = int(rng.choice(choices))
    max_pct = MAX_PCT[ptype]

    if is_fixed and pct_int > 0:
        pct_int = min(pct_int + 5, max_pct)

    pct_int = min(pct_int, max_pct)
    return pct_int / 100.0



# Construcción principal

def build_promociones(productos_df: pd.DataFrame) -> pd.DataFrame:
    """
    Genera el calendario de promociones por año, tipo y categoría.

    Output:
      - promotion_id único por (año, evento, categoría)
      - descuento_pct en decimal (0.25 = 25%)
    """
    # Compatibilidad: si el catálogo trae 'categoría' (tilde) o 'categoria' (sin tilde)
    col_cat = "categoria" if "categoria" in productos_df.columns else "categoría"

    categories = sorted(productos_df[col_cat].dropna().unique().tolist())

    rng_global = np.random.RandomState(42)
    rows = []

    for year in range(YEAR_START, YEAR_END + 1):
        for ptype in PROMO_TYPES:
            start_date, end_date = WINDOW_BY_TYPE[ptype](year)

            nombre    = pretty_name(ptype, year)
            tipo      = tipo_from_ptype(ptype)
            prioridad = PRIORITY_BY_TYPE[ptype]
            abbr      = PTYPE_ABBR[ptype]

            # RNG específico por año+tipo (mantiene tu criterio de reproducibilidad)
            rng = np.random.RandomState(
                rng_global.randint(0, 10_000_000) + year + (hash(ptype) % 1000)
            )

            bucket_of_cat = assign_buckets_for_event(categories, ptype, rng)
            fixed_cats = CATS_FIJAS.get(ptype, set())

            for cat in categories:
                bucket    = bucket_of_cat.get(cat, "light")
                is_fixed  = cat in fixed_cats
                descuento = pick_discount(ptype, bucket, rng, is_fixed)

                code = CATEGORY_CODE.get(cat, cat[:3].upper())
                promotion_id = f"{year}{abbr}-{code}"

                rows.append({
                    "promotion_id":  promotion_id,
                    "nombre":        nombre,
                    "tipo":          tipo,
                    "fecha_inicio":  start_date.isoformat(),
                    "fecha_fin":     end_date.isoformat(),
                    "categoria":     cat,          # sin tilde (estándar para SQL/CSV)
                    "descuento_pct": descuento,
                    "prioridad":     prioridad,
                })

    df_promos = (
        pd.DataFrame(rows)
        .sort_values(["fecha_inicio", "tipo", "categoria"])
        .reset_index(drop=True)
    )
    return df_promos



# Ejecución (usa el catálogo ya cargado en el notebook)

base_df = productos_df if "productos_df" in globals() else productos_df

df_promociones = build_promociones(base_df)

Path("data").mkdir(parents=True, exist_ok=True)
out_path = "data/promociones.csv"
df_promociones.to_csv(out_path, index=False, encoding="utf-8-sig")

print("✅ promociones.csv generado en", out_path, "->", df_promociones.shape)
display(df_promociones.head(30))

con = sqlite3.connect("database/mi_base.db")
df_promociones.to_sql("promociones", con, if_exists="replace", index=False)
con.close()
print("💾 Tabla 'promociones' escrita en SQLite")


✅ promociones.csv generado en data/promociones.csv -> (270, 8)


,promotion_id,nombre,tipo,fecha_inicio,fecha_fin,categoria,descuento_pct,prioridad
0,2017ENE-ABR,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Abrigo,0.40,2
1,2017ENE-BUF,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Bufanda,0.25,2
2,2017ENE-CLC,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Calcetines,0.25,2
3,2017ENE-CLZ,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Calzado,0.10,2
4,2017ENE-CAM,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Camisa,0.15,2
5,2017ENE-CMT,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Camiseta,0.25,2
6,2017ENE-CIN,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Cinturón,0.05,2
7,2017ENE-GOR,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Gorra,0.05,2
8,2017ENE-PAN,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Pantalón,0.20,2
9,2017ENE-SUD,Rebajas Invierno 2017,rebajas,2017-01-07,2017-02-15,Sudadera,0.15,2


💾 Tabla 'promociones' escrita en SQLite


# Clientes

In [9]:
# Generador de clientes (tabla `clientes`)
# Este bloque crea un histórico de altas de clientes y su actividad agregada:
# - provincia/comunidad (geografía)
# - fecha primera y última compra dentro del horizonte del proyecto
# - nº de pedidos y nº de ítems comprados (proxy de engagement)
# Después integra edad en el momento del alta y año de nacimiento.
#
# Nota importante de reproducibilidad:
# - Cada cliente usa un RNG derivado de su ID interno (_tmp_id). Eso hace que los perfiles sean estables
#   aunque cambie el orden de ejecución del notebook (mientras el contador avance igual).

# Si ya tienes los imports en tu celda global, aquí no hace falta repetirlos.
# Este bloque asume disponibles:
#   pandas as pd, random, calendar, math, hashlib
#   date (datetime), y tus módulos: edades, growth_curve, geografia

def _ym_to_int(y: int, m: int) -> int:
    return y * 12 + (m - 1)

def _int_to_ym(x: int):
    y = x // 12
    m = (x % 12) + 1
    return y, m

def _months_between(d1: date, d2: date) -> int:
    a = _ym_to_int(d1.year, d1.month)
    b = _ym_to_int(d2.year, d2.month)
    return max(0, b - a)

def _month_add(d: date, k: int) -> date:
    i0 = _ym_to_int(d.year, d.month)
    y1, m1 = _int_to_ym(i0 + k)

    last_day = (pd.Timestamp(year=y1, month=m1, day=1) + pd.offsets.MonthEnd(0)).day
    day = min(d.day, last_day)
    return date(y1, m1, day)

def _seeded_rng_from_id(u_id: int) -> random.Random:
    # RNG determinista por cliente: el seed depende solo del id interno
    h = hashlib.sha256(str(u_id).encode("utf-8")).hexdigest()
    seed_int = int(h[:16], 16) % (2**31 - 1)
    return random.Random(seed_int)

def _beta_sample(rng: random.Random, a: float, b: float) -> float:
    x = rng.gammavariate(a, 1.0)
    y = rng.gammavariate(b, 1.0)
    if (x + y) == 0:
        return 0.5
    return x / (x + y)

def _normal_int(rng: random.Random, sigma: float) -> int:
    # Box–Muller: entero centrado en 0 para “jitter” temporal
    u1 = max(1e-12, rng.random())
    u2 = rng.random()
    z = math.sqrt(-2.0 * math.log(u1)) * math.cos(2 * math.pi * u2)
    return int(round(z * sigma))

def _geom_trunc(rng: random.Random, p: float = 0.5, lo: int = 1, hi: int = 3) -> int:
    # Geométrica truncada: retroceso de 1–3 meses cuando se evita caer “en pared” al final del horizonte
    k = 1
    while rng.random() > p and k < 100:
        k += 1
    return max(lo, min(hi, k))

def _beta_params_for_k(k: int):
    # Parámetros beta en función del nº de pedidos: permite colas largas (clientes que vuelven tarde)
    if k == 2:
        return 1.05, 2.55
    if 3 <= k <= 4:
        return 1.25, 2.25
    if 5 <= k <= 8:
        return 1.55, 2.05
    if 9 <= k <= 12:
        return 1.85, 1.95
    return 2.10, 1.85  # k >= 13

def _sigma_for_k(k: int) -> float:
    # Dispersión temporal asociada al nº de pedidos
    if k == 2:
        return 0.7
    if 3 <= k <= 4:
        return 1.0
    if 5 <= k <= 8:
        return 1.4
    if 9 <= k <= 12:
        return 1.9
    return 2.5

def sample_num_pedidos(
    rng: random.Random,
    year: int,
    month: int,
    project_end: date,
    k_max: int = 25
) -> int:
    """
    Muestrea nº de pedidos por cliente.
    La distribución evoluciona con el tiempo (t) y, cerca del final del proyecto, empuja hacia “one-shot”
    para evitar acumulación artificial de clientes multi-compra justo al final del horizonte.
    """
    t = (year - 2017) + (month - 1) / 12.0
    beta_base = 1.95 - 0.025 * t
    lam_base  = 0.34 - 0.018 * t

    beta = max(1.40, min(2.35, beta_base * (0.95 + 0.10 * rng.random())))
    lam  = max(0.12, min(0.46, lam_base  * (0.90 + 0.20 * rng.random())))

    weights = []
    for k in range(1, k_max + 1):
        w = (k + 1.0) ** (-beta) * math.exp(-lam * k)
        weights.append(w)

    head_boost_base = 1.76 - 0.04 * t
    head_boost = max(1.35, min(1.75, head_boost_base)) * (0.97 + 0.06 * rng.random())

    _, dim_m = calendar.monthrange(year, month)
    fecha_alta_dt = date(year, month, min(15, dim_m))
    months_to_end = _months_between(fecha_alta_dt, project_end)

    ramp = 0.0
    if months_to_end <= 9:
        ramp = 0.35 * (9 - months_to_end) / 9.0

    head_boost *= (1.0 + ramp)
    weights[0] *= head_boost

    if months_to_end <= 6:
        mid_scale = 0.82 + 0.04 * rng.random()
        for k in (2, 3, 4):
            if k - 1 < len(weights):
                weights[k - 1] *= mid_scale
    else:
        mid_boost = 1.00 + 0.10 * rng.random()
        for k in (2, 3, 4):
            if k - 1 < len(weights):
                weights[k - 1] *= mid_boost

    total = sum(weights)
    r = rng.random() * total
    cum = 0.0
    for idx, w in enumerate(weights, start=1):
        cum += w
        if r <= cum:
            return idx
    return k_max

def build_clientes() -> pd.DataFrame:
    cfg = example_config()
    altas = build_monthly_new_customers(cfg)

    PROJECT_END = date(2025, 9, 30)

    clientes = []
    customer_counter = 1

    for alta in altas:
        n = alta["new_customers"]
        period = alta["period"]
        year, month = alta["year"], alta["month"]

        for _ in range(n):
            rng = _seeded_rng_from_id(customer_counter)

            provincia, comunidad = asignar_provincia(year, period, random_state=customer_counter)

            n_pedidos = sample_num_pedidos(rng, year, month, PROJECT_END)

            mu_low  = 1.10 + 0.06 * math.log1p(n_pedidos)
            mu_high = 1.70 + 0.12 * math.log1p(n_pedidos)
            upt = rng.uniform(mu_low, mu_high)
            upt = max(1.0, min(2.4, upt))
            n_items = max(1, int(round(n_pedidos * upt)))

            # Primera compra: día aleatorio dentro del mes de alta
            _, dim1 = calendar.monthrange(year, month)
            d1 = rng.randint(1, dim1)
            fecha_primer_dt = date(year, month, d1)

            # Última compra: si hay más de 1 pedido, se distribuye en el horizonte restante
            if n_pedidos == 1:
                fecha_ultima_dt = fecha_primer_dt
            else:
                available = _months_between(fecha_primer_dt, PROJECT_END)
                if available <= 0:
                    fecha_ultima_dt = fecha_primer_dt
                else:
                    a, b = _beta_params_for_k(n_pedidos)
                    F = _beta_sample(rng, a, b)
                    delta_base = int(round(available * F))
                    delta = max(
                        0,
                        min(available, delta_base + _normal_int(rng, _sigma_for_k(n_pedidos)))
                    )

                    # Anti-“pared”: si cae justo en el último mes, la mayoría retrocede 1–3 meses
                    if delta >= available:
                        if rng.random() >= 0.03:
                            delta -= _geom_trunc(rng, p=0.5, lo=1, hi=3)
                            delta = max(0, min(delta, available))

                    fecha_ultima_dt = _month_add(fecha_primer_dt, delta)

                    # Stickiness estacional: parte de clientes tiende a recomprar en meses cercanos al de alta
                    if rng.random() < 0.35 and delta > 0:
                        mes_obj = fecha_primer_dt.month
                        if abs(fecha_ultima_dt.month - mes_obj) > 1 and _months_between(fecha_primer_dt, fecha_ultima_dt) >= 1:
                            fecha_ultima_dt = _month_add(fecha_ultima_dt, rng.choice([-1, 1]))
                            if fecha_ultima_dt > PROJECT_END:
                                fecha_ultima_dt = PROJECT_END

                    # Día aleatorio dentro del mes de última compra
                    _, dim2 = calendar.monthrange(fecha_ultima_dt.year, fecha_ultima_dt.month)
                    d2 = rng.randint(1, dim2)
                    fecha_ultima_dt = date(fecha_ultima_dt.year, fecha_ultima_dt.month, d2)

                    if fecha_ultima_dt < fecha_primer_dt:
                        fecha_ultima_dt = fecha_primer_dt
                    if fecha_ultima_dt > PROJECT_END:
                        fecha_ultima_dt = PROJECT_END

            clientes.append({
                "_tmp_id": customer_counter,
                "provincia": provincia,
                "comunidad": comunidad,
                "fecha_primer_compra": fecha_primer_dt.isoformat(),
                "fecha_ultima_compra": fecha_ultima_dt.isoformat(),
                "n_pedidos": n_pedidos,
                "n_items_comprados": n_items,
            })

            customer_counter += 1

    df = pd.DataFrame(clientes)

    # Customer_id final ordenado por fecha de primera compra (estable)
    df["fecha_primer_compra_dt"] = pd.to_datetime(df["fecha_primer_compra"])
    df = df.sort_values("fecha_primer_compra_dt", kind="stable").reset_index(drop=True)

    df["customer_id"] = (df.index + 1).map(lambda i: f"C{i:06d}")
    df = df.drop(columns=["_tmp_id", "fecha_primer_compra_dt"])

    cols = [
        "customer_id", "provincia", "comunidad",
        "fecha_primer_compra", "fecha_ultima_compra",
        "n_pedidos", "n_items_comprados",
    ]
    return df[cols]

def asignar_edad_alta(df_clientes: pd.DataFrame) -> pd.DataFrame:
    """
    Integra:
    - edad_alta: edad del cliente en el momento de su primera compra
    - anio_nacimiento: derivado de (año primera compra - edad_alta)

    La asignación se hace por (mes de alta, provincia) para capturar diferencias geográficas/temporales.
    """
    df = df_clientes.copy()

    if not pd.api.types.is_datetime64_any_dtype(df["fecha_primer_compra"]):
        df["fecha_primer_compra"] = pd.to_datetime(df["fecha_primer_compra"], errors="coerce")

    if "_anio_primera" not in df.columns:
        df["_anio_primera"] = df["fecha_primer_compra"].dt.year

    df["_month_key"] = df["fecha_primer_compra"].dt.strftime("%Y-%m")

    keys = df[["_month_key", "_anio_primera", "provincia"]].drop_duplicates()
    cache = {}
    for _, row in keys.iterrows():
        mk = row["_month_key"]
        yr = int(row["_anio_primera"])
        prov = row["provincia"]
        cache[(mk, prov)] = build_month_samplers(month_key=mk, year=yr, provincia=prov)

    def _asigna(grp: pd.DataFrame) -> pd.Series:
        mk = grp["_month_key"].iat[0]
        prov = grp["provincia"].iat[0]
        rng, w = cache[(mk, prov)]
        return grp.index.to_series().map(lambda _: sample_age_from_weights(rng, w))

    if "edad_alta" not in df.columns or df["edad_alta"].isna().any():
        df["edad_alta"] = (
            df.groupby(["_month_key", "provincia"], group_keys=False)
              .apply(_asigna)
              .astype("int16")
        )

    if "anio_nacimiento" not in df.columns or df["anio_nacimiento"].isna().any():
        df["anio_nacimiento"] = (df["_anio_primera"] - df["edad_alta"]).astype("int16")

    df.drop(columns=["_month_key"], inplace=True)
    return df


# Ejecución en notebook (sin bloque __main__)
df = build_clientes()
df = asignar_edad_alta(df)

display(df.head(10))
print("Total clientes generados:", len(df))
print("Media de pedidos:", df["n_pedidos"].mean())
print("Media de items:", df["n_items_comprados"].mean())
print("Edad (min, p50, max):", int(df["edad_alta"].min()), int(df["edad_alta"].median()), int(df["edad_alta"].max()))

Path("data").mkdir(parents=True, exist_ok=True)
df.to_csv("data/clientes.csv", index=False)
print("Archivo clientes.csv exportado correctamente")

con = sqlite3.connect("database/mi_base.db")
df.to_sql("clientes", con, if_exists="replace", index=False)
con.close()


C:\Users\pmace\AppData\Local\Temp\ipykernel_20676\3416757071.py:284: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_asigna)


,customer_id,provincia,comunidad,fecha_primer_compra,fecha_ultima_compra,n_pedidos,n_items_comprados,_anio_primera,edad_alta,anio_nacimiento
0,C000001,Granada,Andalucía,2017-08-01,2017-08-01,1,1,2017,18,1999
1,C000002,Gipuzkoa,País Vasco,2017-08-01,2017-08-01,1,1,2017,22,1995
2,C000003,Madrid,Madrid,2017-08-01,2021-02-14,2,3,2017,16,2001
3,C000004,Madrid,Madrid,2017-08-01,2023-02-08,2,3,2017,28,1989
4,C000005,Madrid,Madrid,2017-08-01,2019-08-24,2,2,2017,23,1994
5,C000006,Madrid,Madrid,2017-08-01,2017-08-01,1,2,2017,22,1995
6,C000007,Barcelona,Cataluña,2017-08-01,2018-07-07,2,3,2017,21,1996
7,C000008,Madrid,Madrid,2017-08-01,2020-05-20,2,3,2017,19,1998
8,C000009,Valencia,Comunidad Valenciana,2017-08-01,2017-08-01,1,1,2017,18,1999
9,C000010,Madrid,Madrid,2017-08-01,2024-10-31,2,3,2017,33,1984


Total clientes generados: 249005
Media de pedidos: 1.8233288488182968
Media de items: 2.749185759322102
Edad (min, p50, max): 16 23 45
Archivo clientes.csv exportado correctamente


# Tickets

In [10]:
# Tickets online (sin devoluciones)
# Construcción de la tabla `tickets_online` a partir de `clientes.csv`.
# El objetivo es desagregar la información agregada por cliente (nº de pedidos e ítems)
# en tickets individuales, garantizando coherencia temporal y contable.
#
# Invariantes que se respetan:
# - Nº de tickets por cliente ≤ n_pedidos declarado en clientes
# - Suma de ítems por cliente = n_items_comprados
# - Provincia consistente para todos los tickets del mismo cliente


# Semilla fija para reproducibilidad global del generador
RNG = random.Random(42)

CLIENTES_PATH = Path("data/clientes.csv")
OUT_CSV_PATH = Path("data/tickets_online.csv")
SQLITE_PATH = "database/mi_base.db"



# Utilidades de preprocesado


def _to_datetime(df: pd.DataFrame, cols: list[str]) -> None:
    """Tipa columnas de fecha de forma segura."""
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")


def _clean_provincia(s: pd.Series) -> pd.Series:
    """Normaliza el texto de provincia para evitar inconsistencias en joins."""
    return (
        s.astype("string")
         .str.strip()
         .str.replace(r"\s+", " ", regex=True)
    )


def _build_prov_map(clientes: pd.DataFrame) -> pd.DataFrame:
    """
    Construye un mapping estable customer_id → provincia.
    Se toma siempre la primera provincia observada por cliente para evitar duplicados.
    """
    prov = clientes.loc[clientes["customer_id"].notna(), ["customer_id", "provincia"]].copy()
    prov["provincia"] = _clean_provincia(prov["provincia"])
    return (
        prov.sort_values("customer_id")
            .groupby("customer_id", as_index=False, sort=False)
            .agg(provincia=("provincia", "first"))
    )



# Generación de fechas de tickets


def _ticket_dates(fp: pd.Timestamp, fu: pd.Timestamp, k: int, rng: random.Random) -> list[pd.Timestamp]:
    """
    Genera las fechas de los tickets de un cliente:
    - 1 pedido  -> fecha primera compra
    - 2 pedidos -> primera y última compra
    - ≥3        -> extremos + fechas internas repartidas de forma estable
    """
    if k <= 0:
        return []

    if pd.isna(fp) and pd.isna(fu):
        base = pd.Timestamp.today().normalize()
        return [base] * k

    if pd.isna(fp):
        fp = fu
    if pd.isna(fu):
        fu = fp

    if k == 1:
        return [fp]
    if k == 2:
        return sorted([fp, fu])

    delta_days = max(int((fu - fp).days), 0)

    if delta_days <= 1:
        internas = [fp] * (k - 2)
    else:
        need = k - 2
        if delta_days - 1 >= need:
            picks = sorted(rng.sample(range(1, delta_days), need))
        else:
            picks = list(range(1, delta_days))
        internas = [fp + pd.Timedelta(days=d) for d in picks]
        internas += [fp] * (need - len(internas))

    return sorted([fp, fu] + internas)



# Distribución de ítems entre tickets


def _split_items(total_items: int, k_decl: int, rng: random.Random) -> tuple[list[int], int]:
    """
    Distribuye el total de ítems entre los tickets:
    - Se garantiza al menos 1 ítem por ticket si hay suficientes ítems.
    - Si los ítems son insuficientes, se reduce el nº real de tickets.
    """
    if k_decl <= 0:
        return ([], 0)

    if total_items <= 0:
        return ([0] * k_decl, k_decl)

    if total_items < k_decl:
        return ([1] * total_items, total_items)

    items = [1] * k_decl
    remaining = total_items - k_decl
    for _ in range(remaining):
        items[rng.randrange(k_decl)] += 1
    return (items, k_decl)



# Carga de clientes


clientes = pd.read_csv(CLIENTES_PATH)
_to_datetime(clientes, ["fecha_primer_compra", "fecha_ultima_compra"])

if "customer_id" not in clientes.columns:
    raise ValueError("clientes.csv debe contener la columna 'customer_id'.")
if "provincia" not in clientes.columns:
    raise ValueError("clientes.csv debe contener la columna 'provincia'.")

prov_map = _build_prov_map(clientes)



# Construcción de tickets online


rows: list[dict] = []
ticket_seq = 1

for r in clientes.itertuples(index=False):
    cust_id = getattr(r, "customer_id", None)
    if cust_id is None:
        continue

    fp = getattr(r, "fecha_primer_compra", pd.NaT)
    fu = getattr(r, "fecha_ultima_compra", pd.NaT)

    k_decl = int(getattr(r, "n_pedidos", 0) or 0)
    items_total = int(getattr(r, "n_items_comprados", 0) or 0)

    if k_decl <= 0 and items_total <= 0:
        continue

    items_per_ticket, k_real = _split_items(items_total, k_decl, RNG)
    if k_real <= 0:
        continue

    fechas = _ticket_dates(fp, fu, k_real, RNG)

    for i in range(k_real):
        rows.append({
            "ticket_id": f"T{ticket_seq:06d}",
            "customer_id": cust_id,
            "canal": "online",
            "fecha_ticket": pd.Timestamp(fechas[i]).date(),
            "n_items": int(items_per_ticket[i]),
        })
        ticket_seq += 1


tickets_online = pd.DataFrame(rows)



# Enriquecimiento con provincia (sin alterar cardinalidad)


n_before = len(tickets_online)
tickets_online = tickets_online.merge(
    prov_map, on="customer_id", how="left", validate="many_to_one"
)

if len(tickets_online) != n_before:
    raise AssertionError("El merge con provincia ha alterado el número de filas.")

tickets_online = tickets_online[
    ["ticket_id", "customer_id", "canal", "fecha_ticket", "n_items", "provincia"]
].sort_values(
    ["customer_id", "fecha_ticket", "ticket_id"], ignore_index=True
)



# Validaciones de coherencia por cliente


chk = (
    tickets_online.groupby("customer_id", as_index=False)
                  .agg(n_pedidos_real=("ticket_id", "count"),
                       n_items=("n_items", "sum"))
)

base = clientes[["customer_id", "n_pedidos", "n_items_comprados"]]
valid = chk.merge(base, on="customer_id", how="left", validate="one_to_one")

assert (valid["n_items"] == valid["n_items_comprados"]).all(), \
    "No cuadra el total de ítems para algún cliente."

assert (valid["n_pedidos_real"] <= valid["n_pedidos"]).all(), \
    "Algún cliente tiene más tickets que pedidos declarados."



# Exportación


OUT_CSV_PATH.parent.mkdir(parents=True, exist_ok=True)
tickets_online.to_csv(OUT_CSV_PATH, index=False)

con = sqlite3.connect(SQLITE_PATH)
tickets_online.to_sql("tickets_online", con, if_exists="replace", index=False)
con.close()


In [18]:
# ======= REPARADOR OPCIONAL: n_items_devueltos <= n_items por ticket =======
import pandas as pd

if "tickets_online" not in globals() or "clientes" not in globals():
    raise RuntimeError("Carga primero 'tickets_online' y 'clientes'.")

t = tickets_online.copy()

def fix_customer(group: pd.DataFrame) -> pd.DataFrame:
    # Trabajamos por índice para poder devolver con cambios
    g = group.copy()
    idx = g.index.tolist()

    items = g["n_items"].astype(int).values
    devs  = g["n_items_devueltos"].astype(int).values
    devolved_flag = g["tiene_devolucion"].astype(int).values

    # Paso 1: cortar excesos y acumular overflow
    overflow = 0
    for i in range(len(g)):
        if devs[i] > items[i]:
            overflow += (devs[i] - items[i])
            devs[i] = items[i]

    if overflow == 0:
        g["n_items_devueltos"] = devs
        return g

    # Paso 2: repartir overflow en otros tickets devueltos con capacidad
    def capacity(i): return max(items[i] - devs[i], 0)

    while overflow > 0:
        # candidatos devueltos con hueco
        cand = [i for i in range(len(g)) if devolved_flag[i]==1 and capacity(i) > 0]
        if not cand:
            break
        i = cand[0]
        take = min(capacity(i), overflow)
        devs[i] += take
        overflow -= take

    # Paso 3 (si aún sobra): crear capacidad moviendo items desde no-dev a dev
    # escogemos no-dev con items>0 y los pasamos a dev con menor ratio dev/items
    while overflow > 0:
        nondev_src = [i for i in range(len(g)) if devolved_flag[i]==0 and items[i] > 0]
        dev_dst    = [i for i in range(len(g)) if devolved_flag[i]==1]
        if not nondev_src or not dev_dst:
            # sin posibilidad de crear capacidad; forzamos cap y salimos
            # (esto solo se daría si todos los tickets dev ya igualan sus items originales)
            break

        i_from = nondev_src[0]
        i_to   = sorted(dev_dst, key=lambda j: (items[j]-devs[j]), reverse=True)[0]  # más hueco actual
        # mover 1 item del no-dev al dev
        items[i_from] -= 1
        items[i_to]   += 1
        # asignarlo como devuelto en destino
        devs[i_to]    += 1
        overflow      -= 1

    # Por seguridad final: clamp
    for i in range(len(g)):
        if devs[i] > items[i]:
            devs[i] = items[i]

    g.loc[idx, "n_items"] = items
    g.loc[idx, "n_items_devueltos"] = devs
    return g

# Aplica por cliente SOLO donde hay problemas
mask_problem = t["n_items_devueltos"] > t["n_items"]
customers_problem = t.loc[mask_problem, "customer_id"].unique().tolist()

t_fixed = (t.groupby("customer_id", group_keys=False)
             .apply(lambda df: fix_customer(df) if df["customer_id"].iat[0] in customers_problem else df)
             .reset_index(drop=True))

# Recalcula el contador de problemáticos
n_bad_before = (tickets_online["n_items_devueltos"] > tickets_online["n_items"]).sum()
n_bad_after  = (t_fixed["n_items_devueltos"] > t_fixed["n_items"]).sum()

print(f"Tickets problemáticos antes: {n_bad_before}")
print(f"Tickets problemáticos después: {n_bad_after}")

# Si te gusta el resultado, sustituyes el DataFrame original en memoria:
tickets_online = t_fixed


Tickets problemáticos antes: 11100
Tickets problemáticos después: 0


C:\Users\pmace\AppData\Local\Temp\ipykernel_17676\2691785631.py:75: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: fix_customer(df) if df["customer_id"].iat[0] in customers_problem else df)


In [11]:
# Este bloque genera tickets del canal físico (plan 4 tiendas) y los une con los tickets online ya generados.
# No modifica los tickets online (se conservan sus ticket_id). Los tickets físicos se crean con un prefijo distinto.
# La salida mantiene exactamente el esquema: ticket_id, customer_id, canal, fecha_ticket, n_items, provincia.

DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

ANIO_INI, ANIO_FIN = 2022, 2025
PROYECTO_END = date(2025, 9, 30)

RNG = random.Random(7)  # reproducibilidad global del bloque

TARGET_TICKETS_FISICO_YEAR = {
    2022: 14000,  # Madrid (apertura 2022)
    2023: 28000,  # + Barcelona
    2024: 40000,  # + Valencia
    2025: 48000,  # + Sevilla (hasta 2025-09-30)
}

PESOS_TIENDA = {
    "MAD01": 1.00,
    "BCN01": 0.86,
    "VAL01": 0.57,
    "SEV01": 0.43,
}

CAP_SHARE = 0.40  # límite anual por tienda: evita concentraciones excesivas

UPT_LOW, UPT_HIGH = 1.2, 2.2  # rango de items por ticket (proxy de tamaño de cesta)


def _normalize(series: pd.Series) -> pd.Series:
    total = float(series.sum())
    return series * 0.0 if total <= 0 else series / total


def _meses_abierto_en_anio(fecha_apertura: pd.Timestamp, year: int, project_end: date) -> int:
    start = pd.Timestamp(year=year, month=1, day=1)
    end = pd.Timestamp(year=year, month=12, day=31)
    a = max(pd.Timestamp(fecha_apertura), start)
    b = min(pd.Timestamp(project_end), end)
    if b < a:
        return 0
    return (b.year - a.year) * 12 + (b.month - a.month) + 1


def _rampa_annual_factor(year: int, apertura: pd.Timestamp) -> float:
    """
    Ajuste suave de ramp-up tras apertura:
    - año de apertura: menor volumen (arranque operativo)
    - año siguiente: consolidación
    - resto: tienda a ritmo normal
    """
    y0 = int(apertura.year)
    if year < y0:
        return 0.0
    dt_years = year - y0
    if dt_years == 0:
        return 0.45
    if dt_years == 1:
        return 0.80
    return 1.00


def _dias_del_mes(y: int, m: int) -> pd.DatetimeIndex:
    start = pd.Timestamp(year=y, month=m, day=1)
    end = start + pd.offsets.MonthEnd(1)
    return pd.date_range(start, end, freq="D")


def _sample_dates_month(y: int, m: int, k: int, rng: random.Random) -> list[pd.Timestamp]:
    """
    Genera k fechas dentro del mes, con un sesgo semanal realista:
    - sábado y domingo pesan más (mayor tráfico de tienda)
    - viernes ligeramente por encima
    - lunes ligeramente por debajo
    """
    if k <= 0:
        return []
    days = _dias_del_mes(y, m)

    weights = []
    for d in days:
        wd = int(d.weekday())  # 0=lun..6=dom
        if wd == 5:
            w = 1.6
        elif wd == 6:
            w = 1.3
        elif wd == 4:
            w = 1.2
        elif wd == 0:
            w = 0.9
        else:
            w = 1.0
        weights.append(w)

    weights = np.asarray(weights, dtype=float)
    weights = weights / weights.sum()

    idx = rng.choices(range(len(days)), weights=weights.tolist(), k=k)
    fechas = sorted(days[i] for i in idx)
    return [pd.Timestamp(f.date()) for f in fechas]


def _distribute_sum(total: int, k: int, rng: random.Random, min_each: int = 1) -> list[int]:
    """
    Reparte un total entero en k partes, asegurando un mínimo por ticket.
    Se usa para asignar n_items de forma consistente (sin tickets negativos ni vacíos en físico).
    """
    if k <= 0:
        return []
    base = [min_each] * k
    remaining = max(0, int(total) - k * min_each)
    for _ in range(remaining):
        base[rng.randrange(k)] += 1
    return base


# 1) Tickets online: se cargan si no están en memoria y se validan columnas esperadas
if "df_tickets_online" not in globals():
    df_tickets_online = pd.read_csv(DATA_DIR / "tickets_online.csv", parse_dates=["fecha_ticket"])

req_cols = {"ticket_id", "customer_id", "canal", "fecha_ticket", "n_items", "provincia"}
missing = req_cols - set(df_tickets_online.columns)
if missing:
    raise ValueError(f"tickets_online no cumple el esquema esperado (faltan columnas): {missing}")

df_online = df_tickets_online.copy()
df_online["fecha_ticket"] = pd.to_datetime(df_online["fecha_ticket"], errors="coerce")


# 2) Tiendas físicas: preferimos el builder si existe para centralizar configuración
try:
    from tiendas import build_tiendas
    df_tiendas = build_tiendas()
except Exception:
    df_tiendas = pd.DataFrame(
        {
            "store_id": ["MAD01", "BCN01", "VAL01", "SEV01"],
            "provincia": ["Madrid", "Barcelona", "Valencia", "Sevilla"],
            "comunidad": ["Comunidad de Madrid", "Cataluña", "Comunidad Valenciana", "Andalucía"],
            "fecha_apertura": ["2022-05-15", "2023-04-01", "2024-03-15", "2025-03-10"],
        }
    )

df_tiendas["fecha_apertura"] = pd.to_datetime(df_tiendas["fecha_apertura"], errors="coerce")
store2prov = df_tiendas.set_index("store_id")["provincia"].to_dict()

# Export de catálogo de tiendas (útil para joins, BI y documentación)
df_tiendas.to_csv(DATA_DIR / "tiendas.csv", index=False, encoding="utf-8-sig")


# 3) Forma mensual: usamos la curva de crecimiento para heredar estacionalidad coherente con el proyecto
try:
    from growth_curve import example_config, build_monthly_new_customers

    cfg = example_config()
    monthly = pd.DataFrame(build_monthly_new_customers(cfg))
    monthly["fecha"] = pd.to_datetime(monthly["period"].astype(str) + "-01", errors="coerce")
    monthly = monthly[(monthly["fecha"].dt.year >= ANIO_INI) & (monthly["fecha"].dt.year <= ANIO_FIN)]
    monthly["year"] = monthly["fecha"].dt.year
    monthly["month"] = monthly["fecha"].dt.month

    mes_shapes = (
        monthly.groupby(["year", "month"], as_index=False)["new_customers"]
        .sum()
        .rename(columns={"new_customers": "peso_raw"})
    )
    mes_shapes["peso"] = mes_shapes.groupby("year")["peso_raw"].transform(lambda s: s / s.sum())
except Exception:
    # Fallback: reparto plano por mes en caso de no disponer de growth_curve
    grid = [(y, m) for y in range(ANIO_INI, ANIO_FIN + 1) for m in range(1, 13)]
    mes_shapes = pd.DataFrame(grid, columns=["year", "month"])
    mes_shapes["peso"] = mes_shapes.groupby("year")["month"].transform(lambda s: 1.0 / len(s))


# 4) Asignación anual por tienda: respeta apertura, ramp-up y un cap de concentración
rows = []
for _, t in df_tiendas.iterrows():
    sid = t["store_id"]
    apertura = t["fecha_apertura"]
    w_base = float(PESOS_TIENDA.get(sid, 0.10))

    for y in range(ANIO_INI, ANIO_FIN + 1):
        meses_abiertos = _meses_abierto_en_anio(apertura, y, PROYECTO_END)
        if meses_abiertos <= 0:
            continue

        ramp = _rampa_annual_factor(y, apertura)
        w_raw = w_base * (meses_abiertos / 12.0) * ramp
        rows.append({"year": y, "store_id": sid, "meses_abiertos": meses_abiertos, "w_raw": w_raw})

df_w = pd.DataFrame(rows)

asignaciones = []
for y, g in df_w.groupby("year", sort=True):
    total_y = int(TARGET_TICKETS_FISICO_YEAR.get(int(y), 0))
    if total_y <= 0:
        continue

    w = _normalize(g["w_raw"].astype(float)).fillna(0.0)
    target = w * total_y

    base = np.floor(target).astype(int)
    gap = int(total_y - base.sum())
    if gap > 0:
        order = np.argsort(-(target - base))
        base.iloc[order[:gap]] += 1

    tmp = g[["store_id"]].reset_index(drop=True).copy()
    tmp["year"] = int(y)
    tmp["tickets_fisico"] = base.values

    cap = int(math.floor(CAP_SHARE * total_y))

    exceso = (tmp["tickets_fisico"] - cap).clip(lower=0)
    over = int(exceso.sum())
    if over > 0:
        tmp.loc[exceso > 0, "tickets_fisico"] = cap

        mask_rest = exceso.eq(0)
        if mask_rest.any():
            w_rest = _normalize(pd.Series(w.values, index=tmp.index)[mask_rest]).fillna(0.0)
            inc = np.floor(w_rest * over).astype(int)

            gap2 = int(over - inc.sum())
            if gap2 > 0:
                rema = (w_rest * over) - inc
                order2 = rema.sort_values(ascending=False).index.tolist()
                inc.loc[order2[:gap2]] += 1

            inc_full = pd.Series(0, index=tmp.index, dtype=int)
            inc_full.loc[w_rest.index] = inc.values
            tmp["tickets_fisico"] += inc_full

    delta = int(total_y - tmp["tickets_fisico"].sum())
    if delta != 0:
        idxs = tmp["tickets_fisico"].sort_values(ascending=(delta < 0)).index.tolist()
        for i in range(abs(delta)):
            tmp.loc[idxs[i % len(idxs)], "tickets_fisico"] += (1 if delta > 0 else -1)

    asignaciones.append(tmp)

df_year_store = pd.concat(asignaciones, ignore_index=True)


# 5) Expansión a tickets físicos: reparto mensual + fechas diarias + items por ticket
rows = []
seq_fisico = 1  # ids físicos independientes (no se reenumeran los online)

for _, r in df_year_store.iterrows():
    sid = r["store_id"]
    y = int(r["year"])
    n_year = int(r["tickets_fisico"])
    if n_year <= 0:
        continue

    sub = mes_shapes[mes_shapes["year"] == y].set_index("month")["peso"]
    sub = sub / sub.sum()

    meses = sub.index.tolist()
    counts = np.floor(sub.values * n_year).astype(int)
    gap = int(n_year - counts.sum())
    if gap > 0:
        # Ajuste por residuos para que el total anual cuadre exactamente
        order = np.argsort(-(sub.values - counts / max(n_year, 1)))
        counts[order[:gap]] += 1

    for m, n_mes in zip(meses, counts):
        n_mes = int(n_mes)
        if n_mes <= 0:
            continue

        prov = store2prov.get(sid)
        fechas = _sample_dates_month(y, int(m), n_mes, RNG)

        # UPT simple: genera un total de ítems y lo reparte por ticket con mínimo 1
        upt = RNG.uniform(UPT_LOW, UPT_HIGH)
        items_total = max(n_mes, int(round(upt * n_mes)))
        items_por_ticket = _distribute_sum(items_total, n_mes, RNG, min_each=1)

        for i in range(n_mes):
            rows.append(
                {
                    "ticket_id": f"F{seq_fisico:06d}",
                    "customer_id": None,  # canal físico: cliente no identificado en esta simulación
                    "canal": "fisico",
                    "fecha_ticket": pd.Timestamp(fechas[i]).date(),
                    "n_items": int(items_por_ticket[i]),
                    "provincia": prov,
                }
            )
            seq_fisico += 1

df_tickets_fisicos = pd.DataFrame(rows)

# Validaciones mínimas: canal correcto y tickets físicos con al menos un ítem
if len(df_tickets_fisicos) > 0:
    assert df_tickets_fisicos["canal"].eq("fisico").all()
    assert df_tickets_fisicos["n_items"].ge(1).all()


# 6) Unión de tickets: se conservan IDs online; se concatenan con esquema idéntico
cols_out = ["ticket_id", "customer_id", "canal", "fecha_ticket", "n_items", "provincia"]

df_online_out = df_online[cols_out].copy()
df_fisico_out = df_tickets_fisicos[cols_out].copy()

df_all = pd.concat([df_online_out, df_fisico_out], ignore_index=True)

df_all["fecha_ticket"] = pd.to_datetime(df_all["fecha_ticket"], errors="coerce")
df_all = df_all.sort_values(["fecha_ticket", "canal", "ticket_id"], kind="stable").reset_index(drop=True)

assert set(df_online_out.columns) == set(cols_out)
assert set(df_fisico_out.columns) == set(cols_out)
assert df_all["canal"].isin(["online", "fisico"]).all()
assert df_all["n_items"].ge(0).all()
assert pd.api.types.is_datetime64_any_dtype(df_all["fecha_ticket"])


print("\nResumen tickets (sin devoluciones)")
print(f"Online (sin tocar): {len(df_online_out):,}")
print(f"Físicos (nuevos):   {len(df_fisico_out):,}")
print(f"Total:              {len(df_all):,}")

display(df_tickets_fisicos.head(10))
display(df_all.head(10))


# 7) Export a CSV y SQLite
out_fisicos = DATA_DIR / "tickets_fisicos.csv"
out_total = DATA_DIR / "tickets_total.csv"
out_tiendas = DATA_DIR / "tiendas.csv"

df_fisico_out.to_csv(out_fisicos, index=False, encoding="utf-8-sig")
df_all.to_csv(out_total, index=False, encoding="utf-8-sig")
df_tiendas.to_csv(out_tiendas, index=False, encoding="utf-8-sig")

print(f"\nExport OK: {out_fisicos}  (filas={len(df_fisico_out):,}, cols={df_fisico_out.shape[1]})")
print(f"Export OK: {out_total}    (filas={len(df_all):,}, cols={df_all.shape[1]})")
print(f"Export OK: {out_tiendas}  (filas={len(df_tiendas):,}, cols={df_tiendas.shape[1]})")

con = sqlite3.connect("database/mi_base.db")
df_fisico_out.to_sql("tickets_fisico", con, if_exists="replace", index=False)
df_all.to_sql("tickets_total", con, if_exists="replace", index=False)
df_tiendas.to_sql("tiendas", con, if_exists="replace", index=False)
con.close()



Resumen tickets (sin devoluciones)
Online (sin tocar): 454,018
Físicos (nuevos):   130,000
Total:              584,018


,ticket_id,customer_id,canal,fecha_ticket,n_items,provincia
0,F000001,None,fisico,2022-01-01,2,Madrid
1,F000002,None,fisico,2022-01-01,1,Madrid
2,F000003,None,fisico,2022-01-01,2,Madrid
3,F000004,None,fisico,2022-01-01,1,Madrid
4,F000005,None,fisico,2022-01-01,1,Madrid
5,F000006,None,fisico,2022-01-01,2,Madrid
6,F000007,None,fisico,2022-01-01,1,Madrid
7,F000008,None,fisico,2022-01-01,1,Madrid
8,F000009,None,fisico,2022-01-01,2,Madrid
9,F000010,None,fisico,2022-01-01,1,Madrid


,ticket_id,customer_id,canal,fecha_ticket,n_items,provincia
0,T000001,C000001,online,2017-08-01,1,Granada
1,T000002,C000002,online,2017-08-01,1,Gipuzkoa
2,T000003,C000003,online,2017-08-01,2,Madrid
3,T000005,C000004,online,2017-08-01,2,Madrid
4,T000007,C000005,online,2017-08-01,1,Madrid
5,T000009,C000006,online,2017-08-01,2,Madrid
6,T000010,C000007,online,2017-08-01,2,Barcelona
7,T000012,C000008,online,2017-08-01,2,Madrid
8,T000014,C000009,online,2017-08-01,1,Valencia
9,T000015,C000010,online,2017-08-01,2,Madrid



Export OK: data\tickets_fisicos.csv  (filas=130,000, cols=6)
Export OK: data\tickets_total.csv    (filas=584,018, cols=6)
Export OK: data\tiendas.csv  (filas=4, cols=4)


In [40]:
# ====== COMPROBANTES TRAS UNIR ONLINE + FÍSICO ======
import pandas as pd
import numpy as np

# --- requisitos mínimos en memoria ---
objs = ["df_tickets_online", "df_tickets_fisicos", "df_all"]
missing = [o for o in objs if o not in globals()]
if missing:
    raise RuntimeError(f"Faltan en memoria: {missing}. Ejecuta antes la generación.")

# columnas esperadas (mismas que tickets_online)
REQ_COLS = {"ticket_id","customer_id","canal","fecha_ticket","n_items","n_items_devueltos","tiene_devolucion","provincia"}

# ---------------------------
# 0) Tipado básico
# ---------------------------
df_tickets_online["fecha_ticket"] = pd.to_datetime(df_tickets_online["fecha_ticket"], errors="coerce")
df_tickets_fisicos["fecha_ticket"] = pd.to_datetime(df_tickets_fisicos["fecha_ticket"], errors="coerce")
df_all["fecha_ticket"] = pd.to_datetime(df_all["fecha_ticket"], errors="coerce")

for d in (df_tickets_online, df_tickets_fisicos, df_all):
    for c in ["n_items","n_items_devueltos","tiene_devolucion"]:
        d[c] = pd.to_numeric(d[c], errors="coerce").fillna(0).astype(int)

# ---------------------------
# 1) Esquema de columnas y duplicados
# ---------------------------
cols_ok = set(df_all.columns) == REQ_COLS
dups_all = df_all["ticket_id"].duplicated().sum()

print("=== 1) ESQUEMA & DUPLICADOS ===")
print("Columnas correctas en df_all:", cols_ok, f"({sorted(df_all.columns)})")
print("Ticket_id duplicados en df_all:", dups_all)

# ---------------------------
# 2) Integridad ONLINE (se mantienen intactos)
# ---------------------------
all_online = df_all[df_all["canal"] == "online"].copy()
same_count_online = len(all_online) == len(df_tickets_online)
same_ids_online = set(all_online["ticket_id"]) == set(df_tickets_online["ticket_id"])

# compara igualdad por ticket_id en todas las columnas comunes (con tipos coherentes)
cols_common = [c for c in ["ticket_id","customer_id","canal","fecha_ticket","n_items",
                           "n_items_devueltos","tiene_devolucion","provincia"]
               if c in df_tickets_online.columns and c in all_online.columns]

# sub-DFs con mismas columnas
on_all  = all_online[cols_common].copy()
on_orig = df_tickets_online[cols_common].copy()

# Tipado coherente para comparar
if "fecha_ticket" in cols_common:
    on_all["fecha_ticket"]  = pd.to_datetime(on_all["fecha_ticket"], errors="coerce")
    on_orig["fecha_ticket"] = pd.to_datetime(on_orig["fecha_ticket"], errors="coerce")

for cnum in ["n_items","n_items_devueltos","tiene_devolucion"]:
    if cnum in cols_common:
        on_all[cnum]  = pd.to_numeric(on_all[cnum], errors="coerce").fillna(0).astype(int)
        on_orig[cnum] = pd.to_numeric(on_orig[cnum], errors="coerce").fillna(0).astype(int)

for cstr in ["customer_id","canal","provincia"]:
    if cstr in cols_common:
        on_all[cstr]  = on_all[cstr].astype(str).fillna("")
        on_orig[cstr] = on_orig[cstr].astype(str).fillna("")

merge_cmp = on_all.merge(
    on_orig, on="ticket_id", suffixes=("_all","_orig"), how="outer", indicator=True
)

# diferencias por columna (deberían ser 0)
mismatches = {}
for c in cols_common:
    if c == "ticket_id": 
        continue
    ca, co = f"{c}_all", f"{c}_orig"
    mismatches[c] = (merge_cmp[ca].astype(str).fillna("") != merge_cmp[co].astype(str).fillna("")).sum()

print("\n=== 2) ONLINE INTACTO (CORREGIDO) ===")
print("Mismo nº de filas:", same_count_online, f"({len(all_online)} vs {len(df_tickets_online)})")
print("Mismo set de ticket_id:", same_ids_online)
print("Diferencias por columna (deberían ser 0):")
for k, v in mismatches.items():
    print(f"  - {k}: {v}")


# ---------------------------
# 3) Reglas FÍSICO
# ---------------------------
fis = df_tickets_fisicos.copy()
fis_ok_canal = fis["canal"].eq("fisico").all()
fis_ok_items_pos = (fis["n_items"] >= 1).all()
fis_ok_dev_le_items = (fis["n_items_devueltos"] <= fis["n_items"]).all()
fis_ok_flag = (fis["tiene_devolucion"] == (fis["n_items_devueltos"] > 0).astype(int)).all()

print("\n=== 3) REGLAS FÍSICO ===")
print("canal='fisico' en todos:", fis_ok_canal)
print("n_items >= 1:", fis_ok_items_pos)
print("n_items_devueltos <= n_items:", fis_ok_dev_le_items)
print("tiene_devolucion coincide con (n_items_devueltos>0):", fis_ok_flag)

# ---------------------------
# 4) Sumatorios & particiones coherentes
# ---------------------------
tot_all = len(df_all)
tot_online = (df_all["canal"] == "online").sum()
tot_fisico = (df_all["canal"] == "fisico").sum()

print("\n=== 4) SUMATORIOS ===")
print("TOTAL df_all:", tot_all)
print("ONLINE en df_all:", tot_online, "| FÍSICO en df_all:", tot_fisico)
print("Consistencia (online + físico == total):", tot_online + tot_fisico == tot_all)

# ---------------------------
# 5) Metas por año del FÍSICO (si definiste TARGET_TICKETS_FISICO_YEAR)
#     -> exact match por año
# ---------------------------
TARGET_TICKETS_FISICO_YEAR = {
    2022: 14000,
    2023: 28000,
    2024: 40000,
    2025: 48000,
}
fis_year = df_tickets_fisicos.assign(year=df_tickets_fisicos["fecha_ticket"].dt.year)
counts_year = fis_year.groupby("year")["ticket_id"].count().to_dict()
targets_compare = {y: (counts_year.get(y, 0), TARGET_TICKETS_FISICO_YEAR.get(y, 0), counts_year.get(y, 0) == TARGET_TICKETS_FISICO_YEAR.get(y, 0))
                   for y in TARGET_TICKETS_FISICO_YEAR.keys()}

print("\n=== 5) METAS FÍSICO POR AÑO ===")
for y, (real, target, ok) in targets_compare.items():
    print(f"Año {y}: real={real} | target={target} | OK={ok}")

# ---------------------------
# 6) Coherencia global de devoluciones por ticket en df_all
# ---------------------------
bad_dev = (df_all["n_items_devueltos"] > df_all["n_items"]).sum()
print("\n=== 6) CHEQUEO GLOBAL TICKET ===")
print("Tickets con devoluciones > items (debería 0):", bad_dev)

if bad_dev > 0:
    print("\nEjemplos (top 10) de devoluciones > items:")
    display(df_all.loc[df_all["n_items_devueltos"] > df_all["n_items"], 
                       ["ticket_id","canal","fecha_ticket","n_items","n_items_devueltos","tiene_devolucion","provincia"]].head(10))

# ---------------------------
# 7) (Opcional) Si tienes 'clientes', revalida que ONLINE mantiene totales vs clientes
# ---------------------------
if "clientes" in globals():
    c = clientes.copy()
    c["customer_id"] = c["customer_id"].astype(str)
    on = df_tickets_online.copy()
    on["customer_id"] = on["customer_id"].astype(str)

    agg_on = on.groupby("customer_id", as_index=False).agg(
        items_sum=("n_items","sum"),
        items_dev_sum=("n_items_devueltos","sum")
    )
    cmp = c[["customer_id","n_items_comprados","n_items_devueltos"]].merge(agg_on, on="customer_id", how="left")
    cmp[["items_sum","items_dev_sum"]] = cmp[["items_sum","items_dev_sum"]].fillna(0).astype(int)

    bad_items = (cmp["items_sum"] != cmp["n_items_comprados"]).sum()
    bad_devol = (cmp["items_dev_sum"] != cmp["n_items_devueltos"]).sum()

    print("\n=== 7) CLIENTES vs ONLINE (totales por cliente) ===")
    print("Clientes con mismatch de ITEMS:", bad_items)
    print("Clientes con mismatch de DEVOLUCIONES:", bad_devol)
    if bad_items or bad_devol:
        print("\nMuestra de discrepancias (top 10):")
        display(cmp.loc[(cmp["items_sum"] != cmp["n_items_comprados"]) | 
                        (cmp["items_dev_sum"] != cmp["n_items_devueltos"]),
                        ["customer_id","n_items_comprados","items_sum","n_items_devueltos","items_dev_sum"]].head(10))


=== 1) ESQUEMA & DUPLICADOS ===
Columnas correctas en df_all: True (['canal', 'customer_id', 'fecha_ticket', 'n_items', 'n_items_devueltos', 'provincia', 'ticket_id', 'tiene_devolucion'])
Ticket_id duplicados en df_all: 0

=== 2) ONLINE INTACTO (CORREGIDO) ===
Mismo nº de filas: True (454018 vs 454018)
Mismo set de ticket_id: True
Diferencias por columna (deberían ser 0):
  - customer_id: 0
  - canal: 0
  - fecha_ticket: 0
  - n_items: 0
  - n_items_devueltos: 0
  - tiene_devolucion: 0
  - provincia: 0

=== 3) REGLAS FÍSICO ===
canal='fisico' en todos: True
n_items >= 1: True
n_items_devueltos <= n_items: True
tiene_devolucion coincide con (n_items_devueltos>0): True

=== 4) SUMATORIOS ===
TOTAL df_all: 584018
ONLINE en df_all: 454018 | FÍSICO en df_all: 130000
Consistencia (online + físico == total): True

=== 5) METAS FÍSICO POR AÑO ===
Año 2022: real=14000 | target=14000 | OK=True
Año 2023: real=28000 | target=28000 | OK=True
Año 2024: real=40000 | target=40000 | OK=True
Año 2025: r

,customer_id,n_items_comprados,items_sum,n_items_devueltos,items_dev_sum
312,C000313,2,2,2,1
360,C000361,2,2,2,1
378,C000379,2,2,2,1
1061,C001062,2,2,2,1
1199,C001200,2,2,2,1
2064,C002065,2,2,2,1
2229,C002230,2,2,2,1
2289,C002290,2,2,2,1
3049,C003050,2,2,2,1
3261,C003262,2,2,2,1


# items_1

In [22]:
def normaliza_texto(s: pd.Series) -> pd.Series:
    """
    Normalización consistente para claves textuales usadas en cruces y validaciones.
    Hace el match case-insensitive y evita discrepancias por espacios.
    """
    return (
        s.astype("string")
         .str.strip()
         .str.replace(r"\s+", " ", regex=True)
         .str.lower()
    )


def hash_unitario(*valores) -> float:
    """
    Pseudo-aleatoriedad determinista en [0, 1) a partir de una clave compuesta.
    Se usa para asignar SKUs de forma estable sin depender del orden de ejecución.
    """
    key = "|".join(map(str, valores))
    h = hashlib.md5(key.encode("utf-8")).hexdigest()
    return int(h[:16], 16) / float(0xFFFFFFFFFFFFFFFF)


def expandir_tickets_a_items(tickets_df: pd.DataFrame) -> pd.DataFrame:
    """
    Convierte tickets en items individuales.
    No añade store_id aquí: store_id se asigna después solo para canal físico.
    """
    df = tickets_df.copy()

    if "n_items" not in df.columns:
        df["n_items"] = 0

    df["n_items"] = (
        pd.to_numeric(df["n_items"], errors="coerce")
          .fillna(0)
          .astype(int)
          .clip(lower=0)
    )

    base = df.loc[df.index.repeat(df["n_items"])].copy()
    if base.empty:
        return pd.DataFrame(
            columns=[
                "item_id", "ticket_id", "pos_item",
                "customer_id", "canal", "provincia", "fecha_item"
            ]
        )

    base["pos_item"] = base.groupby("ticket_id").cumcount() + 1
    base = base.rename(columns={"fecha_ticket": "fecha_item"})

    base["item_id"] = (
        base["ticket_id"].astype(str)
        + "-"
        + base["pos_item"].astype(str).str.zfill(3)
    )

    return base[
        [
            "item_id", "ticket_id", "pos_item",
            "customer_id", "canal", "provincia", "fecha_item"
        ]
    ]


def asignar_store_id_items(items_df: pd.DataFrame, tiendas_df: pd.DataFrame) -> pd.DataFrame:
    """
    Asigna store_id a items físicos usando tiendas.csv.
    El match es por provincia normalizada (case-insensitive).
    Online queda con store_id vacío (NA).
    """
    out = items_df.copy()

    if "store_id" in out.columns:
        out.drop(columns=["store_id"], inplace=True)

    out["canal"] = normaliza_texto(out["canal"])
    out["provincia"] = normaliza_texto(out["provincia"])

    tiendas = tiendas_df.copy()
    if "store_id" not in tiendas.columns or "provincia" not in tiendas.columns:
        raise ValueError("tiendas.csv debe tener columnas 'store_id' y 'provincia'.")

    tiendas["store_id"] = tiendas["store_id"].astype("string").str.strip()
    tiendas["provincia"] = normaliza_texto(tiendas["provincia"])

    if tiendas["provincia"].isna().any() or tiendas["store_id"].isna().any():
        raise ValueError("tiendas.csv contiene provincias o store_id nulos. Revisa el fichero.")

    dup = tiendas["provincia"].duplicated(keep=False)
    if dup.any():
        sample = tiendas.loc[dup, ["provincia", "store_id"]].head(10)
        raise ValueError(
            "tiendas.csv tiene provincias duplicadas (match no es 1:1). Muestra:\n"
            + sample.to_string(index=False)
        )

    prov_to_store = tiendas.set_index("provincia")["store_id"].to_dict()

    out["store_id"] = pd.NA
    mask_fisico = out["canal"].eq("fisico")
    out.loc[mask_fisico, "store_id"] = out.loc[mask_fisico, "provincia"].map(prov_to_store)

    missing = out.loc[mask_fisico & out["store_id"].isna(), ["ticket_id", "item_id", "provincia", "fecha_item"]]
    if not missing.empty:
        raise ValueError(
            "Hay items físicos sin store_id. Revisa provincia en tickets_total vs tiendas.csv.\n"
            + missing.head(20).to_string(index=False)
        )

    return out


def variantes_activas_en_fecha(variantes_df: pd.DataFrame, fecha: pd.Timestamp) -> pd.DataFrame:
    """
    Filtra variantes activas en una fecha:
    lanzamiento <= fecha y (retiro es nulo o fecha < retiro).
    """
    v = variantes_df.copy()
    if "lanzamiento" not in v.columns:
        raise ValueError("productos_variantes.csv debe tener columna 'lanzamiento'.")
    if "retiro" not in v.columns:
        v["retiro"] = pd.NaT

    v["lanzamiento"] = pd.to_datetime(v["lanzamiento"], errors="coerce")
    v["retiro"] = pd.to_datetime(v["retiro"], errors="coerce")

    cond_lanz = v["lanzamiento"].notna() & (v["lanzamiento"] <= fecha)
    cond_ret = v["retiro"].isna() | (fecha < v["retiro"])
    return v.loc[cond_lanz & cond_ret]


def asignar_sku_determinista(items_df: pd.DataFrame, variantes_df: pd.DataFrame) -> pd.DataFrame:
    """
    Asigna un SKU a cada item de forma determinista, respetando el catálogo activo por día.
    La selección depende de (ticket_id, pos_item).
    """
    items = items_df.copy()
    items["fecha_item"] = pd.to_datetime(items["fecha_item"], errors="coerce")
    if items["fecha_item"].isna().any():
        bad = items.loc[items["fecha_item"].isna(), ["item_id", "ticket_id"]].head(10)
        raise ValueError("Existen items con fecha_item inválida. Muestra:\n" + bad.to_string(index=False))

    v = variantes_df.copy()
    v.columns = v.columns.str.strip()

    if "categoría" not in v.columns and "categoria" in v.columns:
        v = v.rename(columns={"categoria": "categoría"})

    required = ["sku", "id_producto", "categoría", "color", "talla", "precio", "coste bruto", "lanzamiento"]
    missing = [c for c in required if c not in v.columns]
    if missing:
        raise ValueError(f"Faltan columnas en productos_variantes.csv: {missing}")

    cache_activos = {}
    bloques = []

    items["_fecha_dia"] = items["fecha_item"].dt.floor("D")

    for dia, grp in items.groupby("_fecha_dia", sort=False):
        activos = cache_activos.get(dia)
        if activos is None:
            activos = variantes_activas_en_fecha(v, dia)
            if activos.empty:
                activos = v.copy()
            activos = activos.reset_index(drop=True)
            cache_activos[dia] = activos

        n = len(activos)
        if n <= 0:
            raise ValueError("No hay variantes disponibles para asignar SKU.")

        idxs = [
            min(int(np.floor(hash_unitario(tid, pos) * n)), n - 1)
            for tid, pos in zip(grp["ticket_id"].astype(str), grp["pos_item"].astype(int))
        ]

        sel = activos.iloc[idxs].reset_index(drop=True)

        enriched = grp.reset_index(drop=True).join(
            sel[
                ["sku", "id_producto", "categoría", "color", "talla", "precio", "coste bruto"]
            ].rename(columns={"coste bruto": "coste_bruto"})
        )

        bloques.append(enriched)

    out = pd.concat(bloques, ignore_index=True).drop(columns=["_fecha_dia"], errors="ignore")
    return out


def aplicar_promociones(items_df: pd.DataFrame, promos_df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica promociones por (fecha, categoría).
    Si hay solapes, se elige por:
      1) prioridad mayor
      2) descuento mayor
      3) promotion_id mayor (desempate estable)
    """
    out = items_df.copy()
    out["fecha_item"] = pd.to_datetime(out["fecha_item"], errors="coerce")
    out["fecha_dia"] = out["fecha_item"].dt.floor("D")

    out.drop(columns=["promotion_id", "descuento_pct"], errors="ignore", inplace=True)

    if promos_df is None or promos_df.empty:
        out["promotion_id"] = pd.NA
        out["descuento_pct"] = 0.0
        return out.drop(columns="fecha_dia")

    p = promos_df.copy()
    p.columns = p.columns.str.strip()
    if "categoría" not in p.columns and "categoria" in p.columns:
        p = p.rename(columns={"categoria": "categoría"})

    needed = ["promotion_id", "fecha_inicio", "fecha_fin", "categoría", "descuento_pct"]
    miss = [c for c in needed if c not in p.columns]
    if miss:
        raise ValueError(f"Faltan columnas en promociones.csv: {miss}")

    p["fecha_inicio"] = pd.to_datetime(p["fecha_inicio"], errors="coerce")
    p["fecha_fin"] = pd.to_datetime(p["fecha_fin"], errors="coerce")
    p["prioridad"] = pd.to_numeric(p.get("prioridad", 0), errors="coerce").fillna(0).astype(int)
    p["descuento_pct"] = pd.to_numeric(p["descuento_pct"], errors="coerce").fillna(0.0).astype(float)

    fmin = out["fecha_dia"].min()
    fmax = out["fecha_dia"].max()
    p = p.loc[(p["fecha_fin"] >= fmin) & (p["fecha_inicio"] <= fmax)].copy()

    chunks = []
    for dia, grp in out.groupby("fecha_dia", sort=False):
        pf = p.loc[(p["fecha_inicio"] <= dia) & (dia <= p["fecha_fin"])]

        if pf.empty:
            tmp = grp.copy()
            tmp["promotion_id"] = pd.NA
            tmp["descuento_pct"] = 0.0
            chunks.append(tmp)
            continue

        pf = pf.sort_values(
            ["categoría", "prioridad", "descuento_pct", "promotion_id"],
            ascending=[True, False, False, False],
        )

        winners = pf.drop_duplicates("categoría")[
            ["categoría", "promotion_id", "descuento_pct"]
        ]

        tmp = grp.merge(winners, on="categoría", how="left")
        tmp["promotion_id"] = tmp["promotion_id"].astype("string")
        tmp["descuento_pct"] = tmp["descuento_pct"].fillna(0.0).astype(float)
        chunks.append(tmp)

    return pd.concat(chunks, ignore_index=True).drop(columns="fecha_dia")


def calcular_economia_unitaria(items_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula PVP, descuento, neto, coste y margen por item.
    """
    out = items_df.copy()

    out["pvp_unitario"] = pd.to_numeric(out["precio"], errors="coerce").astype(float)
    out["descuento_pct"] = pd.to_numeric(out["descuento_pct"], errors="coerce").fillna(0.0).astype(float)
    out["precio_neto_unit"] = out["pvp_unitario"] * (1.0 - out["descuento_pct"])

    out["coste_bruto"] = pd.to_numeric(out["coste_bruto"], errors="coerce").astype(float)
    out["margen_unit"] = out["precio_neto_unit"] - out["coste_bruto"]

    return out


def validar_consistencia(items_df: pd.DataFrame, tickets_df: pd.DataFrame, clientes_df: pd.DataFrame) -> None:
    """
    Validaciones de contrato entre tablas:
    - items por ticket coincide con n_items de tickets
    - store_id presente para canal físico y ausente para online
    - provincia online coincide con clientes
    """
    recuento_items = items_df.groupby("ticket_id").size().rename("n_items_calc")
    chk = tickets_df.set_index("ticket_id")["n_items"].rename("n_items_decl")

    comp = pd.concat([chk, recuento_items], axis=1).fillna(0)
    comp["n_items_decl"] = comp["n_items_decl"].astype(int)
    comp["n_items_calc"] = comp["n_items_calc"].astype(int)

    bad = comp.loc[comp["n_items_decl"] != comp["n_items_calc"]]
    if not bad.empty:
        raise ValueError(
            "El número de items no coincide con tickets. Muestra:\n"
            + bad.head(20).to_string()
        )

    claves = ["item_id", "ticket_id", "pos_item", "canal", "fecha_item", "sku", "id_producto", "categoría"]
    faltan = [c for c in claves if c not in items_df.columns]
    if faltan:
        raise ValueError(f"Faltan columnas clave en items: {faltan}")

    if items_df[claves].isna().any().any():
        cols_nan = [c for c in claves if items_df[c].isna().any()]
        sample = items_df.loc[items_df[cols_nan].isna().any(axis=1), claves].head(20)
        raise ValueError(
            "Hay NaN en columnas clave: "
            + ", ".join(cols_nan)
            + "\nMuestra:\n"
            + sample.to_string(index=False)
        )

    canal_norm = normaliza_texto(items_df["canal"])
    mask_fisico = canal_norm.eq("fisico")
    mask_online = canal_norm.eq("online")

    if mask_fisico.any():
        miss_store = items_df.loc[mask_fisico & items_df["store_id"].isna(), ["item_id", "provincia"]].head(20)
        if not miss_store.empty:
            raise ValueError("Existen items físicos sin store_id. Muestra:\n" + miss_store.to_string(index=False))

    if mask_online.any():
        if items_df.loc[mask_online, "store_id"].notna().any():
            sample = items_df.loc[mask_online & items_df["store_id"].notna(), ["item_id", "store_id"]].head(20)
            raise ValueError("Existen items online con store_id (no debe). Muestra:\n" + sample.to_string(index=False))

        clientes = clientes_df.copy()
        if "customer_id" not in clientes.columns or "provincia" not in clientes.columns:
            raise ValueError("clientes.csv debe tener customer_id y provincia para validar online.")

        clientes["customer_id"] = clientes["customer_id"].astype("string")
        clientes["provincia"] = normaliza_texto(clientes["provincia"])
        prov_map = clientes.set_index("customer_id")["provincia"].to_dict()

        online = items_df.loc[mask_online].copy()
        online["provincia_norm"] = normaliza_texto(online["provincia"])
        online["prov_expected"] = online["customer_id"].astype("string").map(prov_map)

        badp = online.loc[
            online["customer_id"].notna()
            & online["prov_expected"].notna()
            & (online["provincia_norm"] != online["prov_expected"]),
            ["ticket_id", "customer_id", "provincia", "prov_expected"]
        ].head(20)

        if not badp.empty:
            raise ValueError("Provincia online no coincide con clientes. Muestra:\n" + badp.to_string(index=False))


# Carga de datos base

tickets = pd.read_csv("data/tickets_total.csv", parse_dates=["fecha_ticket"])
promos = pd.read_csv("data/promociones.csv", parse_dates=["fecha_inicio", "fecha_fin"])
variantes = pd.read_csv("data/productos_variantes.csv")
clientes = pd.read_csv("data/clientes.csv", parse_dates=["fecha_primer_compra"])
tiendas = pd.read_csv("data/tiendas.csv")

tickets.columns = tickets.columns.str.strip()
promos.columns = promos.columns.str.strip()
variantes.columns = variantes.columns.str.strip()
clientes.columns = clientes.columns.str.strip()
tiendas.columns = tiendas.columns.str.strip()

if "provincia" not in tickets.columns:
    raise ValueError("tickets_total.csv debe incluir columna 'provincia'.")
if "canal" not in tickets.columns:
    raise ValueError("tickets_total.csv debe incluir columna 'canal'.")
if "ticket_id" not in tickets.columns:
    raise ValueError("tickets_total.csv debe incluir columna 'ticket_id'.")

if "fecha_ticket" not in tickets.columns:
    raise ValueError("tickets_total.csv debe incluir columna 'fecha_ticket'.")

if "n_items" not in tickets.columns:
    tickets["n_items"] = 0

tickets["canal"] = tickets["canal"].astype("string")
tickets["customer_id"] = tickets.get("customer_id", pd.Series(pd.NA, index=tickets.index)).astype("string")
tickets["provincia"] = normaliza_texto(tickets["provincia"])
clientes["customer_id"] = clientes["customer_id"].astype("string")
clientes["provincia"] = normaliza_texto(clientes["provincia"])

# Reconciliación provincia online contra clientes
prov_map = clientes.set_index("customer_id")["provincia"].to_dict()
mask_online = normaliza_texto(tickets["canal"]).eq("online") & tickets["customer_id"].notna()
tickets.loc[mask_online, "provincia"] = tickets.loc[mask_online, "customer_id"].map(prov_map)

# Pipeline principal de generación

items = expandir_tickets_a_items(tickets.rename(columns={"fecha_ticket": "fecha_ticket"}))
items["fecha_item"] = pd.to_datetime(items["fecha_item"], errors="coerce")

items = asignar_store_id_items(items, tiendas)
items = asignar_sku_determinista(items, variantes)
items = aplicar_promociones(items, promos)
items = calcular_economia_unitaria(items)

items_venta = items[
    [
        "item_id", "ticket_id", "pos_item",
        "customer_id", "canal", "store_id", "provincia", "fecha_item",
        "sku", "id_producto", "categoría", "color", "talla",
        "pvp_unitario", "descuento_pct", "precio_neto_unit", "coste_bruto", "margen_unit",
        "promotion_id",
    ]
].copy()

validar_consistencia(items_venta, tickets, clientes)

# Exportación final

out_csv = "data/items_venta.csv"
items_venta.to_csv(out_csv, index=False)

con = sqlite3.connect(r"C:/Users/pmace/Desktop/ropa/database/mi_base.db")
items_venta.to_sql("items_1", con, if_exists="replace", index=False)
con.close()

print(f"OK items_venta generado: {out_csv} (filas={len(items_venta):,}, cols={items_venta.shape[1]})")
print("OK items_venta escrito en SQLite (tabla: items_venta)")


C:\Users\pmace\AppData\Local\Temp\ipykernel_20676\2927098836.py:265: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(chunks, ignore_index=True).drop(columns="fecha_dia")


OK items_venta generado: data/items_venta.csv (filas=905,445, cols=19)
OK items_venta escrito en SQLite (tabla: items_venta)


# items_2

In [6]:
import pandas as pd

df = pd.read_csv("data/items_venta.csv", low_memory=False)

# normalizar categoría
def strip_accents(s):
    trans = str.maketrans("áéíóúüñÁÉÍÓÚÜÑ", "aeiouunAEIOUUN")
    return str(s).translate(trans)

df["cat_norm"] = (
    df["categoría"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map(strip_accents)
)

# extraer mes
df["mes"] = pd.to_datetime(df["fecha_item"], errors="coerce").dt.month


In [7]:
mix_mes_cat = (
    df.groupby(["mes", "cat_norm"])
      .size()
      .reset_index(name="n_items")
      .sort_values(["mes", "n_items"], ascending=[True, False])
)

mix_mes_cat


,mes,cat_norm,n_items
5,1,camiseta,20122
0,1,abrigo,19704
8,1,pantalon,14924
9,1,sudadera,9387
4,1,camisa,4363
...,...,...,...
111,12,calzado,4058
114,12,cinturon,1115
115,12,gorra,1000
109,12,bufanda,723


In [8]:
mix_mes_canal_cat = (
    df.groupby(["mes", "canal", "cat_norm"])
      .size()
      .reset_index(name="n_items")
      .sort_values(["mes", "canal", "n_items"], ascending=[True, True, False])
)

mix_mes_canal_cat


,mes,canal,cat_norm,n_items
0,1,fisico,abrigo,5353
5,1,fisico,camiseta,5115
8,1,fisico,pantalon,3670
9,1,fisico,sudadera,2315
4,1,fisico,camisa,1317
...,...,...,...,...
229,12,online,calzado,3057
232,12,online,cinturon,885
233,12,online,gorra,827
227,12,online,bufanda,543


In [10]:
from __future__ import annotations

import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm


# ============================================================
# CONFIG
# ============================================================
RANDOM_SEED = 42
rng_global = np.random.default_rng(RANDOM_SEED)

INPUT_ITEMS = Path("data/items_venta.csv")
INPUT_PRODUCTOS = Path("data/productos.csv")

OUTPUT_ITEMS = Path("data/items_venta_ajustado.csv")  # <- salida final

TICKET_COL = "ticket_id"
DATE_COL = "fecha_item"
CAT_COL = "categoría"
CANAL_COL = "canal"
PROV_COL = "provincia"
IDPROD_COL = "id_producto"

# Columnas que SIEMPRE deben moverse juntas (bundle coherente)
BUNDLE_COLS = [
    "sku", "id_producto", "categoría", "color", "talla",
    "pvp_unitario", "descuento_pct", "precio_neto_unit",
    "coste_bruto", "margen_unit", "promotion_id"
]

CANALES_VALIDOS = {"online", "fisico"}

MESES_VERANO = {6, 7, 8}
MESES_INVIERNO = {12, 1, 2}

# Promos
MES_REBAJAS_INVIERNO = {1}
MES_REBAJAS_VERANO = {7}

# Capping de cambios por grupo (para que no intente cosas imposibles)
CAP_ABS_PP = 8.0   # max +-8pp por cat en un grupo
CAP_REL = 0.35     # max +-35% relativo vs mix original

# Cooc intensidades -> target lift (realista)
R4_STRONG = 2.5
R4_MED = 1.6
R4_WEAK = 0.8

COOC_REL = {
    "camiseta": {"gorra": R4_MED, "calcetines": R4_WEAK, "pantalon": R4_WEAK, "sudadera": R4_WEAK, "calzado": R4_WEAK},
    "sudadera": {"gorra": R4_MED, "camiseta": R4_MED, "pantalon": R4_WEAK, "calzado": R4_WEAK},
    "pantalon": {"cinturon": R4_STRONG, "calcetines": R4_WEAK, "camiseta": R4_WEAK, "sudadera": R4_WEAK, "calzado": R4_WEAK},
    "abrigo": {"bufanda": R4_STRONG, "gorra": R4_MED, "camiseta": R4_WEAK, "camisa": R4_WEAK, "pantalon": R4_WEAK},
    "calzado": {"calcetines": R4_STRONG, "pantalon": R4_WEAK, "camiseta": R4_WEAK, "sudadera": R4_WEAK},
    "camisa": {"cinturon": R4_MED, "pantalon": R4_MED, "abrigo": R4_WEAK},
    "gorra": {"camiseta": R4_WEAK, "sudadera": R4_WEAK, "bufanda": R4_WEAK},
    "cinturon": {"pantalon": R4_MED, "camisa": R4_WEAK},
    "calcetines": {"calzado": R4_MED, "pantalon": R4_WEAK},
    "bufanda": {"abrigo": R4_STRONG, "gorra": R4_WEAK},
}

# Budget cooc (para que nunca se eternice)
COOC_TIME_BUDGET_S_PER_RULE = 0.35   # por (mes, canal) y regla
COOC_MAX_ATTEMPTS_PER_RULE = 200_000
COOC_CHECK_EVERY_SWAPS = 1500

# Mix budget (para que sea rápido y controlado)
MIX_MAX_SWAPS_PER_GROUP = 40_000
MIX_MAX_ATTEMPTS_CALENDAR = 60  # intentos para encontrar pareja de swap válida por calendario


# ============================================================
# NORMALIZACIÓN
# ============================================================
def strip_accents(text: str) -> str:
    trans = str.maketrans("áéíóúüñÁÉÍÓÚÜÑ", "aeiouunAEIOUUN")
    return text.translate(trans)

def norm_cat(x) -> str:
    if pd.isna(x):
        return ""
    return strip_accents(str(x).strip().lower())

def norm_key(x) -> str:
    if pd.isna(x):
        return ""
    return strip_accents(str(x).strip().lower())

def ym_int_from_date(s: pd.Series) -> pd.Series:
    dt = pd.to_datetime(s, errors="coerce")
    if dt.isna().any():
        bad = int(dt.isna().sum())
        raise ValueError(f"Hay {bad} fechas inválidas en {DATE_COL}")
    return (dt.dt.year * 100 + dt.dt.month).astype(np.int32)

def month_from_ym(ym: int) -> int:
    return int(ym % 100)

def ym_from_str_yyyy_mm(val: object) -> Optional[int]:
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return None
    s = str(val).strip()
    if not s:
        return None
    # "YYYY-MM"
    parts = s.split("-")
    if len(parts) >= 2 and parts[0].isdigit() and parts[1].isdigit():
        y = int(parts[0])
        m = int(parts[1])
        return y * 100 + m
    return None

def renormalize(p: Dict[str, float]) -> Dict[str, float]:
    s = float(sum(p.values()))
    if s <= 0:
        k = len(p)
        return {c: 1.0 / k for c in p}
    return {c: max(0.0, float(v)) / s for c, v in p.items()}

def apply_deltas_pp(p: Dict[str, float], deltas_pp: Dict[str, float]) -> Dict[str, float]:
    q = dict(p)
    for k, dv in deltas_pp.items():
        if k in q:
            q[k] = q.get(k, 0.0) + dv / 100.0
    return q

def cap_changes(p0: Dict[str, float], q: Dict[str, float], cap_abs_pp: float, cap_rel: float) -> Dict[str, float]:
    capped: Dict[str, float] = {}
    for c, target in q.items():
        orig = float(p0.get(c, 0.0))

        rel_min = max(0.0, orig * (1 - cap_rel))
        rel_max = min(1.0, orig * (1 + cap_rel))

        abs_min = max(0.0, orig - cap_abs_pp / 100.0)
        abs_max = min(1.0, orig + cap_abs_pp / 100.0)

        low = max(rel_min, abs_min)
        high = min(rel_max, abs_max)

        capped[c] = min(max(float(target), low), high)
    return renormalize(capped)

def largest_remainder_targets(p: Dict[str, float], n: int, cats: List[str], tie_breaker: Dict[str, int]) -> Dict[str, int]:
    raw = {c: float(p.get(c, 0.0)) * n for c in cats}
    floors = {c: int(np.floor(raw[c])) for c in cats}
    resto = int(n - sum(floors.values()))

    fracs = sorted(
        ((c, raw[c] - floors[c]) for c in cats),
        key=lambda x: (-(x[1]), tie_breaker[x[0]]),
    )
    for i in range(resto):
        floors[fracs[i % len(fracs)][0]] += 1
    return floors


# ============================================================
# ZONAS (simple pero estable) — puedes refinar luego
# ============================================================
ZONA_NORTE = {
    "a coruna","lugo","ourense","pontevedra","asturias","cantabria",
    "alava","bizkaia","gipuzkoa","navarra","la rioja",
    "huesca","teruel","zaragoza",
    "barcelona","girona","lleida","tarragona",
}
ZONA_SUR = {
    "almeria","cadiz","cordoba","granada","huelva","jaen","malaga","sevilla",
    "badajoz","caceres","murcia","ceuta","melilla",
}
ZONA_CENTRO = {
    "madrid","avila","burgos","leon","palencia","salamanca","segovia","soria","valladolid","zamora",
    "albacete","ciudad real","cuenca","guadalajara","toledo",
}
ZONA_ESTE = {"alicante","castellon","valencia","islas baleares"}
ZONA_ISLAS = {"las palmas","santa cruz de tenerife"}

def detect_zone(prov_norm: str) -> str:
    if prov_norm in ZONA_ISLAS:
        return "islas"
    if prov_norm in ZONA_NORTE:
        return "norte"
    if prov_norm in ZONA_SUR:
        return "sur"
    if prov_norm in ZONA_ESTE:
        return "este"
    if prov_norm in ZONA_CENTRO:
        return "centro"
    return "otros"


# ============================================================
# CALENDARIO DE PRODUCTOS (productos.csv)
# ============================================================
def load_catalog(path: Path) -> pd.DataFrame:
    prod = pd.read_csv(path, low_memory=False)
    # Normaliza columnas esperadas
    expected = {"id_producto", "categoria", "lanzamiento", "retiro"}
    if not expected.issubset(set(prod.columns)):
        raise ValueError(f"productos.csv debe tener columnas {sorted(expected)}")
    prod["id_producto"] = prod["id_producto"].astype(str).str.strip()
    prod["cat_norm"] = prod["categoria"].map(norm_cat)
    prod["ym_start"] = prod["lanzamiento"].map(ym_from_str_yyyy_mm).astype("Int64")
    prod["ym_end"] = prod["retiro"].map(ym_from_str_yyyy_mm).astype("Int64")

    # None/NaN en retiro => sigue activo
    prod["ym_end"] = prod["ym_end"].fillna(999912).astype(np.int32)
    prod["ym_start"] = prod["ym_start"].fillna(0).astype(np.int32)

    return prod[["id_producto", "cat_norm", "ym_start", "ym_end"]].copy()

def attach_calendar(df: pd.DataFrame, catalog: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    m = catalog.set_index("id_producto")[["ym_start", "ym_end"]]
    starts = df[IDPROD_COL].map(m["ym_start"]).to_numpy(dtype=np.int32)
    ends = df[IDPROD_COL].map(m["ym_end"]).to_numpy(dtype=np.int32)
    if np.isnan(starts).any() or np.isnan(ends).any():
        missing = df.loc[pd.isna(df[IDPROD_COL].map(m["ym_start"])), IDPROD_COL].astype(str).value_counts().head(10)
        raise ValueError(f"Hay id_producto en items_venta.csv que no están en productos.csv. Ejemplos:\n{missing}")
    return starts, ends

def is_active(ym: int, start: int, end: int) -> bool:
    return (start <= ym) and (ym <= end)


# ============================================================
# SWAP BUNDLE (no toca columnas de grupo: canal/prov/fecha/cliente)
# ============================================================
def swap_bundles(df: pd.DataFrame, i: int, j: int):
    tmp = df.loc[i, BUNDLE_COLS].copy()
    df.loc[i, BUNDLE_COLS] = df.loc[j, BUNDLE_COLS].values
    df.loc[j, BUNDLE_COLS] = tmp.values
    # actualizar cat_norm
    df.at[i, "_cat_norm"] = norm_cat(df.at[i, CAT_COL])
    df.at[j, "_cat_norm"] = norm_cat(df.at[j, CAT_COL])


# ============================================================
# REGLAS MIX (R1-R3 + R4 "ligero" como delta de accesorios)
# ============================================================
def compute_target_mix(p0: Dict[str, float], month: int) -> Dict[str, float]:
    q = dict(p0)

    # R1 verano
    if month in MESES_VERANO:
        q = apply_deltas_pp(q, {"camiseta": +5.0, "abrigo": -5.0, "sudadera": -3.0})

    # R2 invierno
    if month in MESES_INVIERNO:
        q = apply_deltas_pp(q, {"abrigo": +8.0, "calzado": +5.0, "camiseta": -6.0})

    # R3 promos
    if month in MES_REBAJAS_INVIERNO:
        q = apply_deltas_pp(q, {"abrigo": +4.0, "sudadera": +3.0, "camiseta": -3.0})
    if month in MES_REBAJAS_VERANO:
        q = apply_deltas_pp(q, {"camiseta": +4.0, "pantalon": +3.0, "abrigo": -3.0})

    # R4 cross-sell “macro” (muy suave): si hay mucha ropa main, sube un poco accesorios
    share_main = sum(p0.get(c, 0.0) for c in ["camiseta", "sudadera", "pantalon", "abrigo", "camisa"])
    if share_main >= 0.60:
        q = apply_deltas_pp(q, {"gorra": +1.0, "cinturon": +0.8, "calcetines": +0.8, "bufanda": +0.6})

    return renormalize(q)


# ============================================================
# MIX: Ajuste por grupos (ym, canal, zona) con swaps entre grupos del MISMO (canal, zona)
# respetando calendario de productos (swap válido en ambos meses)
# ============================================================
def build_group_index(df: pd.DataFrame) -> Dict[Tuple[int, str, str], np.ndarray]:
    keys = df.groupby(["_ym", "_canal", "_zona"]).indices
    # keys: (ym, canal, zona) -> np.array(index)
    return {k: v for k, v in keys.items()}

def group_counts(df: pd.DataFrame, idxs: np.ndarray, cats_all: List[str]) -> Dict[str, int]:
    vc = df.loc[idxs, "_cat_norm"].value_counts()
    return {c: int(vc.get(c, 0)) for c in cats_all}

def adjust_mix_by_canal_zona(
    df: pd.DataFrame,
    group_idxs: Dict[Tuple[int, str, str], np.ndarray],
    cats_all: List[str],
    tie_breaker: Dict[str, int],
    starts: np.ndarray,
    ends: np.ndarray,
    max_swaps_per_group: int = MIX_MAX_SWAPS_PER_GROUP,
) -> pd.DataFrame:
    """
    Igual que antes, pero con PROGRESO real:
    - imprime avance por canal+zona
    - imprime heartbeat cada X swaps (y ETA)
    """
    t0 = time.time()
    df = df.copy()

    # Targets / deficits / surpluses por grupo
    targets: Dict[Tuple[int, str, str], Dict[str, int]] = {}
    deficits: Dict[Tuple[int, str, str], Dict[str, int]] = {}
    surpluses: Dict[Tuple[int, str, str], Dict[str, int]] = {}

    all_groups = sorted(group_idxs.keys())
    for g in all_groups:
        idxs = group_idxs[g]
        n = len(idxs)
        if n == 0:
            continue
        ym, canal, zona = g
        m = month_from_ym(int(ym))
        vc = df.loc[idxs, "_cat_norm"].value_counts()
        curr = {c: int(vc.get(c, 0)) for c in cats_all}
        p0 = {c: curr[c] / n for c in cats_all}

        qt = compute_target_mix(p0, m)
        qt = cap_changes(p0, qt, CAP_ABS_PP, CAP_REL)
        tgt = largest_remainder_targets(qt, n, cats_all, tie_breaker)

        targets[g] = tgt
        deficits[g] = {c: max(0, tgt[c] - curr[c]) for c in cats_all}
        surpluses[g] = {c: max(0, curr[c] - tgt[c]) for c in cats_all}

    # Pools (group -> cat -> indices)
    pools: Dict[Tuple[int, str, str], Dict[str, np.ndarray]] = {}
    for g in all_groups:
        idxs = group_idxs[g]
        cats = df.loc[idxs, "_cat_norm"].to_numpy()
        pools[g] = {c: idxs[cats == c] for c in cats_all}

    def calendar_ok(i: int, ym_i: int, j: int, ym_j: int) -> bool:
        si, ei = int(starts[i]), int(ends[i])
        sj, ej = int(starts[j]), int(ends[j])
        return is_active(ym_j, si, ei) and is_active(ym_i, sj, ej)

    canal_zonas = sorted({(g[1], g[2]) for g in all_groups})
    total_blocks = len(canal_zonas)

    total_swaps = 0
    total_remaining_deficit0 = sum(sum(d.values()) for d in deficits.values())
    if total_remaining_deficit0 == 0:
        print("[MIX] Ya estaba OK (sin déficit).")
        return df

    print("[MIX] Iniciando ajuste (por canal+zona, monitorizado)...")
    heartbeat_swaps = 25_000  # imprime cada 25k swaps globales
    last_hb = 0

    for b_i, (canal, zona) in enumerate(canal_zonas, 1):
        block_start = time.time()
        gz = [g for g in all_groups if g[1] == canal and g[2] == zona]
        if not gz:
            continue

        # déficit inicial del bloque
        deficit0_block = sum(sum(deficits[g].values()) for g in gz)
        swaps_block = 0

        # tqdm “ligera” por grupos del bloque
        for g in gz:
            ym = int(g[0])

            # saltar si no hay déficit
            if sum(deficits[g].values()) == 0:
                continue

            swaps_g = 0
            while swaps_g < max_swaps_per_group and sum(deficits[g].values()) > 0:
                cats_need = [c for c in cats_all if deficits[g][c] > 0]
                if not cats_need:
                    break
                cat_to = rng_global.choice(cats_need)

                cats_give = [c for c in cats_all if surpluses[g][c] > 0]
                if not cats_give:
                    break
                cat_from = rng_global.choice(cats_give)

                candidates = [g2 for g2 in gz if g2 != g and surpluses[g2][cat_to] > 0]
                if not candidates:
                    deficits[g][cat_to] = 0
                    continue
                g2 = candidates[rng_global.integers(0, len(candidates))]
                ym2 = int(g2[0])

                arr_i = pools[g][cat_from]
                arr_j = pools[g2][cat_to]
                if len(arr_i) == 0 or len(arr_j) == 0:
                    deficits[g][cat_to] = 0
                    continue

                ok = False
                for _ in range(MIX_MAX_ATTEMPTS_CALENDAR):
                    i_idx = int(arr_i[rng_global.integers(0, len(arr_i))])
                    j_idx = int(arr_j[rng_global.integers(0, len(arr_j))])
                    if i_idx == j_idx:
                        continue
                    if calendar_ok(i_idx, ym, j_idx, ym2):
                        swap_bundles(df, i_idx, j_idx)

                        pools[g][cat_from] = pools[g][cat_from][pools[g][cat_from] != i_idx]
                        pools[g][cat_to] = np.append(pools[g][cat_to], i_idx)

                        pools[g2][cat_to] = pools[g2][cat_to][pools[g2][cat_to] != j_idx]
                        pools[g2][cat_from] = np.append(pools[g2][cat_from], j_idx)

                        deficits[g][cat_to] -= 1
                        surpluses[g][cat_from] -= 1
                        surpluses[g2][cat_to] -= 1
                        deficits[g2][cat_from] = max(0, deficits[g2][cat_from] - 1)

                        swaps_g += 1
                        swaps_block += 1
                        total_swaps += 1
                        ok = True
                        break

                if not ok:
                    deficits[g][cat_to] = 0

                # heartbeat global
                if total_swaps - last_hb >= heartbeat_swaps:
                    last_hb = total_swaps
                    remaining = sum(sum(d.values()) for d in deficits.values())
                    done = (1 - remaining / total_remaining_deficit0) * 100
                    elapsed = time.time() - t0
                    rate = total_swaps / elapsed if elapsed > 0 else 0.0
                    eta = (remaining / rate) if rate > 0 else float("inf")
                    print(f"[MIX] {done:5.1f}% | swaps={total_swaps:,} | deficit={remaining:,} | "
                          f"rate={rate:,.0f} swaps/s | ETA~{eta:,.0f}s", flush=True)

        deficit1_block = sum(sum(deficits[g].values()) for g in gz)
        dt_block = time.time() - block_start
        red = (1 - (deficit1_block / deficit0_block)) * 100 if deficit0_block > 0 else 0.0
        print(f"[MIX] Bloque {b_i}/{total_blocks} ({canal}/{zona}) | swaps={swaps_block:,} | "
              f"deficit {deficit0_block:,}->{deficit1_block:,} ({red:.1f}%) | dt={dt_block:.1f}s", flush=True)

    dt = time.time() - t0
    remaining_final = sum(sum(d.values()) for d in deficits.values())
    done_final = (1 - remaining_final / total_remaining_deficit0) * 100
    print(f"[MIX] DONE: {done_final:.1f}% | swaps={total_swaps:,} | deficit_final={remaining_final:,} | dt={dt:.1f}s")
    return df



# ============================================================
# COOC (R4): reforzar cooc dentro de (ym, canal) con budget fijo
# (no toca meses ni canal, sólo mueve bundles entre tickets del mismo bloque)
# ============================================================
def lift_from_ticketset(ticket_set: Dict[str, set], A: str, B: str) -> float:
    tickets = list(ticket_set.keys())
    N = len(tickets)
    if N == 0:
        return np.nan
    nA = sum((A in ticket_set[t]) for t in tickets)
    nB = sum((B in ticket_set[t]) for t in tickets)
    nAB = sum(((A in ticket_set[t]) and (B in ticket_set[t])) for t in tickets)
    if nA == 0 or nB == 0:
        return np.nan
    pB = nB / N
    pB_A = nAB / nA
    return (pB_A / pB) if pB > 0 else np.nan

def enforce_cooc_block(
    df: pd.DataFrame,
    idxs: np.ndarray,
    A: str,
    B: str,
    target_lift: float,
    starts: np.ndarray,
    ends: np.ndarray,
    time_budget_s: float,
    random_seed: int,
) -> Tuple[float, float, int, str]:
    """
    En bloque fijo (ym, canal). No cambia fecha/canal.
    Swaps entre filas de dos tickets para meter B en tickets con A.
    """
    rng = np.random.default_rng(random_seed)
    A = norm_cat(A)
    B = norm_cat(B)

    # ticket -> rows
    sub = df.loc[idxs, [TICKET_COL, "_cat_norm"]]
    ticket_to_rows = sub.groupby(TICKET_COL).indices  # t -> np.array(row_idxs)

    # ticket -> set(categories)
    ticket_set: Dict[str, set] = {}
    for t, rows in ticket_to_rows.items():
        ticket_set[t] = set(df.loc[rows, "_cat_norm"].values)

    l0 = lift_from_ticketset(ticket_set, A, B)
    if np.isnan(l0) or l0 >= target_lift:
        return l0, l0, 0, "already_ok"

    # candidatos
    def refresh_candidates():
        tA = [t for t, s in ticket_set.items() if (A in s and B not in s)]
        tB = [t for t, s in ticket_set.items() if (B in s and A not in s)]
        return np.array(tA, dtype=object), np.array(tB, dtype=object)

    tA, tB = refresh_candidates()
    if len(tA) == 0 or len(tB) == 0:
        return l0, l0, 0, "no_capacity"

    t_start = time.time()
    swaps = 0
    attempts = 0
    stagnant = 0

    # calendario: aquí siempre OK porque es mismo ym; aún así, mantenemos robustez
    ym_block = int(df.loc[idxs[0], "_ym"])

    while (time.time() - t_start) < time_budget_s and attempts < COOC_MAX_ATTEMPTS_PER_RULE:
        attempts += 1

        if attempts % 20_000 == 0:
            tA, tB = refresh_candidates()
            if len(tA) == 0 or len(tB) == 0:
                break

        ta = tA[rng.integers(0, len(tA))]
        tb = tB[rng.integers(0, len(tB))]

        rows_a = ticket_to_rows[ta]
        rows_b = ticket_to_rows[tb]

        cats_a = df.loc[rows_a, "_cat_norm"].values
        cats_b = df.loc[rows_b, "_cat_norm"].values

        # en ticket A: fila que no sea A ni B (para convertirla en B mediante swap)
        cand_a = rows_a[(cats_a != A) & (cats_a != B)]
        if len(cand_a) == 0:
            stagnant += 1
            continue

        # en ticket B: fila que sea B
        cand_b = rows_b[(cats_b == B)]
        if len(cand_b) == 0:
            stagnant += 1
            continue

        ia = int(cand_a[rng.integers(0, len(cand_a))])
        ib = int(cand_b[rng.integers(0, len(cand_b))])

        # check calendario por si acaso (mismo ym, debería ser OK)
        if not (is_active(ym_block, int(starts[ia]), int(ends[ia])) and is_active(ym_block, int(starts[ib]), int(ends[ib]))):
            stagnant += 1
            if stagnant > 30_000:
                break
            continue

        old_cat_a = df.at[ia, "_cat_norm"]  # lo que “sale” de ticket A

        swap_bundles(df, ia, ib)

        # actualiza sets incremental
        ticket_set[ta].discard(old_cat_a)
        ticket_set[ta].add(B)
        ticket_set[tb].discard(B)
        ticket_set[tb].add(old_cat_a)

        swaps += 1
        stagnant = 0

        if swaps % COOC_CHECK_EVERY_SWAPS == 0:
            l_now = lift_from_ticketset(ticket_set, A, B)
            if l_now >= target_lift:
                return l0, l_now, swaps, "target"

    l1 = lift_from_ticketset(ticket_set, A, B)
    return l0, l1, swaps, "partial"


def run_cooc(df: pd.DataFrame, starts: np.ndarray, ends: np.ndarray) -> pd.DataFrame:
    # build rules list
    rules = []
    for A, rels in COOC_REL.items():
        for B, strength in rels.items():
            if strength >= R4_STRONG:
                tgt = 1.12
            elif strength >= R4_MED:
                tgt = 1.08
            else:
                tgt = 1.04
            rules.append((A, B, tgt))

    blocks = df.groupby(["_ym", "_canal"]).indices
    blocks_keys = sorted(blocks.keys())

    print("[COOC] Iniciando (por ym+canal, budget fijo, nunca se atasca)...")
    rows = []
    for k_i, key in enumerate(tqdm(blocks_keys, desc="COOC bloques"), 1):
        ym, canal = int(key[0]), str(key[1])
        idxs = blocks[key]
        if len(idxs) == 0:
            continue

        seed_block = int(ym) * 100 + (0 if canal == "online" else 1)
        for r_i, (A, B, tgt) in enumerate(rules):
            l0, l1, swaps, status = enforce_cooc_block(
                df=df,
                idxs=idxs,
                A=A,
                B=B,
                target_lift=tgt,
                starts=starts,
                ends=ends,
                time_budget_s=COOC_TIME_BUDGET_S_PER_RULE,
                random_seed=seed_block + r_i,
            )
            rows.append({
                "ym": ym,
                "canal": canal,
                "A": norm_cat(A),
                "B": norm_cat(B),
                "target": tgt,
                "lift0": l0,
                "lift1": l1,
                "swaps": swaps,
                "status": status,
            })

    res = pd.DataFrame(rows)
    print("\n[COOC] Top lift1:")
    if not res.empty:
        print(res.sort_values(["lift1", "swaps"], ascending=[False, False]).head(15).to_string(index=False))
    return res


# ============================================================
# VALIDACIÓN FINAL (rápida y útil)
# ============================================================
def validate(df0: pd.DataFrame, df1: pd.DataFrame):
    print("\n[VALID] filas:")
    print("  original:", len(df0))
    print("  final   :", len(df1))
    if len(df0) != len(df1):
        raise ValueError("❌ Cambió el número de filas")

    print("[VALID] columnas iguales:", list(df0.columns) == list(df1.columns))
    if list(df0.columns) != list(df1.columns):
        raise ValueError("❌ Las columnas no coinciden")

    # global counts por categoría (debe ser igual si solo swaps)
    c0 = df0[CAT_COL].map(norm_cat).value_counts().sort_index()
    c1 = df1[CAT_COL].map(norm_cat).value_counts().sort_index()
    diff = (c1 - c0).sort_values()
    changed = diff[diff != 0]
    if len(changed) > 0:
        print("[VALID] ⚠️ cambios globales por categoría (no debería):")
        print(changed.head(20))
        raise ValueError("❌ Cambiaron los totales globales por categoría")
    print("[VALID] ✅ totales globales por categoría OK (swaps puros).")


# ============================================================
# MAIN
# ============================================================
def main():
    t_all = time.time()

    # -------- load items ----------
    df0 = pd.read_csv(INPUT_ITEMS, low_memory=False)
    original_cols = df0.columns.tolist()

    # sanity columns
    for c in [TICKET_COL, DATE_COL, CAT_COL, CANAL_COL, PROV_COL, IDPROD_COL]:
        if c not in df0.columns:
            raise ValueError(f"Falta columna en items_venta.csv: {c}")
    for c in BUNDLE_COLS:
        if c not in df0.columns:
            raise ValueError(f"Falta columna bundle: {c}")

    # normalize group keys (sin tocar columnas originales)
    df = df0.copy()
    df["_ym"] = ym_int_from_date(df[DATE_COL])
    df["_mes"] = (df["_ym"] % 100).astype(np.int16)
    df["_canal"] = df[CANAL_COL].map(norm_key)
    df["_prov"] = df[PROV_COL].map(norm_key)
    df["_zona"] = df["_prov"].map(detect_zone)

    # filtra canales raros (si existieran)
    df["_canal"] = df["_canal"].where(df["_canal"].isin(CANALES_VALIDOS), "otros")

    # cat_norm
    df["_cat_norm"] = df[CAT_COL].map(norm_cat)

    cats_all = sorted(df["_cat_norm"].unique().tolist())
    tie_breaker = {c: i for i, c in enumerate(cats_all)}

    print(f"Filas: {len(df):,} | cats: {len(cats_all)} | bundle_cols: {BUNDLE_COLS}")
    print(f"YM range: {df['_ym'].min()} → {df['_ym'].max()} | canales: {sorted(df['_canal'].unique().tolist())} | zonas: {sorted(df['_zona'].unique().tolist())}")

    # -------- load catalog & calendar arrays ----------
    catalog = load_catalog(INPUT_PRODUCTOS)
    starts, ends = attach_calendar(df, catalog)

    # -------- MIX ----------
    group_idxs = build_group_index(df)
    df = adjust_mix_by_canal_zona(
        df=df,
        group_idxs=group_idxs,
        cats_all=cats_all,
        tie_breaker=tie_breaker,
        starts=starts,
        ends=ends,
        max_swaps_per_group=MIX_MAX_SWAPS_PER_GROUP,
    )

    # -------- COOC ----------
    cooc_res = run_cooc(df, starts, ends)

    # -------- cleanup helper cols ----------
    df_out = df[original_cols].copy()

    # -------- validate ----------
    validate(df0, df_out)

    # -------- save ----------
    OUTPUT_ITEMS.parent.mkdir(parents=True, exist_ok=True)
    df_out.to_csv(OUTPUT_ITEMS, index=False)
    print(f"\nOK: escrito {OUTPUT_ITEMS} (mismas columnas, mismas filas, swaps puros)")

    print(f"\nTOTAL dt={time.time() - t_all:.1f}s")
    # si quieres guardar el resumen cooc también:
    # cooc_res.to_csv("data/cooc_resumen.csv", index=False)


if __name__ == "__main__":
    main()


Filas: 905,445 | cats: 10 | bundle_cols: ['sku', 'id_producto', 'categoría', 'color', 'talla', 'pvp_unitario', 'descuento_pct', 'precio_neto_unit', 'coste_bruto', 'margen_unit', 'promotion_id']
YM range: 201708 → 202509 | canales: ['fisico', 'online'] | zonas: ['centro', 'este', 'islas', 'norte', 'sur']
[MIX] Iniciando ajuste (por canal+zona, monitorizado)...
[MIX] Bloque 1/9 (fisico/centro) | swaps=1,579 | deficit 6,076->0 (100.0%) | dt=240.5s
[MIX] Bloque 2/9 (fisico/este) | swaps=575 | deficit 1,528->0 (100.0%) | dt=90.8s
[MIX] Bloque 3/9 (fisico/norte) | swaps=1,599 | deficit 5,094->0 (100.0%) | dt=248.4s
[MIX] Bloque 4/9 (fisico/sur) | swaps=120 | deficit 355->3 (99.2%) | dt=18.0s
[MIX] Bloque 5/9 (online/centro) | swaps=3,804 | deficit 13,627->5 (100.0%) | dt=601.2s
[MIX] Bloque 6/9 (online/este) | swaps=1,167 | deficit 4,908->7 (99.9%) | dt=178.0s
[MIX] Bloque 7/9 (online/islas) | swaps=353 | deficit 954->4 (99.6%) | dt=53.6s
[MIX] Bloque 8/9 (online/norte) | swaps=2,424 | defic

COOC bloques: 100%|████████████████████████████████████████████████████████████████| 143/143 [1:20:28<00:00, 33.77s/it]



[COOC] Top lift1:
    ym  canal        A        B  target    lift0    lift1  swaps  status
202306 fisico pantalon sudadera    1.04 0.898455 0.910435      3 partial
202306 fisico sudadera pantalon    1.04 0.891792 0.903789      3 partial
202205 fisico pantalon sudadera    1.04 0.853115 0.879500      3 partial
202205 fisico sudadera pantalon    1.04 0.853115 0.870705      2 partial
202206 fisico sudadera camiseta    1.08 0.855873 0.867135      3 partial
202206 fisico   abrigo camiseta    1.04 0.848147 0.863854      3 partial
202309 fisico pantalon sudadera    1.04 0.850874 0.861166      3 partial
202305 fisico sudadera camiseta    1.08 0.854919 0.860856      3 partial
202305 fisico camiseta sudadera    1.04 0.849995 0.855939      3 partial
202206 fisico camiseta sudadera    1.04 0.844612 0.855873      3 partial
202309 fisico sudadera pantalon    1.04 0.844012 0.854305      3 partial
202408 fisico pantalon sudadera    1.04 0.840847 0.850662      3 partial
202408 fisico sudadera pantalon 

In [11]:
from __future__ import annotations

import time
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from tqdm import tqdm


# =========================
# CONFIG
# =========================
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Preferimos el dataset ya ajustado por MIX (para no repetir 3h)
INPUT_PATH_PRIMARY = Path("data/items_venta_ajustado.csv")
INPUT_PATH_FALLBACK = Path("data/items_venta.csv")
OUTPUT_PATH = Path("data/items_venta_cooc.csv")

TICKET_COL = "ticket_id"
DATE_COL   = "fecha_item"
CAT_COL    = "categoría"

# Columnas que SIEMPRE se mueven juntas (bundle coherente)
BUNDLE_COLS = [
    "sku", "id_producto", "categoría", "color", "talla",
    "pvp_unitario", "descuento_pct", "precio_neto_unit",
    "coste_bruto", "margen_unit", "promotion_id"
]

# Intensidades (solo para mapear a targets)
R4_STRONG = 2.5
R4_MED    = 1.6
R4_WEAK   = 0.8

COOC_REL = {
    "camiseta": {"gorra": R4_MED, "calcetines": R4_WEAK, "pantalon": R4_WEAK, "sudadera": R4_WEAK, "calzado": R4_WEAK},
    "sudadera": {"gorra": R4_MED, "camiseta": R4_MED, "pantalon": R4_WEAK, "calzado": R4_WEAK},
    "pantalon": {"cinturon": R4_STRONG, "calcetines": R4_WEAK, "camiseta": R4_WEAK, "sudadera": R4_WEAK, "calzado": R4_WEAK},
    "abrigo":   {"bufanda": R4_STRONG, "gorra": R4_MED, "camiseta": R4_WEAK, "camisa": R4_WEAK, "pantalon": R4_WEAK},
    "calzado":  {"calcetines": R4_STRONG, "pantalon": R4_WEAK, "camiseta": R4_WEAK, "sudadera": R4_WEAK},
    "camisa":   {"cinturon": R4_MED, "pantalon": R4_MED, "abrigo": R4_WEAK},
    "gorra":    {"camiseta": R4_WEAK, "sudadera": R4_WEAK, "bufanda": R4_WEAK},
    "cinturon": {"pantalon": R4_MED, "camisa": R4_WEAK},
    "calcetines":{"calzado": R4_MED, "pantalon": R4_WEAK},
    "bufanda":  {"abrigo": R4_STRONG, "gorra": R4_WEAK},
}

# Targets (ajústalos si quieres más “señal”)
TARGET_STRONG = 1.25
TARGET_MED    = 1.12
TARGET_WEAK   = 1.06

# Control de tiempo por regla (clave: nunca se atasca)
TIME_BUDGET_S_PER_RULE = 0.25  # sube a 0.4 si quieres más empuje, baja a 0.15 si quieres ultrarrápido
MAX_SWAPS_PER_RULE     = 500   # cap duro por regla/bloque


# =========================
# UTIL: normalización categorías
# =========================
def strip_accents(text: str) -> str:
    trans = str.maketrans("áéíóúüñÁÉÍÓÚÜÑ", "aeiouunAEIOUUN")
    return text.translate(trans)

def norm_cat(x) -> str:
    if pd.isna(x):
        return ""
    return strip_accents(str(x).strip().lower())

def map_target(strength: float) -> float:
    if strength >= R4_STRONG:
        return TARGET_STRONG
    if strength >= R4_MED:
        return TARGET_MED
    return TARGET_WEAK


# =========================
# CARGA
# =========================
INPUT_PATH = INPUT_PATH_PRIMARY if INPUT_PATH_PRIMARY.exists() else INPUT_PATH_FALLBACK
df = pd.read_csv(INPUT_PATH, low_memory=False)

# checks mínimos
for col in [TICKET_COL, DATE_COL, CAT_COL]:
    if col not in df.columns:
        raise ValueError(f"Falta columna obligatoria: {col}")
for c in BUNDLE_COLS:
    if c not in df.columns:
        raise ValueError(f"Falta columna bundle: {c}")

# columna auxiliar cat_norm (solo para trabajar)
df["_cat_norm"] = df[CAT_COL].map(norm_cat)

# ym = YYYYMM (respeta calendario real por año-mes)
dt = pd.to_datetime(df[DATE_COL], errors="coerce")
if dt.isna().any():
    raise ValueError("Hay fechas inválidas en fecha_item")
df["_ym"] = (dt.dt.year * 100 + dt.dt.month).astype(int)

# universo de categorías presentes
cats_all = sorted([c for c in df["_cat_norm"].unique().tolist() if c])
cat_to_i = {c: i for i, c in enumerate(cats_all)}

print(f"Input: {INPUT_PATH}")
print(f"Filas: {len(df):,} | cats: {len(cats_all)} | ym range: {df['_ym'].min()} → {df['_ym'].max()}")
print(f"Columnas (finales deben ser estas): {list(df.drop(columns=['_cat_norm','_ym']).columns)}")

# reglas expandidas
rules: List[Tuple[str, str, float]] = []
for A, rels in COOC_REL.items():
    for B, strength in rels.items():
        rules.append((norm_cat(A), norm_cat(B), map_target(strength)))

print(f"Rules COOC: {len(rules)} | budget={TIME_BUDGET_S_PER_RULE}s/regla | max_swaps={MAX_SWAPS_PER_RULE}/regla")


# =========================
# SWAP bundles
# =========================
def swap_bundles(df_: pd.DataFrame, ia: int, ib: int):
    tmp = df_.loc[ia, BUNDLE_COLS].copy()
    df_.loc[ia, BUNDLE_COLS] = df_.loc[ib, BUNDLE_COLS].values
    df_.loc[ib, BUNDLE_COLS] = tmp.values

    # mantener cat_norm coherente con categoría
    df_.at[ia, "_cat_norm"] = norm_cat(df_.at[ia, CAT_COL])
    df_.at[ib, "_cat_norm"] = norm_cat(df_.at[ib, CAT_COL])


# =========================
# Métricas lift rápidas (sobre matriz ticket x cat)
# =========================
def lift_from_matrix(X: np.ndarray, iA: int, iB: int) -> float:
    # X: bool (n_tickets, n_cats)
    A = X[:, iA]
    B = X[:, iB]
    nA = A.sum()
    nB = B.sum()
    if nA == 0 or nB == 0:
        return np.nan
    nAB = (A & B).sum()
    pB = nB / X.shape[0]
    pB_A = nAB / nA
    return (pB_A / pB) if pB > 0 else np.nan


# =========================
# COOC por bloque ym (opti)
# =========================
def build_ticket_struct(block: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, Dict[object, np.ndarray]]:
    """
    Devuelve:
      - tickets: array de ids ticket (n_tickets)
      - X: matriz bool (n_tickets, n_cats) presencia cat por ticket
      - ticket_to_rows: dict ticket -> array de índices (filas originales df)
    Nota: se filtran tickets con >=2 categorías distintas (multi-cat).
    """
    g = block.groupby(TICKET_COL)["_cat_norm"].agg(lambda s: set(s.values))
    g = g[g.map(len) >= 2]
    tickets = g.index.to_numpy()

    if len(tickets) == 0:
        return tickets, np.zeros((0, len(cats_all)), dtype=bool), {}

    # matriz presencia
    X = np.zeros((len(tickets), len(cats_all)), dtype=bool)
    for r, t in enumerate(tickets):
        for c in g.loc[t]:
            if c in cat_to_i:
                X[r, cat_to_i[c]] = True

    # mapping ticket->rows (para swaps)
    ticket_to_rows = block.groupby(TICKET_COL).apply(lambda x: x.index.to_numpy()).to_dict()
    return tickets, X, ticket_to_rows


def enforce_rule_in_block(
    df_: pd.DataFrame,
    block_idxs: np.ndarray,
    A: str, B: str,
    target: float,
    rng: np.random.Generator,
    time_budget_s: float,
    max_swaps: int,
    heartbeat_swaps: int = 50
) -> Dict[str, object]:
    """
    Empuja cooc A->B dentro del bloque (ym) haciendo swaps de bundles entre tickets.
    - Solo usa swaps puros
    - Solo dentro del bloque (respeta calendario)
    - time_budget fijo (no se atasca)
    """
    # si no existen en el universo, fuera
    if A not in cat_to_i or B not in cat_to_i or A == B:
        return {"lift0": np.nan, "lift1": np.nan, "swaps": 0, "status": "skip(no_cat)"}

    iA, iB = cat_to_i[A], cat_to_i[B]

    block = df_.loc[block_idxs, [TICKET_COL, "_cat_norm"]]
    tickets, X, ticket_to_rows = build_ticket_struct(block)

    if X.shape[0] == 0:
        return {"lift0": np.nan, "lift1": np.nan, "swaps": 0, "status": "skip(no_tickets)"}

    l0 = lift_from_matrix(X, iA, iB)
    if np.isnan(l0) or l0 >= target:
        return {"lift0": l0, "lift1": l0, "swaps": 0, "status": "already"}

    # candidatos tickets
    A_mask = X[:, iA]
    B_mask = X[:, iB]
    tA_idx = np.where(A_mask & ~B_mask)[0]
    tB_idx = np.where(B_mask & ~A_mask)[0]
    if len(tA_idx) == 0 or len(tB_idx) == 0:
        return {"lift0": l0, "lift1": l0, "swaps": 0, "status": "no_capacity"}

    t_start = time.time()
    swaps = 0
    stagnant = 0

    while swaps < max_swaps and (time.time() - t_start) < time_budget_s:
        # pick tickets
        ra = int(rng.choice(tA_idx))
        rb = int(rng.choice(tB_idx))
        ta = tickets[ra]
        tb = tickets[rb]

        rows_a = ticket_to_rows.get(ta)
        rows_b = ticket_to_rows.get(tb)
        if rows_a is None or rows_b is None:
            stagnant += 1
            if stagnant > 200:
                break
            continue

        cats_a = df_.loc[rows_a, "_cat_norm"].to_numpy()
        cats_b = df_.loc[rows_b, "_cat_norm"].to_numpy()

        # en ticket A: queremos convertir una fila "OTHER" (no A, no B) a B, trayéndonos un B desde ticket B
        cand_a = rows_a[(cats_a != A) & (cats_a != B)]
        if len(cand_a) == 0:
            stagnant += 1
            if stagnant > 200:
                break
            continue
        ia = int(rng.choice(cand_a))

        # en ticket B: necesitamos una fila B para mandarla al ticket A
        cand_b = rows_b[(cats_b == B)]
        if len(cand_b) == 0:
            stagnant += 1
            if stagnant > 200:
                break
            continue
        ib = int(rng.choice(cand_b))

        if ia == ib:
            continue

        old_a = df_.at[ia, "_cat_norm"]  # cat del "other" que sale del ticket A

        # swap real
        swap_bundles(df_, ia, ib)
        swaps += 1

        # actualizar matriz X incrementalmente (muy barato)
        # ticket A gana B
        X[ra, iB] = True
        # ticket B pierde B si era su único B? (aprox): lo recalculamos solo si hacía falta
        # Para no hacer groupby, comprobamos si aún queda algún B en ticket B tras swap:
        # - ib era B y salió; podría quedar otro B en ticket B -> mirar filas actuales del ticket B
        cats_b_now = df_.loc[rows_b, "_cat_norm"].to_numpy()
        X[rb, iB] = np.any(cats_b_now == B)

        # ticket A pudo perder old_a si era su único (lo miramos con filas actuales del ticket A)
        cats_a_now = df_.loc[rows_a, "_cat_norm"].to_numpy()
        X[ra, cat_to_i[old_a]] = np.any(cats_a_now == old_a)

        # ticket B gana old_a
        X[rb, cat_to_i[old_a]] = True

        stagnant = 0

        if swaps % heartbeat_swaps == 0:
            l_now = lift_from_matrix(X, iA, iB)
            if not np.isnan(l_now) and l_now >= target:
                return {"lift0": l0, "lift1": l_now, "swaps": swaps, "status": "target"}

    l1 = lift_from_matrix(X, iA, iB)
    return {"lift0": l0, "lift1": l1, "swaps": swaps, "status": "partial"}


# =========================
# EJECUCIÓN COOC
# =========================
yms = sorted(df["_ym"].unique().tolist())
print(f"[COOC] Bloques ym: {len(yms)} (calendario OK). Empieza...")

rng_global = np.random.default_rng(RANDOM_SEED)
rows_out = []

t_all = time.time()
pbar = tqdm(yms, desc="COOC por ym", ncols=110)

for k, ym in enumerate(pbar, 1):
    t0 = time.time()
    idxs = df.index[df["_ym"].to_numpy() == ym].to_numpy()
    if len(idxs) == 0:
        continue

    # Pequeño “pre-check” para saltar reglas sin sentido:
    present_cats = set(df.loc[idxs, "_cat_norm"].unique().tolist())

    swaps_ym = 0
    touched = 0

    # rng por bloque
    rng = np.random.default_rng([RANDOM_SEED, int(ym)])

    for (A, B, tgt) in rules:
        if A not in present_cats or B not in present_cats:
            continue

        res = enforce_rule_in_block(
            df, idxs, A, B, tgt,
            rng=rng,
            time_budget_s=TIME_BUDGET_S_PER_RULE,
            max_swaps=MAX_SWAPS_PER_RULE,
            heartbeat_swaps=80
        )
        touched += 1
        swaps_ym += int(res["swaps"])
        rows_out.append({
            "ym": int(ym),
            "A": A, "B": B,
            "target": float(tgt),
            "lift0": res["lift0"],
            "lift1": res["lift1"],
            "swaps": int(res["swaps"]),
            "status": res["status"],
        })

    dt = time.time() - t0
    # actualización visual rápida en barra
    pbar.set_postfix({"rules": touched, "swaps": swaps_ym, "dt_s": f"{dt:.1f}"})


res_df = pd.DataFrame(rows_out)
print("\n[COOC] Resumen top lifts finales (global):")
if len(res_df):
    print(
        res_df.sort_values(["lift1", "swaps"], ascending=False)
              .head(25)
              .to_string(index=False)
    )
else:
    print("No se ejecutó ninguna regla (¿categorías no presentes?).")

print(f"\n[COOC] dt total: {time.time() - t_all:.1f}s")


# =========================
# VALIDACIONES FINALES (rápidas)
# =========================
print("\n[VALID] filas iguales:", len(df), "(no debe cambiar)")
print("[VALID] columnas finales:", list(df.drop(columns=["_cat_norm","_ym"]).columns))

# Guardar CSV final con mismas columnas originales
df_final = df.drop(columns=["_cat_norm","_ym"])
df_final.to_csv(OUTPUT_PATH, index=False)
print(f"\nOK: escrito {OUTPUT_PATH} (mismas filas, mismas columnas, swaps puros)")


Input: data\items_venta_ajustado.csv
Filas: 905,445 | cats: 10 | ym range: 201708 → 202509
Columnas (finales deben ser estas): ['item_id', 'ticket_id', 'pos_item', 'customer_id', 'canal', 'store_id', 'provincia', 'fecha_item', 'sku', 'id_producto', 'categoría', 'color', 'talla', 'pvp_unitario', 'descuento_pct', 'precio_neto_unit', 'coste_bruto', 'margen_unit', 'promotion_id']
Rules COOC: 35 | budget=0.25s/regla | max_swaps=500/regla
[COOC] Bloques ym: 98 (calendario OK). Empieza...


COOC por ym:   0%|                                                                     | 0/98 [00:00<?, ?it/s]C:\Users\PEDRO\AppData\Local\Temp\ipykernel_10188\3274905305.py:178: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ticket_to_rows = block.groupby(TICKET_COL).apply(lambda x: x.index.to_numpy()).to_dict()
C:\Users\PEDRO\AppData\Local\Temp\ipykernel_10188\3274905305.py:178: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this wa


[COOC] Resumen top lifts finales (global):
    ym          A          B  target    lift0    lift1  swaps  status
202212    bufanda      gorra    1.06 1.033010 1.446214      2 partial
202301    bufanda      gorra    1.06 0.978389 1.369745      2 partial
202409    bufanda      gorra    1.06 0.904321 1.356481      2 partial
202310    bufanda      gorra    1.06 0.871198 1.306798      2 partial
202411    bufanda      gorra    1.06 0.966925 1.289233      2 partial
202501    bufanda      gorra    1.06 0.962316 1.283088      2 partial
202210    bufanda      gorra    1.06 0.850979 1.276468      2 partial
202401    bufanda      gorra    1.06 0.847618 1.271427      2 partial
202412    bufanda      gorra    1.06 0.906125 1.268576      2 partial
202207   cinturon     camisa    1.06 0.897101 1.196135      2 partial
202502    bufanda      gorra    1.06 0.993459 1.192151      2 partial
202402    bufanda      gorra    1.06 0.984618 1.181542      2 partial
202310 calcetines    calzado    1.12 1.073747 

In [12]:
# res_df es el DataFrame de resultados COOC que ya generaste

print("Total reglas evaluadas:", len(res_df))
print(res_df["status"].value_counts())

print("\nPorcentaje por estado:")
print((res_df["status"].value_counts(normalize=True) * 100).round(2))


Total reglas evaluadas: 2189
status
partial    2188
already       1
Name: count, dtype: int64

Porcentaje por estado:
status
partial    99.95
already     0.05
Name: proportion, dtype: float64


In [13]:
no_cap = res_df[res_df["status"] == "no_capacity"]

print("Reglas sin capacidad:", len(no_cap))
print("\nTop 10 reglas sin capacidad:")
print(
    no_cap.groupby(["A", "B"])
          .size()
          .sort_values(ascending=False)
          .head(10)
)

Reglas sin capacidad: 0

Top 10 reglas sin capacidad:
Series([], dtype: int64)


In [14]:
res_df["delta_lift"] = res_df["lift1"] - res_df["lift0"]

strong_pairs = [
    ("abrigo", "bufanda"),
    ("bufanda", "abrigo"),
    ("calzado", "calcetines"),
    ("calcetines", "calzado"),
    ("pantalon", "cinturon"),
    ("cinturon", "pantalon"),
]

strong_df = res_df[
    res_df[["A","B"]].apply(tuple, axis=1).isin(strong_pairs)
]

print(
    strong_df.groupby(["A","B"])
             .agg(
                 n_reglas=("delta_lift","count"),
                 lift0_mean=("lift0","mean"),
                 lift1_mean=("lift1","mean"),
                 delta_mean=("delta_lift","mean"),
             )
             .sort_values("delta_mean", ascending=False)
)


                       n_reglas  lift0_mean  lift1_mean  delta_mean
A          B                                                       
calcetines calzado           35    0.650350    0.767131    0.116781
calzado    calcetines        35    0.544471    0.660106    0.115635
cinturon   pantalon          49    0.605767    0.636175    0.030408
pantalon   cinturon          49    0.579223    0.609525    0.030302
abrigo     bufanda           37    0.642567    0.669053    0.026486
bufanda    abrigo            37    0.661146    0.687581    0.026435


In [16]:
import pandas as pd
import numpy as np

df_tmp = pd.read_csv("data/items_venta_cooc.csv", low_memory=False)

df_tmp["ym"] = pd.to_datetime(df_tmp["fecha_item"], errors="coerce").dt.strftime("%Y%m")

# normalizar categoría (sin acentos)
df_tmp["_cat"] = (
    df_tmp["categoría"].astype(str)
    .str.lower()
    .str.normalize("NFD")
    .str.replace(r"[\u0300-\u036f]", "", regex=True)
)

ticket_sets = (
    df_tmp.groupby(["ym", "ticket_id"])["_cat"]
          .nunique()
          .reset_index(name="n_cats")
)

g = ticket_sets.groupby("ym")["n_cats"]

out = pd.DataFrame({
    "tickets": g.count(),
    "multi_item": g.apply(lambda x: (x > 1).sum()),
    "pct_multi": g.apply(lambda x: (x > 1).mean())
}).reset_index()

print(out.head(15))


        ym  tickets  multi_item  pct_multi
0   201708      397         130   0.327456
1   201709      519         172   0.331407
2   201710      554         163   0.294224
3   201711      785         242   0.308280
4   201712      795         252   0.316981
5   201801      538         161   0.299257
6   201802      489         149   0.304703
7   201803      539         164   0.304267
8   201804      554         157   0.283394
9   201805      587         159   0.270869
10  201806      647         189   0.292117
11  201807      774         243   0.313953
12  201808      605         154   0.254545
13  201809      772         264   0.341969
14  201810      829         254   0.306393


In [17]:
df_tmp = pd.read_csv("data/items_venta_cooc.csv", low_memory=False)

df_tmp["ym"] = pd.to_datetime(df_tmp["fecha_item"], errors="coerce").dt.strftime("%Y%m")
df_tmp["canal"] = df_tmp["canal"].astype(str).str.strip().str.lower()

df_tmp["_cat"] = (
    df_tmp["categoría"].astype(str)
    .str.lower()
    .str.normalize("NFD")
    .str.replace(r"[\u0300-\u036f]", "", regex=True)
)

ticket_sets = (
    df_tmp.groupby(["ym", "canal", "ticket_id"])["_cat"]
          .nunique()
          .reset_index(name="n_cats")
)

g = ticket_sets.groupby(["ym","canal"])["n_cats"]

out2 = pd.DataFrame({
    "tickets": g.count(),
    "multi_item": g.apply(lambda x: (x > 1).sum()),
    "pct_multi": g.apply(lambda x: (x > 1).mean())
}).reset_index()

print(out2.head(20))


        ym   canal  tickets  multi_item  pct_multi
0   201708  online      397         130   0.327456
1   201709  online      519         172   0.331407
2   201710  online      554         163   0.294224
3   201711  online      785         242   0.308280
4   201712  online      795         252   0.316981
5   201801  online      538         161   0.299257
6   201802  online      489         149   0.304703
7   201803  online      539         164   0.304267
8   201804  online      554         157   0.283394
9   201805  online      587         159   0.270869
10  201806  online      647         189   0.292117
11  201807  online      774         243   0.313953
12  201808  online      605         154   0.254545
13  201809  online      772         264   0.341969
14  201810  online      829         254   0.306393
15  201811  online     1092         326   0.298535
16  201812  online     1103         397   0.359927
17  201901  online     1370         456   0.332847
18  201902  online     1261    

In [18]:
cat_dist = (
    df.groupby("categoría")
      .size()
      .reset_index(name="n_items")
)

cat_dist["pct"] = cat_dist["n_items"] / cat_dist["n_items"].sum() * 100
cat_dist = cat_dist.sort_values("pct", ascending=False)

cat_dist


,categoría,n_items,pct
5,Camiseta,256193,28.294706
0,Abrigo,226265,24.989370
8,Pantalón,192108,21.216971
9,Sudadera,85883,9.485170
4,Camisa,56172,6.203800
3,Calzado,40252,4.445549
7,Gorra,15612,1.724235
6,Cinturón,15263,1.685690
1,Bufanda,9322,1.029549
2,Calcetines,8375,0.924960


In [19]:
from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path

# =========================
# CONFIG
# =========================
ITEMS_PATH = Path("data/items_venta_cooc.csv")     # <- tu salida final
CATALOG_PATH = Path("data/productos.csv")         # <- catálogo
ORIGINAL_PATH = Path("data/items_venta.csv")      # <- opcional (para comparar filas/columnas)

EXPECTED_COLS = [
    "item_id","ticket_id","pos_item","customer_id","canal","store_id","provincia","fecha_item",
    "sku","id_producto","categoría","color","talla",
    "pvp_unitario","descuento_pct","precio_neto_unit","coste_bruto","margen_unit","promotion_id"
]

CANALES_VALIDOS = {"online", "fisico"}

# tolerancias numéricas (por si hay floats)
TOL_PRICE = 1e-6

# cuántos ejemplos guardar por tipo de error
SAMPLE_N = 200

# =========================
# HELPERS
# =========================
def norm_text(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.lower()
         .str.normalize("NFD")
         .str.replace(r"[\u0300-\u036f]", "", regex=True)
    )

def ym_to_int(ym: str) -> int:
    # "YYYY-MM" -> YYYYMM int
    if ym is None or (isinstance(ym, float) and np.isnan(ym)):
        return None
    ym = str(ym).strip()
    if ym.lower() in ("none", "", "nan"):
        return None
    return int(ym.replace("-", ""))

def to_ym_int_from_date(d: pd.Series) -> pd.Series:
    dt = pd.to_datetime(d, errors="coerce")
    return (dt.dt.year * 100 + dt.dt.month).astype("Int64")

def report_block(title: str, ok: bool, details: str = ""):
    status = "✅ OK" if ok else "❌ FAIL"
    print(f"\n[{status}] {title}")
    if details:
        print(details)

# =========================
# LOAD
# =========================
df = pd.read_csv(ITEMS_PATH, low_memory=False)
prod = pd.read_csv(CATALOG_PATH, low_memory=False)

print(f"Items: {ITEMS_PATH} | filas={len(df):,} cols={df.shape[1]}")
print(f"Catálogo: {CATALOG_PATH} | filas={len(prod):,} cols={prod.shape[1]}")

# =========================
# 1) COLUMNAS EXACTAS
# =========================
cols_ok = list(df.columns) == EXPECTED_COLS
if not cols_ok:
    missing = [c for c in EXPECTED_COLS if c not in df.columns]
    extra   = [c for c in df.columns if c not in EXPECTED_COLS]
    report_block(
        "Columnas EXACTAS como el CSV base",
        False,
        f"Missing: {missing}\nExtra: {extra}\nOrden actual: {list(df.columns)}"
    )
else:
    report_block("Columnas EXACTAS como el CSV base", True)

# =========================
# 2) CHECKS CLAVE DE INTEGRIDAD
# =========================
# item_id único
dup_item = df["item_id"].duplicated(keep=False)
report_block(
    "item_id único",
    ok=not dup_item.any(),
    details=f"Duplicados: {dup_item.sum():,}" if dup_item.any() else ""
)

# (ticket_id, pos_item) único
dup_tp = df.duplicated(subset=["ticket_id", "pos_item"], keep=False)
report_block(
    "(ticket_id, pos_item) único",
    ok=not dup_tp.any(),
    details=f"Duplicados: {dup_tp.sum():,}" if dup_tp.any() else ""
)

# canal válido
canal_norm = df["canal"].astype(str).str.strip().str.lower()
bad_canal = ~canal_norm.isin(CANALES_VALIDOS)
report_block(
    "canal dentro de {'online','fisico'}",
    ok=not bad_canal.any(),
    details=f"Filas con canal inválido: {bad_canal.sum():,} | valores: {sorted(canal_norm[bad_canal].unique().tolist())[:20]}"
    if bad_canal.any() else ""
)

# fecha válida
dt = pd.to_datetime(df["fecha_item"], errors="coerce")
bad_date = dt.isna()
report_block(
    "fecha_item parseable",
    ok=not bad_date.any(),
    details=f"Fechas inválidas: {bad_date.sum():,}" if bad_date.any() else ""
)

# =========================
# 3) SKU COHERENTE CON id_producto
#    ejemplo: P003-BRN-XS -> id_producto debe ser P003
# =========================
sku_prefix = df["sku"].astype(str).str.split("-", n=1, expand=True)[0]
bad_sku = sku_prefix != df["id_producto"].astype(str)
report_block(
    "SKU prefijo coincide con id_producto",
    ok=not bad_sku.any(),
    details=f"Filas incoherentes: {bad_sku.sum():,}" if bad_sku.any() else ""
)

# =========================
# 4) CATÁLOGO: id_producto existe + categoría coincide
# =========================
# normalizamos nombres de columnas del catálogo (por si difiere en mayúsculas)
prod_cols = {c.lower(): c for c in prod.columns}
req_prod_cols = ["id_producto", "categoria", "lanzamiento", "retiro", "precio", "coste bruto"]
missing_prod = [c for c in req_prod_cols if c not in prod_cols]
if missing_prod:
    raise ValueError(f"En productos.csv faltan columnas requeridas: {missing_prod}. Columnas actuales: {list(prod.columns)}")

idp_col = prod_cols["id_producto"]
catp_col = prod_cols["categoria"]
lan_col = prod_cols["lanzamiento"]
ret_col = prod_cols["retiro"]
price_col = prod_cols["precio"]
cost_col = prod_cols["coste bruto"]

prod_small = prod[[idp_col, catp_col, lan_col, ret_col, price_col, cost_col]].copy()
prod_small = prod_small.rename(columns={
    idp_col: "id_producto",
    catp_col: "_cat_prod",
    lan_col: "_lan",
    ret_col: "_ret",
    price_col: "_precio",
    cost_col: "_coste"
})

# normalizar categoría de catálogo y de items
df["_cat_item"] = norm_text(df["categoría"])
prod_small["_cat_prod"] = norm_text(prod_small["_cat_prod"])

merged = df.merge(prod_small, on="id_producto", how="left", validate="many_to_one")

missing_idp = merged["_cat_prod"].isna()
report_block(
    "id_producto existe en productos.csv",
    ok=not missing_idp.any(),
    details=f"Filas con id_producto no encontrado: {missing_idp.sum():,}" if missing_idp.any() else ""
)

# categoría coincide (solo donde exista id_producto)
cat_mismatch = (~missing_idp) & (merged["_cat_item"] != merged["_cat_prod"])
report_block(
    "categoría del item coincide con categoría del catálogo",
    ok=not cat_mismatch.any(),
    details=f"Filas con mismatch de categoría: {cat_mismatch.sum():,}" if cat_mismatch.any() else ""
)

# =========================
# 5) CALENDARIO: fecha_item dentro de [lanzamiento, retiro]
#    lanzamiento: "YYYY-MM"
#    retiro: "YYYY-MM" o None
# =========================
merged["_ym_item"] = to_ym_int_from_date(merged["fecha_item"])
merged["_ym_lan"]  = merged["_lan"].map(ym_to_int).astype("Int64")
merged["_ym_ret"]  = merged["_ret"].map(ym_to_int).astype("Int64")

# reglas:
# ym_item >= ym_lan
# si hay ret: ym_item <= ym_ret
cal_bad = (~missing_idp) & (
    (merged["_ym_item"] < merged["_ym_lan"]) |
    ((~merged["_ym_ret"].isna()) & (merged["_ym_item"] > merged["_ym_ret"]))
)
report_block(
    "Calendario OK (fecha_item dentro de lanzamiento/retiro del producto)",
    ok=not cal_bad.any(),
    details=f"Filas fuera de calendario: {cal_bad.sum():,}" if cal_bad.any() else ""
)

# =========================
# 6) PRECIOS / COSTES vs catálogo (opcional pero recomendado)
# =========================
# pvp_unitario ~ precio
price_bad = (~missing_idp) & (merged["pvp_unitario"].astype(float) - merged["_precio"].astype(float)).abs() > TOL_PRICE
report_block(
    "pvp_unitario coincide con precio de catálogo (tolerancia)",
    ok=not price_bad.any(),
    details=f"Filas con pvp_unitario != precio catálogo: {price_bad.sum():,}" if price_bad.any() else ""
)

# coste_bruto ~ coste bruto
cost_bad = (~missing_idp) & (merged["coste_bruto"].astype(float) - merged["_coste"].astype(float)).abs() > TOL_PRICE
report_block(
    "coste_bruto coincide con coste bruto de catálogo (tolerancia)",
    ok=not cost_bad.any(),
    details=f"Filas con coste_bruto != coste catálogo: {cost_bad.sum():,}" if cost_bad.any() else ""
)

# =========================
# 7) (OPCIONAL) Comparación con original: mismas filas/columnas
# =========================
if ORIGINAL_PATH.exists():
    df0 = pd.read_csv(ORIGINAL_PATH, low_memory=False)
    report_block("Mismas filas que el original", ok=(len(df0) == len(df)), details=f"original={len(df0):,} final={len(df):,}")
    report_block("Mismas columnas que el original", ok=(list(df0.columns) == list(df.columns)))
else:
    print(f"\n[INFO] No existe {ORIGINAL_PATH} (saltando comparación con original).")

# =========================
# 8) EXPORTAR EJEMPLOS DE ERRORES (si los hay)
# =========================
out_dir = Path("data/validacion")
out_dir.mkdir(parents=True, exist_ok=True)

def save_bad(mask: pd.Series, name: str):
    if mask.any():
        cols_show = EXPECTED_COLS + ["_cat_item","_cat_prod","_lan","_ret","_ym_item","_ym_lan","_ym_ret","_precio","_coste"]
        cols_show = [c for c in cols_show if c in merged.columns]
        merged.loc[mask, cols_show].head(SAMPLE_N).to_csv(out_dir / f"{name}.csv", index=False)
        print(f"[SAVE] {name}.csv -> {out_dir} (muestras={min(SAMPLE_N, mask.sum())})")

save_bad(dup_item, "err_dup_item_id")
save_bad(dup_tp, "err_dup_ticket_pos")
save_bad(bad_canal, "err_bad_canal")
save_bad(bad_date, "err_bad_fecha")
save_bad(bad_sku, "err_bad_sku_vs_id_producto")
save_bad(missing_idp, "err_id_producto_no_en_catalogo")
save_bad(cat_mismatch, "err_categoria_no_coincide_catalogo")
save_bad(cal_bad, "err_fuera_calendario")
save_bad(price_bad, "err_pvp_no_coincide")
save_bad(cost_bad, "err_coste_no_coincide")

# =========================
# 9) RESUMEN FINAL (aprobado / no)
# =========================
fail_counts = {
    "dup_item_id": int(dup_item.sum()),
    "dup_ticket_pos": int(dup_tp.sum()),
    "bad_canal": int(bad_canal.sum()),
    "bad_fecha": int(bad_date.sum()),
    "bad_sku": int(bad_sku.sum()),
    "missing_id_producto": int(missing_idp.sum()),
    "cat_mismatch": int(cat_mismatch.sum()),
    "fuera_calendario": int(cal_bad.sum()),
    "pvp_mismatch": int(price_bad.sum()),
    "cost_mismatch": int(cost_bad.sum()),
}
summary = pd.Series(fail_counts).sort_values(ascending=False)
print("\n===== RESUMEN ERRORES =====")
print(summary)

if summary.sum() == 0 and cols_ok:
    print("\n✅ TODO PERFECTO: CSV consistente con catálogo, calendario, sku/id_producto y columnas.")
else:
    print("\n⚠️ Hay problemas. Revisa data/validacion/*.csv para ver ejemplos concretos.")


Items: data\items_venta_cooc.csv | filas=905,445 cols=19
Catálogo: data\productos.csv | filas=69 cols=7

[✅ OK] Columnas EXACTAS como el CSV base

[✅ OK] item_id único

[✅ OK] (ticket_id, pos_item) único

[✅ OK] canal dentro de {'online','fisico'}

[✅ OK] fecha_item parseable

[✅ OK] SKU prefijo coincide con id_producto

[✅ OK] id_producto existe en productos.csv

[✅ OK] categoría del item coincide con categoría del catálogo

[❌ FAIL] Calendario OK (fecha_item dentro de lanzamiento/retiro del producto)
Filas fuera de calendario: 97

[✅ OK] pvp_unitario coincide con precio de catálogo (tolerancia)

[✅ OK] coste_bruto coincide con coste bruto de catálogo (tolerancia)

[✅ OK] Mismas filas que el original
original=905,445 final=905,445

[❌ FAIL] Mismas columnas que el original
[SAVE] err_fuera_calendario.csv -> data\validacion (muestras=97)

===== RESUMEN ERRORES =====
fuera_calendario       97
dup_item_id             0
dup_ticket_pos          0
bad_canal               0
bad_sku           

# items_3

In [26]:
INPUT_CSV = Path("data/items_venta_cooc.csv")
OUTPUT_CSV = Path("data/items_tallas_ajustadas.csv")
RANDOM_SEED = 42


def strip_accents_lower(x: object) -> str:
    """
    Normaliza texto para comparaciones robustas:
    elimina acentos, recorta espacios y pasa a minúsculas.
    Se usa para provincia/categoría cuando vienen con variaciones de escritura.
    """
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    s = str(x).strip()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    return s.lower()


def find_column(df: pd.DataFrame, logical_name: str, candidates: list[str], required: bool = True) -> str | None:
    """
    Localiza una columna en un CSV “realista” (nombres variables):
    primero busca coincidencia exacta, luego case-insensitive y finalmente “contiene”.
    """
    cols = list(df.columns)

    for c in candidates:
        if c in cols:
            return c

    lower_map = {c.lower(): c for c in cols}
    for c in candidates:
        hit = lower_map.get(c.lower())
        if hit is not None:
            return hit

    for real in cols:
        rlow = real.lower()
        for c in candidates:
            if c.lower() in rlow:
                return real

    if required:
        raise KeyError(f"No se encontró columna para '{logical_name}'. Columnas disponibles: {cols}")
    return None


# Categorías lógicas para decidir el tipo de talla
ROPA_CATS = {"camiseta", "sudadera", "pantalon", "abrigo", "camisa"}
CALZADO_CAT = "calzado"
ONESIZE_CATS = {"gorra", "cinturon", "calcetines", "bufanda"}

# Tallas de ropa y distribución base global (sesgo hacia M y L)
ROPA_TALLAS = ["XS", "S", "M", "L", "XL"]
BASE_ROPA_PROBS = {"XS": 0.03, "S": 0.12, "M": 0.38, "L": 0.32, "XL": 0.15}

# Ajuste por “zona” (modelo simple: ciudad vs norte_rural vs sur)
ZONE_MULTIPLIERS = {
    "ciudad": {"XS": 1.0, "S": 1.05, "M": 1.0, "L": 0.95, "XL": 0.95},
    "norte_rural": {"XS": 0.7, "S": 0.9, "M": 1.0, "L": 1.2, "XL": 1.4},
    "sur": {"XS": 1.0, "S": 1.1, "M": 1.1, "L": 0.9, "XL": 0.8},
}

# Ajuste por categoría (por ejemplo, abrigo tiende a tallar “más grande”)
CATEGORY_MULTIPLIERS = {
    "abrigo": {"XS": 0.5, "S": 0.7, "M": 1.0, "L": 1.2, "XL": 1.4},
    "sudadera": {"XS": 1.2, "S": 0.9, "M": 0.9, "L": 1.0, "XL": 1.2},
    "camiseta": {"XS": 0.8, "S": 1.0, "M": 1.1, "L": 1.1, "XL": 1.0},
    "camisa": {"XS": 0.8, "S": 1.0, "M": 1.1, "L": 1.1, "XL": 1.0},
    "pantalon": {"XS": 0.8, "S": 1.0, "M": 1.1, "L": 1.1, "XL": 1.0},
}

# Tallas de calzado (distribución global) y distribución condicionada por talla de ropa
CALZADO_TALLAS = [39, 40, 41, 42, 43, 44, 45]
CALZADO_BASE_PROBS = {39: 0.05, 40: 0.10, 41: 0.20, 42: 0.25, 43: 0.20, 44: 0.12, 45: 0.08}
_s = sum(CALZADO_BASE_PROBS.values())
CALZADO_BASE_PROBS = {k: v / _s for k, v in CALZADO_BASE_PROBS.items()}

# Si conocemos la talla de ropa del cliente, sesgamos el calzado a un rango coherente
COND_CALZADO_PROBS = {
    "XS": {39: 0.50, 40: 0.30, 41: 0.15, 42: 0.05, 43: 0.00, 44: 0.00, 45: 0.00},
    "S":  {39: 0.10, 40: 0.35, 41: 0.30, 42: 0.15, 43: 0.07, 44: 0.03, 45: 0.00},
    "M":  {39: 0.02, 40: 0.08, 41: 0.30, 42: 0.30, 43: 0.18, 44: 0.08, 45: 0.04},
    "L":  {39: 0.00, 40: 0.03, 41: 0.12, 42: 0.28, 43: 0.28, 44: 0.17, 45: 0.12},
    "XL": {39: 0.00, 40: 0.00, 41: 0.05, 42: 0.17, 43: 0.28, 44: 0.25, 45: 0.25},
}

# Clasificación de provincias a zonas con una regla simple y reproducible
CIUDAD_PROVINCIAS = {"madrid", "barcelona", "valencia", "sevilla", "zaragoza", "malaga", "bilbao"}
NORTE_RURAL_PROVINCIAS = {
    "a coruna", "la coruna", "coruna", "lugo", "ourense", "orense", "pontevedra",
    "asturias", "cantabria", "bizkaia", "vizcaya", "gipuzkoa", "guipuzcoa",
    "araba", "alava", "navarra", "la rioja", "rioja", "burgos", "leon", "palencia",
    "zamora", "soria", "huesca", "lleida", "lerida", "girona", "gerona",
}


def normalize_category(raw: object) -> str:
    """
    Lleva categorías “sucias” a un conjunto controlado.
    Esto evita que el modelo de tallas dependa de variantes de texto del catálogo.
    """
    base = strip_accents_lower(raw)

    if base.startswith("camiseta"):
        return "camiseta"
    if base.startswith("sudadera"):
        return "sudadera"
    if base.startswith("pantalon") or base.startswith("pantal"):
        return "pantalon"
    if base.startswith("abrigo"):
        return "abrigo"
    if base.startswith("camisa"):
        return "camisa"
    if base.startswith("calzado") or base.startswith("zapat") or "sneaker" in base:
        return "calzado"
    if "gorra" in base or "cap" in base:
        return "gorra"
    if "cinturon" in base or "cintur" in base:
        return "cinturon"
    if "calcetin" in base or "calcet" in base:
        return "calcetines"
    if "bufanda" in base or "scarf" in base:
        return "bufanda"
    return "otro"


def detect_zone(provincia: object) -> str:
    """
    Clasifica provincia en una zona simple.
    La zona se usa para sesgar la distribución de tallas sin añadir columnas al dataset final.
    """
    p = strip_accents_lower(provincia)
    if p in CIUDAD_PROVINCIAS:
        return "ciudad"
    if p in NORTE_RURAL_PROVINCIAS:
        return "norte_rural"
    return "sur"


def ropa_probs(zona: str, cat_norm: str) -> np.ndarray:
    """
    Construye la distribución final de tallas de ropa combinando:
    base global × ajuste por zona × ajuste por categoría, y normaliza a suma 1.
    """
    z = zona if zona in ZONE_MULTIPLIERS else "sur"
    cat_mult = CATEGORY_MULTIPLIERS.get(cat_norm)

    weights = np.empty(len(ROPA_TALLAS), dtype=float)
    for i, talla in enumerate(ROPA_TALLAS):
        w = BASE_ROPA_PROBS[talla] * ZONE_MULTIPLIERS[z][talla]
        if cat_mult is not None:
            w *= cat_mult[talla]
        weights[i] = w

    weights /= weights.sum()
    return weights


def sku_token(talla: object) -> str:
    """
    Convierte talla a token de SKU:
    ropa → XS..XL, calzado → número, OneSize → OS.
    """
    if talla is None or (isinstance(talla, float) and np.isnan(talla)):
        return ""
    t = str(talla).strip()
    if t.lower() == "onesize":
        return "OS"
    if t.isdigit():
        return t
    return t.upper()


def update_sku(sku: object, talla: object) -> object:
    """
    Sustituye únicamente el último token del SKU por la talla.
    Mantiene intacta la parte identificativa del producto.
    """
    if pd.isna(sku) or pd.isna(talla):
        return sku

    sku_str = str(sku)
    token = sku_token(talla)

    if "-" in sku_str:
        parts = sku_str.split("-")
        parts[-1] = token
        return "-".join(parts)

    if "_" in sku_str:
        parts = sku_str.split("_")
        parts[-1] = token
        return "_".join(parts)

    return f"{sku_str}-{token}"


def sample_roba_sizes_per_customer(df: pd.DataFrame, rng: np.random.Generator) -> dict:
    """
    Asigna una talla de ropa fija por cliente cuando existe customer_id.
    Esto genera coherencia inter-temporal: un cliente no cambia de talla entre compras.
    """
    ropa = df[df["cat_norm"].isin(ROPA_CATS)]
    if ropa.empty or "customer_id" not in ropa.columns:
        return {}

    perfil = (
        ropa.dropna(subset=["customer_id"])
            .groupby("customer_id", sort=False)
            .agg(
                zona_mode=("zona", lambda s: s.mode(dropna=True).iat[0] if not s.mode(dropna=True).empty else "sur"),
                cat_mode=("cat_norm", lambda s: s.mode(dropna=True).iat[0] if not s.mode(dropna=True).empty else "camiseta"),
            )
    )

    out: dict[str, str] = {}
    for cid, row in perfil.iterrows():
        p = ropa_probs(row["zona_mode"], row["cat_mode"])
        out[cid] = rng.choice(ROPA_TALLAS, p=p)
    return out


def sample_shoe_sizes_per_customer(df: pd.DataFrame, rng: np.random.Generator, ropa_by_customer: dict | None) -> dict:
    """
    Asigna talla de calzado fija por cliente.
    Si existe talla de ropa previa, usa una distribución condicionada; si no, usa el patrón global.
    """
    calzado = df[df["cat_norm"].eq(CALZADO_CAT)]
    if calzado.empty or "customer_id" not in calzado.columns:
        return {}

    cids = calzado["customer_id"].dropna().unique()
    if len(cids) == 0:
        return {}

    out: dict[str, int] = {}
    base_sizes = np.array(CALZADO_TALLAS, dtype=int)
    base_probs = np.array([CALZADO_BASE_PROBS[t] for t in CALZADO_TALLAS], dtype=float)

    for cid in cids:
        ropa_t = ropa_by_customer.get(cid) if ropa_by_customer else None
        cond = COND_CALZADO_PROBS.get(ropa_t)

        if cond is None:
            out[cid] = int(rng.choice(base_sizes, p=base_probs))
            continue

        sizes = np.array(list(cond.keys()), dtype=int)
        probs = np.array([cond[k] for k in sizes], dtype=float)
        out[cid] = int(rng.choice(sizes, p=probs))

    return out


def validate_sizes_and_sku(df: pd.DataFrame) -> None:
    """
    Garantiza que el output cumple contrato:
    tallas válidas por categoría y coherencia entre la talla y el último token del SKU.
    """
    mask_ropa = df["cat_norm"].isin(ROPA_CATS)
    mask_calzado = df["cat_norm"].eq(CALZADO_CAT)
    mask_os = df["cat_norm"].isin(ONESIZE_CATS)

    ropa_talla = df.loc[mask_ropa, "talla"].astype("string").str.upper()
    if not ropa_talla.isin(ROPA_TALLAS).all():
        bad = ropa_talla[~ropa_talla.isin(ROPA_TALLAS)].value_counts(dropna=False).head()
        raise ValueError(f"Tallas de ropa fuera de XS/S/M/L/XL. Ejemplos: {bad.to_dict()}")

    calzado_talla = df.loc[mask_calzado, "talla"].astype("string").str.replace(r"\.0$", "", regex=True)
    valid_shoes = pd.Index([str(x) for x in CALZADO_TALLAS])
    if not calzado_talla.isin(valid_shoes).all():
        bad = calzado_talla[~calzado_talla.isin(valid_shoes)].value_counts(dropna=False).head()
        raise ValueError(f"Tallas de calzado fuera de 39–45. Ejemplos: {bad.to_dict()}")

    os_talla = df.loc[mask_os, "talla"].astype("string")
    if not os_talla.isin(["OneSize"]).all():
        bad = os_talla[~os_talla.isin(["OneSize"])].value_counts(dropna=False).head()
        raise ValueError(f"OneSize con talla distinta de 'OneSize'. Ejemplos: {bad.to_dict()}")

    def last_token(x: object) -> str:
        s = "" if x is None else str(x)
        if "-" in s:
            return s.split("-")[-1]
        if "_" in s:
            return s.split("_")[-1]
        return s

    last = df["sku"].astype(str).map(last_token)

    if mask_ropa.any() and not (last[mask_ropa] == ropa_talla).all():
        raise ValueError("Inconsistencias entre SKU y talla en ropa.")
    if mask_calzado.any() and not (last[mask_calzado] == calzado_talla).all():
        raise ValueError("Inconsistencias entre SKU y talla en calzado.")
    if mask_os.any():
        tok = last[mask_os].astype("string").str.upper()
        if not tok.isin(["OS", "ONESIZE"]).all():
            raise ValueError("Inconsistencias entre SKU y talla en OneSize.")


def main() -> None:
    df = pd.read_csv(INPUT_CSV, low_memory=False)
    original_cols = list(df.columns)

    categoria_col = find_column(
        df,
        "categoria",
        ["categoria", "categoría", "category", "categoria_final", "categoria_ajustada", "cat"],
    )
    provincia_col = find_column(
        df,
        "provincia",
        ["provincia", "provincia_ticket", "province"],
    )

    invariant_cols = [c for c in original_cols if c not in ("talla", "sku")]
    invariants_snapshot = df[invariant_cols].copy()

    df["cat_norm"] = df[categoria_col].map(normalize_category)
    df["zona"] = df[provincia_col].map(detect_zone)

    rng = np.random.default_rng(RANDOM_SEED)

    ropa_by_customer = sample_roba_sizes_per_customer(df, rng)
    shoe_by_customer = sample_shoe_sizes_per_customer(df, rng, ropa_by_customer)

    if "talla" not in df.columns:
        df["talla"] = pd.NA
        original_cols.append("talla")

    df["talla"] = df["talla"].astype("string")

    mask_ropa = df["cat_norm"].isin(ROPA_CATS)
    mask_calzado = df["cat_norm"].eq(CALZADO_CAT)
    mask_os = df["cat_norm"].isin(ONESIZE_CATS)

    if ropa_by_customer:
        df.loc[mask_ropa, "talla"] = df.loc[mask_ropa, "customer_id"].map(ropa_by_customer).astype("string")

    if shoe_by_customer:
        df.loc[mask_calzado, "talla"] = df.loc[mask_calzado, "customer_id"].map(shoe_by_customer).astype("string")

    df.loc[mask_os, "talla"] = "OneSize"

    missing_ropa = mask_ropa & df["talla"].isna()
    if missing_ropa.any():
        sub = df.loc[missing_ropa, ["zona", "cat_norm"]]
        sampled = [
            rng.choice(ROPA_TALLAS, p=ropa_probs(z, c))
            for z, c in zip(sub["zona"].tolist(), sub["cat_norm"].tolist())
        ]
        df.loc[missing_ropa, "talla"] = pd.Series(sampled, index=df.index[missing_ropa], dtype="string")

    missing_shoes = mask_calzado & df["talla"].isna()
    if missing_shoes.any():
        sizes = np.array(CALZADO_TALLAS, dtype=int)
        probs = np.array([CALZADO_BASE_PROBS[t] for t in CALZADO_TALLAS], dtype=float)
        sampled = rng.choice(sizes, size=int(missing_shoes.sum()), p=probs)
        df.loc[missing_shoes, "talla"] = pd.Series(sampled, index=df.index[missing_shoes], dtype="string")

    if mask_calzado.any():
        df.loc[mask_calzado, "talla"] = (
            df.loc[mask_calzado, "talla"]
              .astype("string")
              .str.strip()
              .str.replace(r"\.0$", "", regex=True)
        )

    df["sku"] = df.apply(lambda r: update_sku(r["sku"], r["talla"]), axis=1)

    validate_sizes_and_sku(df)

    df_out = df.drop(columns=["cat_norm", "zona"], errors="ignore")

    if list(df_out.columns) != original_cols:
        raise ValueError("La estructura de columnas ha cambiado respecto al input.")

    if not invariants_snapshot.equals(df_out[invariant_cols]):
        raise ValueError("Se han modificado columnas que debían permanecer invariantes.")

    OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
    df_out.to_csv(OUTPUT_CSV, index=False)

    con = sqlite3.connect("database/mi_base.db")
    df_out.to_sql("items_3", con, if_exists="replace", index=False)
    con.close()


if __name__ == "__main__":
    main()


# items_4

In [27]:
INPUT_CSV = Path("data/items_tallas_ajustadas.csv")
OUTPUT_CSV = Path("data/items_colores_ajustados.csv")
RANDOM_SEED = 42


def strip_accents_lower(value: object) -> str:
    """
    Normaliza un texto a minúsculas sin acentos.
    Se usa para comparar categorías/provincias sin depender de tildes o mayúsculas.
    """
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return ""
    s = str(value).strip()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    return s.lower()


def find_column(df: pd.DataFrame, logical_name: str, candidates: Iterable[str], required: bool = True) -> Optional[str]:
    """
    Resuelve el nombre real de una columna a partir de un nombre lógico.
    Prueba coincidencia exacta, coincidencia case-insensitive y coincidencia por contención.
    """
    cols = list(df.columns)

    for c in candidates:
        if c in cols:
            return c

    lower_map = {c.lower(): c for c in cols}
    for c in candidates:
        hit = lower_map.get(str(c).lower())
        if hit is not None:
            return hit

    for col in cols:
        lc = col.lower()
        for cand in candidates:
            if str(cand).lower() in lc:
                return col

    if required:
        raise KeyError(f"No se encontró columna para '{logical_name}'. Columnas disponibles: {cols}")
    return None


def normalize_category(raw: object) -> str:
    """
    Reduce categorías a un set controlado para poder aplicar reglas estables de color.
    """
    base = strip_accents_lower(raw)

    if base.startswith("camiseta"):
        return "camiseta"
    if base.startswith("sudadera"):
        return "sudadera"
    if base.startswith("pantalon") or base.startswith("pantal"):
        return "pantalon"
    if base.startswith("abrigo"):
        return "abrigo"
    if base.startswith("camisa"):
        return "camisa"
    if base.startswith("calzado") or base.startswith("zapat") or "sneaker" in base:
        return "calzado"
    if "gorra" in base or "cap" in base:
        return "gorra"
    if "cinturon" in base or "cintur" in base:
        return "cinturon"
    if "calcetin" in base or "calcet" in base:
        return "calcetines"
    if "bufanda" in base or "scarf" in base:
        return "bufanda"

    return "otro"


CITY_PROVINCES = {
    "madrid", "barcelona", "valencia", "sevilla", "zaragoza",
    "malaga", "bilbao",
}
NORTH_RURAL_PROVINCES = {
    "a coruna", "la coruna", "coruna",
    "lugo", "ourense", "orense", "pontevedra",
    "asturias", "cantabria",
    "bizkaia", "vizcaya", "gipuzkoa", "guipuzcoa",
    "araba", "alava", "navarra", "la rioja", "rioja",
    "burgos", "leon", "palencia", "zamora", "soria",
    "huesca", "lleida", "lerida", "girona", "gerona",
}


def detect_zone(province_raw: object) -> str:
    """
    Clasifica provincia en un conjunto pequeño de zonas para introducir sesgos suaves.
    """
    p = strip_accents_lower(province_raw)
    if p in CITY_PROVINCES:
        return "ciudad"
    if p in NORTH_RURAL_PROVINCES:
        return "norte_rural"
    return "sur"


def season_bucket(dt: pd.Timestamp) -> str:
    """
    Etiqueta temporada por mes:
    SS para junio–septiembre, FW para diciembre–marzo y IN para el resto.
    """
    if pd.isna(dt):
        return "IN"
    m = int(dt.month)
    if 6 <= m <= 9:
        return "SS"
    if m in (12, 1, 2, 3):
        return "FW"
    return "IN"


COLOR_NAME: Dict[str, str] = {
    "BLK": "Negro",
    "GRY": "Gris",
    "NAV": "Azul marino",
    "RED": "Rojo",
    "LME": "Verde lima",
    "WHT": "Blanco",
    "BEI": "Beige",
    "BRN": "Marrón",
    "BLU": "Azul",
}
COLOR_TYPE: Dict[str, str] = {
    "BLK": "oscuro",
    "GRY": "oscuro",
    "NAV": "oscuro",
    "BRN": "oscuro",
    "RED": "vivo",
    "LME": "vivo",
    "BLU": "vivo",
    "WHT": "neutro",
    "BEI": "neutro",
}

COLOR_TYPES = ("oscuro", "vivo", "neutro")
CODES_BY_TYPE: Dict[str, np.ndarray] = {
    t: np.array([c for c, tt in COLOR_TYPE.items() if tt == t], dtype=object)
    for t in COLOR_TYPES
}

BASE_TYPE_PROBS: Dict[str, float] = {"oscuro": 0.45, "vivo": 0.35, "neutro": 0.20}

ZONE_TYPE_MULT: Dict[str, Dict[str, float]] = {
    "ciudad": {"oscuro": 1.0, "vivo": 1.0, "neutro": 1.0},
    "norte_rural": {"oscuro": 1.3, "vivo": 0.7, "neutro": 1.0},
    "sur": {"oscuro": 0.8, "vivo": 1.3, "neutro": 1.0},
}

SEASON_TYPE_MULT: Dict[str, Dict[str, float]] = {
    "SS": {"oscuro": 0.8, "vivo": 1.3, "neutro": 1.1},
    "FW": {"oscuro": 1.3, "vivo": 0.7, "neutro": 1.0},
    "IN": {"oscuro": 1.0, "vivo": 1.0, "neutro": 1.0},
}

CATEGORY_TYPE_MULT: Dict[str, Dict[str, float]] = {
    "camiseta": {"oscuro": 0.8, "vivo": 1.4, "neutro": 1.0},
    "abrigo": {"oscuro": 1.5, "vivo": 0.7, "neutro": 1.0},
    "calzado": {"oscuro": 1.4, "vivo": 0.8, "neutro": 1.0},
    "sudadera": {"oscuro": 1.2, "vivo": 1.0, "neutro": 0.9},
    "pantalon": {"oscuro": 1.0, "vivo": 0.9, "neutro": 1.2},
    "camisa": {"oscuro": 1.0, "vivo": 1.0, "neutro": 1.1},
}

SS_FAVORED = {"WHT", "LME", "RED", "BLU", "BEI", "NAV"}
SS_PENALIZED = {"BLK", "GRY", "BRN"}
FW_FAVORED = {"BLK", "GRY", "NAV", "BRN", "BEI"}
FW_PENALIZED = {"LME", "RED", "BLU", "WHT"}


def type_probs(zone: str, season: str, cat_norm: str) -> np.ndarray:
    """
    Construye probabilidades sobre tipos de color (oscuro/vivo/neutro)
    combinando una base global con multiplicadores de zona, temporada y categoría.
    """
    z = zone if zone in ZONE_TYPE_MULT else "sur"
    s = season if season in SEASON_TYPE_MULT else "IN"
    cat_mult = CATEGORY_TYPE_MULT.get(cat_norm)

    weights = []
    for t in COLOR_TYPES:
        w = BASE_TYPE_PROBS[t] * ZONE_TYPE_MULT[z][t] * SEASON_TYPE_MULT[s][t]
        if cat_mult is not None:
            w *= cat_mult[t]
        weights.append(w)

    w = np.array(weights, dtype=float)
    total = w.sum()
    if total <= 0:
        return np.array([1 / 3, 1 / 3, 1 / 3], dtype=float)
    return w / total


def pick_color_codes(df: pd.DataFrame, rng: np.random.Generator) -> pd.Series:
    """
    Genera un color por fila en dos pasos:
    primero se decide el tipo (oscuro/vivo/neutro) y después se elige un código dentro del tipo.
    """
    zones = df["zona"].astype(str).to_numpy()
    seasons = df["temporada"].astype(str).to_numpy()
    cats = df["cat_norm"].astype(str).to_numpy()

    out = np.empty(len(df), dtype=object)
    for i in range(len(df)):
        p = type_probs(zones[i], seasons[i], cats[i])
        t = rng.choice(np.array(COLOR_TYPES, dtype=object), p=p)
        out[i] = rng.choice(CODES_BY_TYPE[str(t)])
    return pd.Series(out, index=df.index, dtype="string")


def favored_flag(code: str, season: str) -> int:
    if season == "SS":
        return int(code in SS_FAVORED)
    if season == "FW":
        return int(code in FW_FAVORED)
    return 0


def mismatch_flag(code: str, season: str) -> int:
    if season == "SS":
        return int(code in SS_PENALIZED)
    if season == "FW":
        return int(code in FW_PENALIZED)
    return 0


def validate_invariants(before: pd.DataFrame, after: pd.DataFrame, invariant_cols: list[str]) -> None:
    """
    Comprueba que no se han modificado columnas fuera del alcance del generador de colores.
    """
    if list(before[invariant_cols].columns) != list(after[invariant_cols].columns):
        raise ValueError("La lista de columnas invariantes no coincide entre entrada y salida.")
    if not before[invariant_cols].equals(after[invariant_cols]):
        raise ValueError("Se han modificado columnas que debían permanecer invariantes.")


def validate_color_schema(df: pd.DataFrame, color_col: str) -> None:
    """
    Garantiza que el color resultante pertenece al diccionario de códigos soportados.
    """
    valid = set(COLOR_NAME.keys())
    bad = df.loc[~df[color_col].astype(str).isin(valid), color_col].value_counts(dropna=False).head(10)
    if not bad.empty:
        raise ValueError(f"Existen códigos de color fuera del catálogo. Ejemplos: {bad.to_dict()}")


def run() -> None:
    """
    Lee el dataset con tallas, asigna colores con reglas por contexto y exporta una versión final lista para análisis.
    """
    df = pd.read_csv(INPUT_CSV, low_memory=False)
    original_cols = list(df.columns)

    categoria_col = find_column(df, "categoria", ["categoria", "categoría", "category", "cat"])
    provincia_col = find_column(df, "provincia", ["provincia", "provincia_ticket", "province"])
    color_col = find_column(df, "color", ["color", "color_code", "cod_color", "color_producto"], required=False)
    fecha_col = find_column(df, "fecha", ["fecha_item", "fecha", "fecha_compra"], required=False)

    if color_col is None:
        raise KeyError("No se encontró una columna de color para sobrescribir. Añádela o pasa un nombre compatible.")

    invariant_cols = [c for c in original_cols if c != color_col]
    snapshot = df[invariant_cols].copy()

    df["cat_norm"] = df[categoria_col].map(normalize_category)
    df["zona"] = df[provincia_col].map(detect_zone)

    if fecha_col is not None:
        dt = pd.to_datetime(df[fecha_col], errors="coerce")
        df["temporada"] = dt.map(season_bucket)
    else:
        df["temporada"] = "IN"

    rng = np.random.default_rng(RANDOM_SEED)

    df[color_col] = pick_color_codes(df, rng)

    df["es_color_favorecido"] = [
        favored_flag(code, season) for code, season in zip(df[color_col].astype(str), df["temporada"].astype(str))
    ]
    df["color_mismatch"] = [
        mismatch_flag(code, season) for code, season in zip(df[color_col].astype(str), df["temporada"].astype(str))
    ]

    validate_color_schema(df, color_col)

    cleaned = df.drop(columns=["cat_norm", "zona", "temporada", "es_color_favorecido", "color_mismatch"], errors="ignore")
    if list(cleaned.columns) != original_cols:
        raise ValueError("La estructura de columnas ha cambiado respecto al input.")
    validate_invariants(snapshot, cleaned, invariant_cols)

    OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
    cleaned.to_csv(OUTPUT_CSV, index=False)

    con = sqlite3.connect("database/mi_base.db")
    cleaned.to_sql("items_4", con, if_exists="replace", index=False)
    con.close()

    print(f"OK: exportado {OUTPUT_CSV} (filas={len(cleaned):,}, cols={cleaned.shape[1]})")


if __name__ == "__main__":
    run()


OK: exportado data\items_colores_ajustados.csv (filas=905,445, cols=19)


# items_5

In [28]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import hashlib
import sqlite3
import unicodedata
from pathlib import Path

# ==========================
# RUTAS
# ==========================
INPUT_CSV  = Path("data/items_colores_ajustados.csv")
OUTPUT_CSV = Path("data/items_altura_peso_ajustados.csv")

# ==========================
# PARAMETROS (control del "ruido")
# ==========================
MISMATCH_RATE = 0.02   # % de clientes cuyo cuerpo no encaja con su talla ancla (muy pocos)
MISMATCH_P2   = 0.08   # dentro del mismatch, prob de salto de 2 tallas (el resto ±1)

# ==========================
# 1) CARGA
# ==========================
df = pd.read_csv(
    INPUT_CSV,
    parse_dates=["fecha_item"],
    dtype={
        "item_id": "string",
        "ticket_id": "string",
        "pos_item": "int64",
        "customer_id": "string",
        "canal": "string",
        "store_id": "string",
        "provincia": "string",
        "sku": "string",
        "id_producto": "string",
        "color": "string",
        "talla": "string",
        "promotion_id": "string",
    },
    low_memory=False
)
df.columns = df.columns.str.strip()

# Arreglo defensivo para variantes de 'categoria'
if "categoria" not in df.columns:
    for col in df.columns:
        if "categor" in col.lower():
            df = df.rename(columns={col: "categoria"})
            break
if "categoria" not in df.columns:
    raise ValueError("No se encontró columna 'categoria' en el CSV de entrada.")

original_cols = df.columns.tolist()

# ==========================
# 2) NORMALIZACION SOLO INTERNA (sin crear columnas en salida)
# ==========================
def _norm_text(x) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    s = str(x).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))  # quita acentos
    s = " ".join(s.split())
    return s

# categorías ropa (tu dataset viene en minúsculas: camiseta/abrigo/pantalon...)
ROPA_CATS_NORM = {"camiseta", "sudadera", "pantalon", "abrigo", "camisa"}

# tallas ropa válidas
TALLAS_ORDEN = ["XS", "S", "M", "L", "XL"]
TALLA_TO_IDX = {t: i for i, t in enumerate(TALLAS_ORDEN)}
IDX_TO_TALLA = {i: t for t, i in TALLA_TO_IDX.items()}

MAP_TALLA = {
    "xs": "XS",
    "s": "S",
    "m": "M",
    "l": "L",
    "xl": "XL",
    "xxl": "XL",
    "one size": "OneSize",
    "onesize": "OneSize",
    "talla unica": "OneSize",
    "unica": "OneSize",
}

# ==========================
# 3) RNG DETERMINISTA POR CLIENTE
# ==========================
def _rng_for_customer(customer_id: str) -> np.random.Generator:
    h = hashlib.md5(str(customer_id).encode("utf-8")).hexdigest()
    seed = int(h[:8], 16)
    return np.random.default_rng(seed)

def _trunc_normal(rng, low, high, mean, sd, max_tries=1000):
    for _ in range(max_tries):
        x = rng.normal(loc=mean, scale=sd)
        if low <= x <= high:
            return float(x)
    return float(np.clip(mean, low, high))

# ==========================
# 4) RANGOS REALISTAS POR TALLA (150–210) + BMI POR TALLA
#    (solapados a propósito)
# ==========================
ALTURA_RANGE = {
    "XS": (150, 165),
    "S":  (158, 172),
    "M":  (166, 180),
    "L":  (174, 188),
    "XL": (182, 210),
}
ALTURA_MEAN = {
    "XS": 158.0,
    "S":  165.0,
    "M":  173.0,
    "L":  181.0,
    "XL": 190.0,
}

# BMI ligeramente creciente por talla (no exagerado)
BMI_MEAN = {
    "XS": 21.5,
    "S":  23.0,
    "M":  24.5,
    "L":  26.0,
    "XL": 28.0,
}
BMI_SD = 2.6
BMI_MIN, BMI_MAX = 17.0, 40.0

PESO_MIN, PESO_MAX = 40.0, 160.0  # para evitar imposibles en extremos

# ==========================
# 5) TALLA ANCLA POR CLIENTE + GENERACION ALTURA/PESO/BMI
# ==========================
def _mode(series: pd.Series) -> str | None:
    s = series.dropna().astype("string")
    if len(s) == 0:
        return None
    vc = s.value_counts()
    if vc.empty:
        return None
    return str(vc.index[0])

def infer_fisico_por_cliente(group: pd.DataFrame) -> pd.Series:
    cid = group["customer_id"].iloc[0]
    rng = _rng_for_customer(cid)

    # normalización interna
    cat_norm = group["categoria"].map(_norm_text)
    talla_norm = group["talla"].map(_norm_text).map(MAP_TALLA).fillna(group["talla"].astype("string"))

    # talla ancla: moda de tallas XS..XL dentro de ropa
    mask_ropa = cat_norm.isin(ROPA_CATS_NORM) & talla_norm.isin(TALLAS_ORDEN)
    talla_anchor = _mode(talla_norm.loc[mask_ropa])

    # fallback si el cliente no tiene ropa con XS..XL (raro): usa moda global XS..XL, y si no, M
    if talla_anchor is None:
        talla_anchor = _mode(talla_norm.loc[talla_norm.isin(TALLAS_ORDEN)]) or "M"

    # mismatch muy pequeño: “cuerpo” desplazado respecto a su talla ancla
    idx = TALLA_TO_IDX[talla_anchor]
    if rng.random() < MISMATCH_RATE:
        step = 2 if rng.random() < MISMATCH_P2 else 1
        direction = -1 if rng.random() < 0.5 else 1
        idx = int(np.clip(idx + direction * step, 0, 4))

    talla_body = IDX_TO_TALLA[idx]

    # altura condicionada por talla_body
    h_low, h_high = ALTURA_RANGE[talla_body]
    h_mean = ALTURA_MEAN[talla_body]
    altura = _trunc_normal(rng, h_low, h_high, mean=h_mean, sd=(h_high - h_low) / 5.5)

    # bmi condicionado por talla_body
    bmi = _trunc_normal(rng, BMI_MIN, BMI_MAX, mean=BMI_MEAN[talla_body], sd=BMI_SD)

    # peso derivado (con caps)
    peso = bmi * (altura / 100.0) ** 2
    peso = float(np.clip(peso, PESO_MIN, PESO_MAX))

    # recalcular bmi consistente tras caps de peso
    bmi = float(peso / (altura / 100.0) ** 2)

    return pd.Series({
        "customer_id": cid,
        "altura_cm": round(altura, 1),
        "peso_kg": round(peso, 1),
        "bmi": round(bmi, 2),
    })

clientes_fisico = (
    df.groupby("customer_id", as_index=False)
      .apply(infer_fisico_por_cliente)
      .reset_index(drop=True)
)

# ==========================
# 6) MERGE
# ==========================
df_merged = df.merge(clientes_fisico, on="customer_id", how="left")

# ==========================
# 7) SALIDA: EXACTAMENTE MISMAS COLUMNAS + 3 NUEVAS (y colocadas)
#     Las insertamos justo detrás de 'talla'
# ==========================
for c in ["altura_cm", "peso_kg", "bmi"]:
    if c in original_cols:
        original_cols.remove(c)

if "talla" in original_cols:
    i = original_cols.index("talla") + 1
    final_cols = original_cols[:i] + ["altura_cm", "peso_kg", "bmi"] + original_cols[i:]
else:
    # si por lo que sea no existe 'talla', las ponemos al final
    final_cols = original_cols + ["altura_cm", "peso_kg", "bmi"]

df_out = df_merged.loc[:, final_cols]

df_out.to_csv(OUTPUT_CSV, index=False)
print("Tabla exportada correctamente:", OUTPUT_CSV)
print("Filas:", len(df_out))
print("Columnas:", df_out.columns.tolist())

# ==========================
# 8) SQLITE
# ==========================
con = sqlite3.connect("database/mi_base.db")
df_out.to_sql("items_5", con, if_exists="replace", index=False)
con.close()


C:\Users\PEDRO\AppData\Local\Temp\ipykernel_10188\1762878893.py:197: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(infer_fisico_por_cliente)


Tabla exportada correctamente: data\items_altura_peso_ajustados.csv
Filas: 905445
Columnas: ['item_id', 'ticket_id', 'pos_item', 'customer_id', 'canal', 'store_id', 'provincia', 'fecha_item', 'sku', 'id_producto', 'categoria', 'color', 'talla', 'altura_cm', 'peso_kg', 'bmi', 'pvp_unitario', 'descuento_pct', 'precio_neto_unit', 'coste_bruto', 'margen_unit', 'promotion_id']


# items_6

In [29]:
# -*- coding: utf-8 -*-
# =========================================================
# GENERADOR DEVOLUCIONES (BLOQUE ÚNICO, AUTOCONTENIDO)
# - No depende de variables previas del notebook (evita NameError)
# - FIT por producto SOLO para ropa (no calzado, no onesize)
# - SIGNO del fit correcto:
#     "talla pequeño" -> FIT_SHIFT_SMALL NEGATIVO (producto queda más pequeño)
#     "talla grande"  -> FIT_SHIFT_BIG   POSITIVO (producto queda más grande)
# - TALLA IDEAL "PERFECTA":
#     Ropa: por categoría usando ALTURA + BMI (pantalón pondera BMI; camiseta/abrigo altura)
#     Calzado: por altura (sin fit por producto)
# - Mantiene: gap canal, promo, descuento, precio, color, estacionalidad, BMI, historial, sesgo cliente
# =========================================================

import pandas as pd
import numpy as np
import hashlib
import sqlite3
import unicodedata
from pathlib import Path

# ==========================
# RUTAS
# ==========================
INPUT_CSV  = Path("data/items_altura_peso_ajustados.csv")
OUTPUT_CSV = Path("data/items_devoluciones_ajustadas.csv")

# ==========================
# CONTROL GAP CANAL (CLAVE)
# ==========================
PHYSICAL_MULT = 0.65
ONLINE_MULT   = 1.30
TARGET_MEAN   = 0.225  # escala global antes del historial

# ==========================
# FIT POR PRODUCTO (SOLO ROPA)
# ==========================
FIT_NORMAL_RATE = 0.78
FIT_SMALL_RATE  = 0.11
FIT_BIG_RATE    = 0.11

# ✅ SIGNO CORRECTO
FIT_SHIFT_SMALL = -0.60  # talla pequeño
FIT_SHIFT_BIG   = +0.60  # talla grande

# ==========================
# 1) CARGA
# ==========================
df = pd.read_csv(
    INPUT_CSV,
    parse_dates=["fecha_item"],
    dtype={
        "item_id": "string",
        "ticket_id": "string",
        "pos_item": "int64",
        "customer_id": "string",
        "canal": "string",
        "store_id": "string",
        "provincia": "string",
        "sku": "string",
        "id_producto": "string",
        "color": "string",
        "talla": "string",
        "promotion_id": "string",
    },
    low_memory=False
)
df.columns = df.columns.str.strip()

# Arreglo defensivo para variantes de 'categoria'
if "categoria" not in df.columns:
    for col in df.columns:
        if "categor" in col.lower():
            df = df.rename(columns={col: "categoria"})
            break
if "categoria" not in df.columns:
    raise ValueError("No se encontró columna 'categoria' en el CSV de entrada.")

# ==========================
# 2) NORMALIZACIÓN
# ==========================
def norm_text(x) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    s = str(x).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = " ".join(s.split())
    return s

MAP_TALLA = {
    "xs": "XS", "s": "S", "m": "M", "l": "L", "xl": "XL", "xxl": "XL",
    "one size": "OneSize", "onesize": "OneSize", "talla unica": "OneSize", "unica": "OneSize",
}

# ==========================
# 3) CONFIG
# ==========================
ROPA_CATS_NORM    = {"camiseta", "sudadera", "pantalon", "abrigo", "camisa"}
CALZADO_CAT_NORM  = "calzado"
ONESIZE_CATS_NORM = {"gorra", "cinturon", "calcetines", "bufanda"}

TALLAS_ROPA_ORDEN = ["XS", "S", "M", "L", "XL"]
talla_idx_ropa = {t: i for i, t in enumerate(TALLAS_ROPA_ORDEN)}
TALLAS_EXTREMAS = set(["XS", "XL", "39", "45"])

# Rangos calzado (para talla ideal de calzado)
CALZADO_RANGES = {
    39: (150, 168),
    40: (155, 172),
    41: (160, 176),
    42: (165, 182),
    43: (170, 188),
    44: (175, 195),
    45: (180, 210),
}

def rng_from_key(key: str) -> np.random.Generator:
    h = hashlib.md5(str(key).encode("utf-8")).hexdigest()
    seed = int(h[:8], 16)
    return np.random.default_rng(seed)

# ==========================
# 4) TALLA IDEAL "PERFECTA"
#    - Ropa: por categoría (altura + BMI)
#    - Calzado: por altura
# ==========================
clientes = (
    df.groupby("customer_id", as_index=False)[["altura_cm", "peso_kg"]]
      .first()
)

h_c = pd.to_numeric(clientes["altura_cm"], errors="coerce").to_numpy(dtype="float64")
w_c = pd.to_numeric(clientes["peso_kg"], errors="coerce").to_numpy(dtype="float64")

bmi_c = np.full(len(clientes), np.nan, dtype="float64")
mask_bmi_c = (~np.isnan(h_c)) & (~np.isnan(w_c)) & (h_c > 0)
bmi_c[mask_bmi_c] = w_c[mask_bmi_c] / (h_c[mask_bmi_c] / 100.0) ** 2
clientes["bmi_cliente"] = bmi_c

# Centros calibrados con tus stats por talla (los que ya viste en tu tabla)
H_CENTERS = np.array([158.0, 165.0, 173.0, 181.0, 190.3], dtype="float64")
B_CENTERS = np.array([21.70, 23.08, 24.49, 25.98, 27.94], dtype="float64")

SIGMA_H = 3.0
SIGMA_B = 2.6

# Pesos por categoría: para que PESO importe donde toca
W_H_DEFAULT = 0.65
W_B_DEFAULT = 0.35

W_H_PANTALON = 0.35
W_B_PANTALON = 0.65

W_H_CAMISA = 0.50
W_B_CAMISA = 0.50

def infer_talla_ideal_ropa_from_h_bmi(h: float, bmi: float, categoria_norm: str):
    if np.isnan(h) or np.isnan(bmi):
        return np.nan

    if categoria_norm == "pantalon":
        w_h, w_b = W_H_PANTALON, W_B_PANTALON
    elif categoria_norm == "camisa":
        w_h, w_b = W_H_CAMISA, W_B_CAMISA
    else:
        w_h, w_b = W_H_DEFAULT, W_B_DEFAULT

    z_h = (h - H_CENTERS) / SIGMA_H
    z_b = (bmi - B_CENTERS) / SIGMA_B
    score = w_h * (z_h ** 2) + w_b * (z_b ** 2)
    return TALLAS_ROPA_ORDEN[int(np.argmin(score))]

def infer_talla_ideal_calzado(h):
    if pd.isna(h):
        return np.nan
    best_talla, best_score = None, None
    for talla, (h_low, h_high) in CALZADO_RANGES.items():
        h_mid = (h_low + h_high) / 2
        score = (h - h_mid) ** 2
        if best_score is None or score < best_score:
            best_score = score
            best_talla = talla
    return str(best_talla) if best_talla is not None else np.nan

clientes["talla_ideal_calzado"] = [infer_talla_ideal_calzado(h) for h in clientes["altura_cm"]]

cust_altura = dict(zip(clientes["customer_id"], clientes["altura_cm"]))
cust_bmi    = dict(zip(clientes["customer_id"], clientes["bmi_cliente"]))
cust_talla_calzado = dict(zip(clientes["customer_id"], clientes["talla_ideal_calzado"]))

# ==========================
# 5) EFECTO CLIENTE
# ==========================
def customer_factor(customer_id: str) -> float:
    rng = rng_from_key("risk_" + str(customer_id))
    return float(rng.uniform(0.9, 1.2))

cust_factor = {cid: customer_factor(cid) for cid in clientes["customer_id"]}

# ==========================
# 6) EFECTO PRODUCTO
#    - FIT SOLO ROPA
#    - BIAS SIEMPRE
# ==========================
prod_cat = (
    df.groupby("id_producto", as_index=True)["categoria"]
      .first()
      .map(norm_text)
      .to_dict()
)

productos = df["id_producto"].astype("string").dropna().drop_duplicates().tolist()

def product_fit_shift(pid: str) -> float:
    c = prod_cat.get(pid, "")
    if c not in ROPA_CATS_NORM:
        return 0.0

    rng = rng_from_key("fit_prod_" + str(pid))
    u = rng.random()
    if u < FIT_NORMAL_RATE:
        return 0.0
    if u < FIT_NORMAL_RATE + FIT_SMALL_RATE:
        return FIT_SHIFT_SMALL
    return FIT_SHIFT_BIG

def product_bias(pid: str) -> float:
    rng = rng_from_key("bias_prod_" + str(pid))
    return float(np.clip(rng.normal(0.0, 0.02), -0.05, 0.08))

fit_shift_dict = {}
prod_bias_dict = {}
for pid in productos:
    fit_shift_dict[pid] = product_fit_shift(pid)
    prod_bias_dict[pid] = product_bias(pid)

# ==========================
# 7) ORDENAR
# ==========================
df = df.sort_values(["customer_id", "fecha_item", "ticket_id", "pos_item"]).reset_index(drop=True)

# ==========================
# 8) ARRAYS
# ==========================
cat_arr = df["categoria"].map(norm_text).to_numpy(dtype=object)
talla_arr = df["talla"].map(norm_text).map(MAP_TALLA).fillna(df["talla"].astype("string")).to_numpy(dtype=object)
canal_lower = df["canal"].fillna("").str.lower().to_numpy()
cust_ids_arr = df["customer_id"].astype("string").fillna("__MISSING__").to_numpy()
prod_ids_arr = df["id_producto"].astype("string").fillna("UNKNOWN_PROD").to_numpy()

n = len(df)

altura_arr = np.array([pd.to_numeric(cust_altura.get(cid, np.nan), errors="coerce") for cid in cust_ids_arr], dtype="float64")
bmi_cli_arr = np.array([pd.to_numeric(cust_bmi.get(cid, np.nan), errors="coerce") for cid in cust_ids_arr], dtype="float64")

# ✅ talla ideal de ropa por ITEM (depende de la categoría)
t_ideal_ropa_arr = np.array(
    [infer_talla_ideal_ropa_from_h_bmi(h, b, c) if c in ROPA_CATS_NORM else np.nan
     for h, b, c in zip(altura_arr, bmi_cli_arr, cat_arr)],
    dtype=object
)

# calzado ideal por cliente
t_ideal_calz_arr = np.array([cust_talla_calzado.get(cid, np.nan) for cid in cust_ids_arr], dtype=object)

fit_shift_arr = np.array([fit_shift_dict.get(pid, 0.0) for pid in prod_ids_arr], dtype="float64")
prod_bias_arr = np.array([prod_bias_dict.get(pid, 0.0) for pid in prod_ids_arr], dtype="float64")

m_fisico = np.isin(canal_lower, ["fisico", "físico", "tienda", "store"])
m_online = np.isin(canal_lower, ["online", "web", "ecommerce"])

m_ropa    = np.isin(cat_arr, list(ROPA_CATS_NORM))
m_calzado = (cat_arr == CALZADO_CAT_NORM)

# ==========================
# 9) DISTANCIAS
# ==========================
dist_ropa = np.full(n, np.nan, dtype="float64")
dist_calz = np.full(n, np.nan, dtype="float64")

# ropa: talla (XS..XL) vs ideal + fit producto (fit ya solo aplica a ropa)
idx_item_ropa  = np.array([talla_idx_ropa.get(str(t), np.nan) for t in talla_arr], dtype="float64")
idx_ideal_ropa = np.array([talla_idx_ropa.get(str(t), np.nan) for t in t_ideal_ropa_arr], dtype="float64")
mask_valid_ropa = m_ropa & ~np.isnan(idx_item_ropa) & ~np.isnan(idx_ideal_ropa)

dist_ropa[mask_valid_ropa] = (
    idx_item_ropa[mask_valid_ropa]
    - idx_ideal_ropa[mask_valid_ropa]
    + fit_shift_arr[mask_valid_ropa]
)

# calzado: SIN fit por producto
talla_num = pd.to_numeric(pd.Series(talla_arr), errors="coerce").to_numpy(dtype="float64")
talla_ideal_num = pd.to_numeric(pd.Series(t_ideal_calz_arr), errors="coerce").to_numpy(dtype="float64")
mask_valid_calz = m_calzado & ~np.isnan(talla_num) & ~np.isnan(talla_ideal_num)

dist_calz[mask_valid_calz] = (
    talla_num[mask_valid_calz]
    - talla_ideal_num[mask_valid_calz]
)

# ==========================
# 10) BASELINES
# ==========================
p_base = np.zeros(n, dtype="float64")

m_camiseta   = (cat_arr == "camiseta")
m_sudadera   = (cat_arr == "sudadera")
m_pantalon   = (cat_arr == "pantalon")
m_abrigo     = (cat_arr == "abrigo")
m_camisa     = (cat_arr == "camisa")
m_calzado_cat= (cat_arr == "calzado")

m_gorra      = (cat_arr == "gorra")
m_cinturon   = (cat_arr == "cinturon")
m_calcetines = (cat_arr == "calcetines")
m_bufanda    = (cat_arr == "bufanda")
m_onesize    = m_gorra | m_cinturon | m_calcetines | m_bufanda

p_base[m_camiseta]    = 0.17
p_base[m_sudadera]    = 0.18
p_base[m_pantalon]    = 0.22
p_base[m_abrigo]      = 0.21
p_base[m_camisa]      = 0.23
p_base[m_calzado_cat] = 0.19
p_base[m_onesize]     = 0.10

m_otros = ~(m_camiseta | m_sudadera | m_pantalon | m_abrigo | m_camisa | m_calzado_cat | m_onesize)
p_base[m_otros] = 0.16

# ==========================
# 10.a EFECTO TALLA (más suave en físico)
# ==========================
dist_ropa_abs = np.abs(dist_ropa)
dist_calz_abs = np.abs(dist_calz)

m_ropa_dist = m_ropa & ~np.isnan(dist_ropa)
m_calz_dist = m_calzado & ~np.isnan(dist_calz)

d = dist_ropa_abs.copy()
d[np.isnan(d)] = 0.0

m_r0 = m_ropa_dist & (d < 0.5)
m_r1 = m_ropa_dist & (d >= 0.5) & (d < 1.5)
m_r2 = m_ropa_dist & (d >= 1.5)

# online/general
p_base[m_r0 & ~m_fisico] *= 0.75
p_base[m_r1 & ~m_fisico] *= 1.40
p_base[m_r2 & ~m_fisico] *= 1.90

# físico
p_base[m_r0 & m_fisico] *= 0.90
p_base[m_r1 & m_fisico] *= 1.15
p_base[m_r2 & m_fisico] *= 1.50

dc = dist_calz_abs.copy()
dc[np.isnan(dc)] = 0.0

m_c0 = m_calz_dist & (dc < 0.5)
m_c1 = m_calz_dist & (dc >= 0.5) & (dc < 1.5)
m_c2 = m_calz_dist & (dc >= 1.5)

# online/general
p_base[m_c0 & ~m_fisico] *= 0.80
p_base[m_c1 & ~m_fisico] *= 1.45
p_base[m_c2 & ~m_fisico] *= 2.00

# físico
p_base[m_c0 & m_fisico] *= 0.90
p_base[m_c1 & m_fisico] *= 1.20
p_base[m_c2 & m_fisico] *= 1.60

# tallas extremas
m_extrema = np.isin(talla_arr, list(TALLAS_EXTREMAS))
p_base[m_extrema] *= 1.20

# sesgo por producto (siempre)
p_base += prod_bias_arr

# ==========================
# 10.b EFECTO CANAL (GAP)
# ==========================
p_base[m_fisico] *= PHYSICAL_MULT
p_base[m_online] *= ONLINE_MULT

# ==========================
# 10.c AJUSTES: DESCUENTO / PROMO / PRECIO / COLOR / ESTACIONALIDAD / BMI
# ==========================
color_code = df["color"].fillna("").str.upper().to_numpy()
descuento  = df["descuento_pct"].fillna(0).to_numpy(dtype="float64")
precio     = df["precio_neto_unit"].fillna(df.get("pvp_unitario", 0)).to_numpy(dtype="float64")

promo_id   = df["promotion_id"]
m_en_promo = promo_id.notna().to_numpy()

m_desc_medio = (descuento > 0.10) & (descuento <= 0.30)
m_desc_alto  = (descuento > 0.30)

p_base[m_desc_medio] *= 1.12
p_base[m_desc_alto]  *= 1.25
p_base[m_onesize & m_desc_alto] *= 1.15

p_base[m_en_promo] *= 1.18
p_base[m_online & m_en_promo] *= 1.10

precio_clipped = np.clip(precio, 0, 150.0)
precio_norm = precio_clipped / 100.0
p_base *= (1.0 + 0.18 * precio_norm)

p_base[(m_abrigo | m_calzado_cat) & (precio > 80)] *= 1.15

m_wht = (color_code == "WHT")
m_red = (color_code == "RED")
m_lme = (color_code == "LME")
m_blk = (color_code == "BLK")
m_nav = (color_code == "NAV")
m_gry = (color_code == "GRY")

p_base[m_wht] += 0.05
p_base[m_red | m_lme] += 0.03
p_base[m_blk | m_nav | m_gry] -= 0.01

mes = df["fecha_item"].dt.month.to_numpy()
m_enero     = (mes == 1)
m_febrero   = (mes == 2)
m_noviembre = (mes == 11)
m_julio     = (mes == 7)
m_agosto    = (mes == 8)

p_base[m_enero]     *= 1.20
p_base[m_febrero]   *= 1.05
p_base[m_noviembre] *= 1.08
p_base[m_julio]     *= 0.97
p_base[m_agosto]    *= 0.95

bmi_arr = pd.to_numeric(df["bmi"], errors="coerce").to_numpy(dtype="float64")
m_bmi_low  = (bmi_arr < 18.5)
m_bmi_high = (bmi_arr >= 30)
p_base[m_bmi_low | m_bmi_high] *= 1.08

# refuerzo por mismatch grande
p_base[m_ropa_dist & (d >= 1.5)]  *= 1.10
p_base[m_calz_dist & (dc >= 1.5)] *= 1.10

# ==========================
# 10.d NORMALIZACIÓN GLOBAL
# ==========================
current_mean = float(np.nanmean(p_base))
if current_mean > 0:
    scale = TARGET_MEAN / current_mean
    scale = np.clip(scale, 0.5, 1.5)
    p_base = p_base * scale

p_base = np.clip(p_base, 0.01, 0.80)

# ==========================
# 11) SIMULACIÓN DEVUELTOS (HISTORIAL)
# ==========================
cust_factor_arr = np.array([cust_factor.get(cid, 1.0) for cid in cust_ids_arr], dtype="float64")
devuelto = np.zeros(n, dtype="int8")

prev_cid     = None
prev_devs    = 0
prev_compras = 0
rng = None

for i in range(n):
    cid = cust_ids_arr[i]
    if cid != prev_cid:
        prev_cid = cid
        prev_devs = 0
        prev_compras = 0
        rng = rng_from_key("dev_" + str(cid))

    p = p_base[i] * cust_factor_arr[i]

    ratio_prev = (prev_devs / prev_compras) if prev_compras > 0 else 0.0
    p *= (1.0 + 0.6 * ratio_prev)

    if prev_compras >= 5 and prev_devs == 0:
        p *= 0.8

    if prev_devs > 0:
        p *= (1.0 + min(prev_devs * 0.10, 0.5))

    p += rng.normal(loc=0.0, scale=0.01)
    p = max(0.01, min(0.99, p))

    devuelto[i] = 1 if rng.random() < p else 0

    prev_compras += 1
    if devuelto[i] == 1:
        prev_devs += 1

df["devuelto"] = devuelto

# ==========================
# 12) EXPORT + LOGS
# ==========================
df.to_csv(OUTPUT_CSV, index=False)

print("✔ Tabla con devoluciones generada y guardada en:", OUTPUT_CSV)
print("  Filas:", len(df))
print("  % devoluciones global:", round(float(df["devuelto"].mean()), 4))

m_fisico_final = df["canal"].astype("string").str.lower().isin(["fisico", "físico", "tienda", "store"])
print("  % devoluciones online:", round(float(df.loc[~m_fisico_final, "devuelto"].mean()), 4))
print("  % devoluciones físico:", round(float(df.loc[m_fisico_final, "devuelto"].mean()), 4))

# ==========================
# 13) SQLITE
# ==========================
con = sqlite3.connect("database/mi_base.db")
df.to_sql("items_6", con, if_exists="replace", index=False)
con.close()


✔ Tabla con devoluciones generada y guardada en: data\items_devoluciones_ajustadas.csv
  Filas: 905445
  % devoluciones global: 0.2988
  % devoluciones online: 0.3196
  % devoluciones físico: 0.2344


# Devoluciones

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
import unicodedata

# ==========================
# RUTAS
# ==========================
SQLITE_DB   = Path("database/mi_base.db")
INPUT_TABLE = "items_6"

OUTPUT_CSV  = Path("data/devoluciones.csv")
OUTPUT_TABLE = "devoluciones"

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ==========================
# CONFIG
# ==========================
MAX_DIAS_DEVOLUCION = 30

# Coste base por categoría (zona estándar Z1)
COSTE_BASE_CAT = {
    "Camiseta": 1.96,
    "Camisa": 2.38,
    "Pantalón": 2.89,
    "Sudadera": 3.34,
    "Abrigo": 4.42,
    "Calzado": 3.98,
    "Gorra": 1.54,
    "Calcetines": 2.47,
    "Bufanda": 2.61,
    "Cinturón": 1.88,
}

# Recargo por zona (online)
RECARGO_ZONA = {
    "Z1": 0.00,
    "Z2": 0.42,
    "Z3": 0.93,
    "Z4": 1.47,
    "Z5": 2.08,
}

# Ajuste canal físico
RECARGO_FISICO_FACTOR = 0.15
AJUSTE_FISICO_FIJO    = -0.55

# Ruido del coste (euros)
COSTE_RUIDO_SIGMA = 0.12

# ==========================
# PROVINCIAS -> ZONA
# ==========================
PROV_TO_ZONA = {
    # Z1
    "madrid": "Z1",
    "barcelona": "Z1",
    "valencia": "Z1",
    "sevilla": "Z1",
    "bizkaia": "Z1",
    "gipuzkoa": "Z1",
    "zaragoza": "Z1",
    "malaga": "Z1",

    # Z2
    "alicante": "Z2",
    "almeria": "Z2",
    "asturias": "Z2",
    "cadiz": "Z2",
    "cantabria": "Z2",
    "castellon": "Z2",
    "cordoba": "Z2",
    "girona": "Z2",
    "granada": "Z2",
    "huelva": "Z2",
    "jaen": "Z2",
    "la rioja": "Z2",
    "leon": "Z2",
    "lleida": "Z2",
    "murcia": "Z2",
    "navarra": "Z2",
    "pontevedra": "Z2",
    "tarragona": "Z2",
    "toledo": "Z2",
    "valladolid": "Z2",

    # Z3
    "a coruna": "Z3",
    "alava": "Z3",
    "albacete": "Z3",
    "avila": "Z3",
    "badajoz": "Z3",
    "burgos": "Z3",
    "caceres": "Z3",
    "ciudad real": "Z3",
    "cuenca": "Z3",
    "guadalajara": "Z3",
    "huesca": "Z3",
    "lugo": "Z3",
    "ourense": "Z3",
    "palencia": "Z3",
    "salamanca": "Z3",
    "segovia": "Z3",
    "soria": "Z3",
    "teruel": "Z3",
    "zamora": "Z3",

    # Z4
    "islas baleares": "Z4",

    # Z5
    "las palmas": "Z5",
    "santa cruz de tenerife": "Z5",
    "ceuta": "Z5",
    "melilla": "Z5",
}

# ==========================
# DEVOLUCIÓN: DISTRIBUCIÓN POR CANAL
# ==========================
ONLINE_TRAMOS = [
    ((1, 2),  0.05),
    ((3, 7),  0.40),
    ((8, 14), 0.30),
    ((15, 21),0.15),
    ((22, 30),0.10),
]

FISICO_TRAMOS = [
    ((0, 1),  0.25),
    ((2, 5),  0.40),
    ((6, 10), 0.20),
    ((11, 20),0.10),
    ((21, 30),0.05),
]

def _norm_text(x: str) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    s = str(x).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = " ".join(s.split())
    return s

def _sample_days_from_tramos(tramos, n, prob_jitter=0.04):
    ranges = [r for r, _ in tramos]
    probs  = np.array([p for _, p in tramos], dtype=float)

    noise = np.random.normal(0.0, prob_jitter, size=len(probs))
    probs = np.clip(probs + noise, 0.001, None)
    probs = probs / probs.sum()

    idx = np.random.choice(len(ranges), size=n, p=probs)

    days = np.empty(n, dtype=int)
    for i, (a, b) in enumerate(ranges):
        mask = (idx == i)
        k = int(mask.sum())
        if k == 0:
            continue

        mode = a + max(0, int(round(0.2 * (b - a))))
        sampled = np.random.triangular(left=a, mode=mode, right=b, size=k)
        days[mask] = np.rint(sampled).astype(int)

    return np.clip(days, 0, MAX_DIAS_DEVOLUCION)

def sample_dias_hasta_devolucion(canal_norm: str, n: int) -> np.ndarray:
    if canal_norm in {"fisico", "físico", "tienda", "store"}:
        return _sample_days_from_tramos(FISICO_TRAMOS, n)
    return _sample_days_from_tramos(ONLINE_TRAMOS, n)

def normalizar_categoria(cat: str) -> str:
    c = str(cat).strip()
    if c == "Pantalon":
        return "Pantalón"
    if c == "Cinturon":
        return "Cinturón"
    return c

def es_fisico(canal_norm: str) -> bool:
    return canal_norm in {"fisico", "físico", "tienda", "store"}

# ==========================
# 1) CARGA DESDE SQLITE
# ==========================
con = sqlite3.connect(SQLITE_DB)
df = pd.read_sql_query(f"SELECT * FROM {INPUT_TABLE}", con)
con.close()

df.columns = df.columns.str.strip()

if "devuelto" not in df.columns:
    raise ValueError("La tabla de entrada no tiene la columna 'devuelto'.")

if "fecha_item" not in df.columns:
    raise ValueError("La tabla de entrada no tiene la columna 'fecha_item'.")

df["fecha_item"] = pd.to_datetime(df["fecha_item"], errors="coerce")

# ==========================
# 2) FILTRAR SOLO DEVOLUCIONES
# ==========================
dev = df.loc[df["devuelto"].astype(int) == 1].copy()
dev.reset_index(drop=True, inplace=True)

# ==========================
# 3) NORMALIZACIONES BÁSICAS
# ==========================
dev["provincia_norm"] = dev["provincia"].map(_norm_text)
dev["canal_norm"]     = dev["canal"].map(_norm_text)

dev["categoria"] = dev["categoria"].apply(normalizar_categoria)

# ==========================
# 4) ZONA LOGÍSTICA
# ==========================
dev["zona_logistica"] = dev["provincia_norm"].map(PROV_TO_ZONA).fillna("Z2")

# ==========================
# 5) DÍAS + FECHA DEVOLUCIÓN
# ==========================
# sample por fila (canal puede variar)
dias = np.empty(len(dev), dtype=int)
for i, c in enumerate(dev["canal_norm"].values):
    dias[i] = int(sample_dias_hasta_devolucion(c, 1)[0])

dev["dias_hasta_devolucion"] = dias
dev["fecha_devolucion"] = dev["fecha_item"] + pd.to_timedelta(dev["dias_hasta_devolucion"], unit="D")

# ==========================
# 6) COSTE DEVOLUCIÓN
# ==========================
base = dev["categoria"].map(COSTE_BASE_CAT).fillna(2.80).astype(float)

recargo = dev["zona_logistica"].map(RECARGO_ZONA).fillna(0.42).astype(float)

m_fis = dev["canal_norm"].apply(es_fisico).to_numpy(dtype=bool)

recargo_aj = recargo.copy()
recargo_aj[m_fis] = recargo_aj[m_fis] * RECARGO_FISICO_FACTOR

ajuste = np.zeros(len(dev), dtype=float)
ajuste[m_fis] = AJUSTE_FISICO_FIJO

ruido = np.random.normal(0.0, COSTE_RUIDO_SIGMA, size=len(dev))

coste = base.to_numpy() + recargo_aj.to_numpy() + ajuste + ruido
coste = np.clip(coste, 0.50, None)

dev["coste_devolucion"] = np.round(coste, 2)

# ==========================
# 7) SELECCIÓN DE COLUMNAS (solo devoluciones)
# ==========================
cols_out = [
    "item_id","ticket_id","pos_item","customer_id","canal","store_id","provincia","fecha_item",
    "sku","id_producto","categoria","color","talla",
    "pvp_unitario","descuento_pct","precio_neto_unit",
    "coste_bruto","margen_unit","promotion_id",
    "altura_cm","peso_kg",
    "zona_logistica","dias_hasta_devolucion","fecha_devolucion","coste_devolucion"
]

cols_out = [c for c in cols_out if c in dev.columns]
out = dev[cols_out].copy()

# ==========================
# 8) EXPORTAR CSV + SQLITE
# ==========================
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUTPUT_CSV, index=False)

con = sqlite3.connect(SQLITE_DB)
out.to_sql(OUTPUT_TABLE, con, if_exists="replace", index=False)
con.close()

print("✔ Tabla 'devoluciones' generada.")
print("  CSV:", OUTPUT_CSV)
print("  SQLite:", SQLITE_DB, "| tabla:", OUTPUT_TABLE)
print("  Filas:", len(out))
print("  % canal físico:", (out["canal"].astype(str).str.lower().isin(["fisico","físico","tienda","store"]).mean()).round(4))
print("  Rango fecha_compra:", out["fecha_item"].min(), "→", out["fecha_item"].max())
print("  Rango fecha_devolucion:", out["fecha_devolucion"].min(), "→", out["fecha_devolucion"].max())


✔ Tabla 'devoluciones' generada.
  CSV: data\devoluciones.csv
  SQLite: database\mi_base.db | tabla: devoluciones
  Filas: 270552
  % canal físico: 0.1914
  Rango fecha_compra: 2017-08-01 00:00:00 → 2025-09-30 00:00:00
  Rango fecha_devolucion: 2017-08-03 00:00:00 → 2025-10-28 00:00:00


# nuevos añadimos devoluiones como columna

In [30]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
import unicodedata

# ==========================
# RUTAS / NOMBRES (como quieres)
# ==========================
SQLITE_DB   = Path("database/mi_base.db")
TABLE_NAME  = "items_6"
OUTPUT_CSV  = Path("data/items_devoluciones_ajustadas.csv")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ==========================
# CONFIG (igual que tu script)
# ==========================
MAX_DIAS_DEVOLUCION = 30

COSTE_BASE_CAT = {
    "Camiseta": 1.96,
    "Camisa": 2.38,
    "Pantalón": 2.89,
    "Sudadera": 3.34,
    "Abrigo": 4.42,
    "Calzado": 3.98,
    "Gorra": 1.54,
    "Calcetines": 2.47,
    "Bufanda": 2.61,
    "Cinturón": 1.88,
}

RECARGO_ZONA = {
    "Z1": 0.00,
    "Z2": 0.42,
    "Z3": 0.93,
    "Z4": 1.47,
    "Z5": 2.08,
}

RECARGO_FISICO_FACTOR = 0.15
AJUSTE_FISICO_FIJO    = -0.55
COSTE_RUIDO_SIGMA     = 0.12

PROV_TO_ZONA = {
    "madrid": "Z1", "barcelona": "Z1", "valencia": "Z1", "sevilla": "Z1",
    "bizkaia": "Z1", "gipuzkoa": "Z1", "zaragoza": "Z1", "malaga": "Z1",

    "alicante": "Z2","almeria": "Z2","asturias": "Z2","cadiz": "Z2","cantabria": "Z2",
    "castellon": "Z2","cordoba": "Z2","girona": "Z2","granada": "Z2","huelva": "Z2",
    "jaen": "Z2","la rioja": "Z2","leon": "Z2","lleida": "Z2","murcia": "Z2",
    "navarra": "Z2","pontevedra": "Z2","tarragona": "Z2","toledo": "Z2","valladolid": "Z2",

    "a coruna": "Z3","alava": "Z3","albacete": "Z3","avila": "Z3","badajoz": "Z3",
    "burgos": "Z3","caceres": "Z3","ciudad real": "Z3","cuenca": "Z3","guadalajara": "Z3",
    "huesca": "Z3","lugo": "Z3","ourense": "Z3","palencia": "Z3","salamanca": "Z3",
    "segovia": "Z3","soria": "Z3","teruel": "Z3","zamora": "Z3",

    "islas baleares": "Z4",

    "las palmas": "Z5","santa cruz de tenerife": "Z5","ceuta": "Z5","melilla": "Z5",
}

ONLINE_TRAMOS = [
    ((1, 2),  0.05),
    ((3, 7),  0.40),
    ((8, 14), 0.30),
    ((15, 21),0.15),
    ((22, 30),0.10),
]

FISICO_TRAMOS = [
    ((0, 1),  0.25),
    ((2, 5),  0.40),
    ((6, 10), 0.20),
    ((11, 20),0.10),
    ((21, 30),0.05),
]

# ==========================
# HELPERS (igual que tu lógica)
# ==========================
def _norm_text(x: str) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    s = str(x).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = " ".join(s.split())
    return s

def _sample_days_from_tramos(tramos, n, prob_jitter=0.04):
    ranges = [r for r, _ in tramos]
    probs  = np.array([p for _, p in tramos], dtype=float)

    noise = np.random.normal(0.0, prob_jitter, size=len(probs))
    probs = np.clip(probs + noise, 0.001, None)
    probs = probs / probs.sum()

    idx = np.random.choice(len(ranges), size=n, p=probs)

    days = np.empty(n, dtype=int)
    for i, (a, b) in enumerate(ranges):
        mask = (idx == i)
        k = int(mask.sum())
        if k == 0:
            continue
        mode = a + max(0, int(round(0.2 * (b - a))))
        sampled = np.random.triangular(left=a, mode=mode, right=b, size=k)
        days[mask] = np.rint(sampled).astype(int)

    return np.clip(days, 0, MAX_DIAS_DEVOLUCION)

def sample_dias_hasta_devolucion(canal_norm: str, n: int) -> np.ndarray:
    if canal_norm in {"fisico", "físico", "tienda", "store"}:
        return _sample_days_from_tramos(FISICO_TRAMOS, n)
    return _sample_days_from_tramos(ONLINE_TRAMOS, n)

def normalizar_categoria(cat: str) -> str:
    c = str(cat).strip()
    if c == "Pantalon":
        return "Pantalón"
    if c == "Cinturon":
        return "Cinturón"
    return c

def es_fisico(canal_norm: str) -> bool:
    return canal_norm in {"fisico", "físico", "tienda", "store"}

# ==========================
# 1) CARGA items_6
# ==========================
con = sqlite3.connect(SQLITE_DB)
df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME}", con)
con.close()

df.columns = df.columns.str.strip()

# checks mínimos
for col in ["devuelto", "fecha_item", "categoria", "provincia", "canal"]:
    if col not in df.columns:
        raise ValueError(f"Falta columna obligatoria en items_6: {col}")

df["fecha_item"] = pd.to_datetime(df["fecha_item"], errors="coerce")
df["devuelto"]   = pd.to_numeric(df["devuelto"], errors="coerce").fillna(0).astype(int)

# ==========================
# 2) NORMALIZACIONES + ZONA (para TODOS)
# ==========================
df["provincia_norm"] = df["provincia"].map(_norm_text)
df["canal_norm"]     = df["canal"].map(_norm_text)
df["categoria"]      = df["categoria"].apply(normalizar_categoria)

df["zona_logistica"] = df["provincia_norm"].map(PROV_TO_ZONA).fillna("Z2")

# ==========================
# 3) COSTE DEVOLUCIÓN (para TODOS, hipotético)
#     - NO depende de devuelto
# ==========================
base = df["categoria"].map(COSTE_BASE_CAT).fillna(2.80).astype(float)
recargo = df["zona_logistica"].map(RECARGO_ZONA).fillna(0.42).astype(float)

m_fis = df["canal_norm"].apply(es_fisico).to_numpy(dtype=bool)

recargo_aj = recargo.copy()
recargo_aj[m_fis] = recargo_aj[m_fis] * RECARGO_FISICO_FACTOR

ajuste = np.zeros(len(df), dtype=float)
ajuste[m_fis] = AJUSTE_FISICO_FIJO

ruido = np.random.normal(0.0, COSTE_RUIDO_SIGMA, size=len(df))

coste = base.to_numpy() + recargo_aj.to_numpy() + ajuste + ruido
coste = np.clip(coste, 0.50, None)

df["coste_devolucion"] = np.round(coste, 2)

# ==========================
# 4) DÍAS + FECHA DEVOLUCIÓN (SOLO devuelto==1)
# ==========================
df["dias_hasta_devolucion"] = np.nan
df["fecha_devolucion"]      = pd.NaT

mask_dev = df["devuelto"] == 1
n_dev = int(mask_dev.sum())

if n_dev > 0:
    dias = np.empty(n_dev, dtype=int)
    canales = df.loc[mask_dev, "canal_norm"].values
    for i, c in enumerate(canales):
        dias[i] = int(sample_dias_hasta_devolucion(c, 1)[0])

    df.loc[mask_dev, "dias_hasta_devolucion"] = dias
    df.loc[mask_dev, "fecha_devolucion"] = df.loc[mask_dev, "fecha_item"] + pd.to_timedelta(dias, unit="D")

# Tipos finales consistentes
df["dias_hasta_devolucion"] = pd.to_numeric(df["dias_hasta_devolucion"], errors="coerce")

# ==========================
# 5) EXPORT CSV (nombre exacto)
# ==========================
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUTPUT_CSV, index=False)

# ==========================
# 6) ESCRIBIR EN SQLITE EN LA MISMA TABLA items_6 (manteniendo el nombre)
#     - Creamos una tabla temporal y renombramos
# ==========================
con = sqlite3.connect(SQLITE_DB)
cur = con.cursor()

tmp = "items_6_tmp_rebuild"

cur.execute(f"DROP TABLE IF EXISTS {tmp};")
con.commit()

df.to_sql(tmp, con, if_exists="replace", index=False)

cur.execute(f"DROP TABLE IF EXISTS {TABLE_NAME};")
cur.execute(f"ALTER TABLE {tmp} RENAME TO {TABLE_NAME};")
con.commit()
con.close()

print("✔ OK: CSV generado:", OUTPUT_CSV)
print("✔ OK: SQLite actualizado en tabla:", SQLITE_DB, "| tabla:", TABLE_NAME)
print("Filas:", len(df))
print("Devueltos:", int((df['devuelto']==1).sum()))
print("NAs dias_hasta_devolucion (deberían ser = no devueltos):", int(df["dias_hasta_devolucion"].isna().sum()))
print("NAs fecha_devolucion (deberían ser = no devueltos):", int(df["fecha_devolucion"].isna().sum()))
print("NAs coste_devolucion (deberían ser 0):", int(df["coste_devolucion"].isna().sum()))


✔ OK: CSV generado: data\items_devoluciones_ajustadas.csv
✔ OK: SQLite actualizado en tabla: database\mi_base.db | tabla: items_6
Filas: 905445
Devueltos: 270552
NAs dias_hasta_devolucion (deberían ser = no devueltos): 634893
NAs fecha_devolucion (deberían ser = no devueltos): 634893
NAs coste_devolucion (deberían ser 0): 0


In [4]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path

SQLITE_DB = Path("database/mi_base.db")
TABLE = "items_6"

con = sqlite3.connect(SQLITE_DB)
df = pd.read_sql_query(f"SELECT * FROM {TABLE}", con)
con.close()

print("Shape:", df.shape)
print("Columnas nuevas presentes:", all(c in df.columns for c in [
    "zona_logistica","dias_hasta_devolucion","fecha_devolucion","coste_devolucion"
]))

# 1) coste_devolucion debe estar 100% completo
pct_cost_na = df["coste_devolucion"].isna().mean()
print("Pct NA coste_devolucion:", pct_cost_na)

# 2) dias/fecha SOLO en devueltos
mask_dev = df["devuelto"].astype(int) == 1
mask_nodev = ~mask_dev

dias_na_dev = df.loc[mask_dev, "dias_hasta_devolucion"].isna().mean() if mask_dev.any() else np.nan
dias_notna_nodev = df.loc[mask_nodev, "dias_hasta_devolucion"].notna().mean()

fec_na_dev = df.loc[mask_dev, "fecha_devolucion"].isna().mean() if mask_dev.any() else np.nan
fec_notna_nodev = df.loc[mask_nodev, "fecha_devolucion"].notna().mean()

print("Pct NA dias en devueltos (debería ser 0):", dias_na_dev)
print("Pct NO-NA dias en NO devueltos (debería ser 0):", dias_notna_nodev)
print("Pct NA fecha_devolucion en devueltos (debería ser 0):", fec_na_dev)
print("Pct NO-NA fecha_devolucion en NO devueltos (debería ser 0):", fec_notna_nodev)

# 3) consistencia fecha_item + dias = fecha_devolucion (solo devueltos)
df["fecha_item_dt"] = pd.to_datetime(df["fecha_item"], errors="coerce")
df["fecha_devolucion_dt"] = pd.to_datetime(df["fecha_devolucion"], errors="coerce")

dev = df.loc[mask_dev].copy()
dev["dias_hasta_devolucion"] = pd.to_numeric(dev["dias_hasta_devolucion"], errors="coerce")

calc = dev["fecha_item_dt"] + pd.to_timedelta(dev["dias_hasta_devolucion"], unit="D")
ok = (calc.dt.date == dev["fecha_devolucion_dt"].dt.date)

print("Check fecha_item + dias == fecha_devolucion (devueltos):",
      f"{ok.mean():.4f} OK | fallos:", int((~ok).sum()))

# 4) rangos básicos
print("\nRangos:")
print("coste_devolucion min/max:", float(df["coste_devolucion"].min()), float(df["coste_devolucion"].max()))
print("dias_hasta_devolucion min/max (devueltos):",
      float(dev["dias_hasta_devolucion"].min()), float(dev["dias_hasta_devolucion"].max()))

# 5) sanity: devueltos y no devueltos
print("\nDevueltos:", int(mask_dev.sum()), "| No devueltos:", int(mask_nodev.sum()))


Shape: (905445, 29)
Columnas nuevas presentes: True
Pct NA coste_devolucion: 0.0
Pct NA dias en devueltos (debería ser 0): 0.0
Pct NO-NA dias en NO devueltos (debería ser 0): 0.0
Pct NA fecha_devolucion en devueltos (debería ser 0): 0.0
Pct NO-NA fecha_devolucion en NO devueltos (debería ser 0): 0.0
Check fecha_item + dias == fecha_devolucion (devueltos): 1.0000 OK | fallos: 0

Rangos:
coste_devolucion min/max: 1.73 5.35
dias_hasta_devolucion min/max (devueltos): 0.0 30.0

Devueltos: 270642 | No devueltos: 634803
